<a href="https://colab.research.google.com/github/Tar-ive/Dashboard/blob/main/krishna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection + Pre Processing


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/texas_state_nsf_grants.csv')

In [ ]:
# prompt: make a single list of all the names from the  principal_investigator	co_principal_investigators columns, the co_principle_investigators column have names separated by ; delimitor.

import pandas as pd
 # Replace 'your_file.csv' with the actual file name

all_names = []
for index, row in df.iterrows():
  all_names.append(row['principal_investigator'])
  co_pis = row['co_principal_investigators']
  if pd.notnull(co_pis):
    all_names.extend([name.strip() for name in co_pis.split(';')])

all_names


In [ ]:
# prompt: make a  df  of  only these 2 columns.

relevant_df = df[['principal_investigator', 'co_principal_investigators']]
relevant_df

In [ ]:
import requests
import pandas as pd
import time
from urllib.parse import quote

# List of researchers
researchers = [
    "Alice Olmstead", "Apan Qasem", "Araceli Martinez Ortiz", "Barbara Hewitt",
    "Byoung-Hee You", "Carolyn Chang", "Chiu Au", "Christopher Rhodes",
    "Chul-Ho Lee", "Cindy Royal", "Clara Novoa", "Craig Hanks",
    "Cynthia Luxford", "Damian Valles Molina", "Danny Wescott", "David Gibbs",
    "Denise Gobert", "Diane Dolozel", "Dincer Konur", "Dominick Fazarro",
    "Eduardo Perez", "Edward Yu", "Edwin Chow", "Edwin Piner",
    "Eleanor Close", "Emanuel Alanis", "Emily Zhu", "Erica Nason",
    "Eunsang Cho", "Feng Wang", "Francis Mendez", "Gabriel Lopez",
    "Gregory Lakomski", "Heather Galloway", "Holly Lewis", "Holly Veselka",
    "Hsing-Huang Tseng", "Hyunhwan Kim", "In-Hyouk Song", "Ivan Ojeda-Ruiz",
    "Jana Minifie", "Jelena Tesic", "Jennifer Irvin", "Jie Zhu",
    "Jitendra Tate", "Karen Lewis", "Keshav Bhandari", "Kimberly Talley",
    "Larry Price", "Li Feng", "Lucia Summers", "Maria Resendiz",
    "Mario Cristian Gaedicke-Hornung", "Mike Ferrara", "Mina Guirguis",
    "Monica Hughes", "Mylene Farias", "Nadim Adi", "Nicole Taylor",
    "Nikoleta Theodoropoulou", "Rasim Musal", "Ravindranath Droopad",
    "Sarah Fritts", "Satyajit Dutta", "Sean Bauld", "Semih Aslan",
    "Shannon Williams", "Stefan Zauscher", "Steven Whitten", "Subasish Das",
    "Tahir Ekin", "Tania Betancourt", "Thomas Keller", "Togay Ozbakkaloglu",
    "Tongdan Jin", "Toni Watt", "Ty Schepis", "Vangelis Metsis",
    "Vishan Shen", "Wenquan Dong", "Wilhelmus Geerts", "William Brittain",
    "Xiangping Liu", "Xiaoxi Shen", "Yaroslava Yingling", "Yihong 'Maggie' Chen",
    "Yihong Yuan", "Yong Yang", "Young Ju Lee", "Ziliang Zong"
]

# Function to search OpenAlex for a researcher
def search_openalex(name):
    """Search OpenAlex API for researcher and return results"""
    try:
        # URL encode the name for the API call
        encoded_name = quote(name)
        url = f"https://api.openalex.org/authors?search={encoded_name}"

        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        return data.get('results', [])

    except Exception as e:
        print(f"Error searching for {name}: {e}")
        return []

# Function to check if researcher is affiliated with Texas State
def check_texas_state_affiliation(author_data):
    """Check if author has Texas State affiliation"""
    affiliations = author_data.get('affiliations', [])

    for affiliation in affiliations:
        institution = affiliation.get('institution', {})
        display_name = institution.get('display_name', '').lower()

        # Check for Texas State University variations
        if any(term in display_name for term in ['texas state', 'texas state university']):
            return True, institution.get('display_name', 'Texas State University')

    return False, None

# Main processing
results = []

print("Starting OpenAlex lookup for 90 researchers...")
print("=" * 50)

for i, name in enumerate(researchers, 1):
    print(f"Processing {i}/90: {name}")

    # Search OpenAlex
    search_results = search_openalex(name)

    if not search_results:
        results.append({
            'name': name,
            'openalex_id': 'NOT_FOUND',
            'texas_state_affiliated': False,
            'institution_name': '',
            'display_name': '',
            'works_count': 0,
            'cited_by_count': 0
        })
        print(f"  → No results found")
        continue

    # Check each result for Texas State affiliation
    found_texas_state = False

    for author in search_results:
        is_texas_state, institution_name = check_texas_state_affiliation(author)

        if is_texas_state:
            results.append({
                'name': name,
                'openalex_id': author.get('id', ''),
                'texas_state_affiliated': True,
                'institution_name': institution_name,
                'display_name': author.get('display_name', ''),
                'works_count': author.get('works_count', 0),
                'cited_by_count': author.get('cited_by_count', 0)
            })
            print(f"  ✓ Found at Texas State: {author.get('display_name', name)}")
            found_texas_state = True
            break

    if not found_texas_state:
        # Take the first result even if not at Texas State
        first_result = search_results[0]
        results.append({
            'name': name,
            'openalex_id': first_result.get('id', ''),
            'texas_state_affiliated': False,
            'institution_name': '',
            'display_name': first_result.get('display_name', ''),
            'works_count': first_result.get('works_count', 0),
            'cited_by_count': first_result.get('cited_by_count', 0)
        })
        print(f"  ⚠ Found but not at Texas State: {first_result.get('display_name', name)}")

    # Small delay to be respectful to the API
    time.sleep(0.1)

# Create DataFrame and save to CSV
df = pd.DataFrame(results)

# Summary statistics
total_found = len(df[df['openalex_id'] != 'NOT_FOUND'])
texas_state_count = len(df[df['texas_state_affiliated'] == True])
not_found_count = len(df[df['openalex_id'] == 'NOT_FOUND'])

print("\n" + "=" * 50)
print("SUMMARY:")
print(f"Total researchers: {len(researchers)}")
print(f"Found in OpenAlex: {total_found}")
print(f"Confirmed at Texas State: {texas_state_count}")
print(f"Not found: {not_found_count}")

# Save to CSV
csv_filename = "texas_state_researchers_openalex.csv"
df.to_csv(csv_filename, index=False)
print(f"\nResults saved to: {csv_filename}")

# Display first few results
print(f"\nFirst 5 results:")
print(df.head().to_string(index=False))

# Show researchers not found
if not_found_count > 0:
    print(f"\nResearchers not found in OpenAlex:")
    not_found = df[df['openalex_id'] == 'NOT_FOUND']['name'].tolist()
    for name in not_found:
        print(f"  - {name}")

# Show researchers found but not at Texas State
non_texas_state = df[(df['texas_state_affiliated'] == False) & (df['openalex_id'] != 'NOT_FOUND')]
if len(non_texas_state) > 0:
    print(f"\nResearchers found but not affiliated with Texas State:")
    for _, row in non_texas_state.iterrows():
        print(f"  - {row['name']}: {row['display_name']}")

hardcoding these now for testing purposes but any relevent data scientist could patch this up with real data


In [ ]:
df.columns

In [ ]:
# prompt: drop columns _meta', '_captcha', '_login', '_error',

# Assuming df is already created and populated from the previous code
# List of columns to drop
columns_to_drop = ['_meta', '_captcha', '_login', '_error']

# Drop the specified columns if they exist in the DataFrame
# Using errors='ignore' will prevent an error if a column is not found
df = df.drop(columns=columns_to_drop, errors='ignore')

# Display the columns to verify
df.columns

In [ ]:
df

In [ ]:
df1 = pd.read_csv('/content/texas_state_researchers_openalex.csv')

In [ ]:
df1.head()

In [ ]:
df = pd.read_csv('/content/texas_state_nsf_grants.csv')

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

# Researchers to remove (not found in OpenAlex or not at Texas State)
researchers_to_remove = [
    # Not found in OpenAlex
    "Danny Wescott", "Diane Dolozel", "Emanuel Alanis", "Holly Veselka",
    "Mario Cristian Gaedicke-Hornung", "Sean Bauld", "Vishan Shen",
    # Found but not affiliated with Texas State
    "Chiu Au", "Edward Yu", "Feng Wang", "Gabriel Lopez", "Holly Lewis",
    "Jie Zhu", "Li Feng", "Mike Ferrara", "Nadim Adi", "Stefan Zauscher",
    "Yaroslava Yingling"
]

# Original list of researchers (90 total)
original_researchers = [
    "Alice Olmstead", "Apan Qasem", "Araceli Martinez Ortiz", "Barbara Hewitt",
    "Byoung-Hee You", "Carolyn Chang", "Chiu Au", "Christopher Rhodes",
    "Chul-Ho Lee", "Cindy Royal", "Clara Novoa", "Craig Hanks",
    "Cynthia Luxford", "Damian Valles Molina", "Danny Wescott", "David Gibbs",
    "Denise Gobert", "Diane Dolozel", "Dincer Konur", "Dominick Fazarro",
    "Eduardo Perez", "Edward Yu", "Edwin Chow", "Edwin Piner",
    "Eleanor Close", "Emanuel Alanis", "Emily Zhu", "Erica Nason",
    "Eunsang Cho", "Feng Wang", "Francis Mendez", "Gabriel Lopez",
    "Gregory Lakomski", "Heather Galloway", "Holly Lewis", "Holly Veselka",
    "Hsing-Huang Tseng", "Hyunhwan Kim", "In-Hyouk Song", "Ivan Ojeda-Ruiz",
    "Jana Minifie", "Jelena Tesic", "Jennifer Irvin", "Jie Zhu",
    "Jitendra Tate", "Karen Lewis", "Keshav Bhandari", "Kimberly Talley",
    "Larry Price", "Li Feng", "Lucia Summers", "Maria Resendiz",
    "Mario Cristian Gaedicke-Hornung", "Mike Ferrara", "Mina Guirguis",
    "Monica Hughes", "Mylene Farias", "Nadim Adi", "Nicole Taylor",
    "Nikoleta Theodoropoulou", "Rasim Musal", "Ravindranath Droopad",
    "Sarah Fritts", "Satyajit Dutta", "Sean Bauld", "Semih Aslan",
    "Shannon Williams", "Stefan Zauscher", "Steven Whitten", "Subasish Das",
    "Tahir Ekin", "Tania Betancourt", "Thomas Keller", "Togay Ozbakkaloglu",
    "Tongdan Jin", "Toni Watt", "Ty Schepis", "Vangelis Metsis",
    "Vishan Shen", "Wenquan Dong", "Wilhelmus Geerts", "William Brittain",
    "Xiangping Liu", "Xiaoxi Shen", "Yaroslava Yingling", "Yihong 'Maggie' Chen",
    "Yihong Yuan", "Yong Yang", "Young Ju Lee", "Ziliang Zong"
]

# Filter out researchers to remove
valid_researchers = [name for name in original_researchers if name not in researchers_to_remove]

print(f"Original researchers: {len(original_researchers)}")
print(f"Researchers removed: {len(researchers_to_remove)}")
print(f"Valid researchers remaining: {len(valid_researchers)}")

# Function to parse dates in format "June 1, 2012"
def parse_date(date_str):
    """Parse date string in format 'Month Day, Year'"""
    if pd.isna(date_str) or date_str == '':
        return None
    try:
        return datetime.strptime(date_str.strip(), "%B %d, %Y")
    except:
        try:
            # Try alternative format without comma
            return datetime.strptime(date_str.strip(), "%B %d %Y")
        except:
            print(f"Could not parse date: {date_str}")
            return None

# Function to normalize names for matching
def normalize_name(name):
    """Normalize name for matching"""
    if pd.isna(name):
        return ""
    # Remove extra whitespace, convert to lowercase
    name = re.sub(r'\s+', ' ', str(name).strip().lower())
    # Remove common prefixes/suffixes
    name = re.sub(r'\b(dr\.?|prof\.?|professor)\b', '', name).strip()
    return name

# Function to check if names match (flexible matching)
def names_match(name1, name2):
    """Check if two names likely refer to the same person"""
    norm1 = normalize_name(name1)
    norm2 = normalize_name(name2)

    if norm1 == norm2:
        return True

    # Split into parts
    parts1 = norm1.split()
    parts2 = norm2.split()

    if len(parts1) < 2 or len(parts2) < 2:
        return False

    # Check if first and last names match
    first1, last1 = parts1[0], parts1[-1]
    first2, last2 = parts2[0], parts2[-1]

    return first1 == first2 and last1 == last2

# Load your dataframes (replace with your actual loading code)
# df = pd.read_csv('grants_data.csv')  # Your grants dataframe
# df1 = pd.read_csv('texas_state_researchers_openalex.csv')  # Your OpenAlex dataframe

# For demonstration, I'll create the structure you need:
def analyze_grants_data(df, df1, valid_researchers):
    """
    Analyze grants data to count PI and Co-PI roles in last 7 years

    Parameters:
    df: DataFrame with grants data
    df1: DataFrame with researcher OpenAlex data
    valid_researchers: List of valid researcher names
    """

    # Filter df1 to only include valid researchers
    df1_filtered = df1[df1['name'].isin(valid_researchers)].copy()

    # Parse dates in grants dataframe
    df['start_date_parsed'] = df['start_date'].apply(parse_date)
    df['expiration_date_parsed'] = df['expiration_date'].apply(parse_date)

    # Calculate cutoff date (7 years ago from today)
    cutoff_date = datetime.now() - timedelta(days=7*365)

    # Filter grants to last 7 years (using start date)
    recent_grants = df[
        (df['start_date_parsed'].notna()) &
        (df['start_date_parsed'] >= cutoff_date)
    ].copy()

    print(f"Total grants in dataset: {len(df)}")
    print(f"Grants from last 7 years: {len(recent_grants)}")

    # Initialize results dictionary
    results = []

    for _, researcher in df1_filtered.iterrows():
        name = researcher['name']
        openalex_id = researcher['openalex_id']
        works_count = researcher['works_count']
        cited_by_count = researcher['cited_by_count']

        pi_count = 0
        copi_count = 0
        affiliated_awards = []

        # Count PI roles
        pi_matches = recent_grants[
            recent_grants['principal_investigator'].apply(
                lambda x: names_match(x, name) if pd.notna(x) else False
            )
        ]
        pi_count = len(pi_matches)
        affiliated_awards.extend(pi_matches['award_id'].tolist())

        # Count Co-PI roles
        for _, grant in recent_grants.iterrows():
            co_pis = grant['co_principal_investigators']
            if pd.notna(co_pis):
                # Split by semicolon and check each co-PI
                co_pi_list = [copi.strip() for copi in str(co_pis).split(';')]
                for co_pi in co_pi_list:
                    if names_match(co_pi, name):
                        copi_count += 1
                        affiliated_awards.append(grant['award_id'])
                        break  # Only count once per grant

        # Remove duplicate award IDs
        affiliated_awards = list(set([aid for aid in affiliated_awards if pd.notna(aid)]))

        results.append({
            'name': name,
            'openalex_id': openalex_id,
            'citations': cited_by_count,
            'works_count': works_count,
            'pi_7_years': pi_count,
            'copi_7_years': copi_count,
            'affiliated_award_ids': '; '.join(map(str, affiliated_awards)) if affiliated_awards else ''
        })

        if pi_count > 0 or copi_count > 0:
            print(f"{name}: PI={pi_count}, Co-PI={copi_count}, Awards={len(affiliated_awards)}")

    return pd.DataFrame(results)

# Create the analysis function call
# Replace 'df' and 'df1' with your actual dataframe variables
# final_df = analyze_grants_data(df, df1, valid_researchers)

# Save results
# final_df.to_csv('texas_state_researchers_final_analysis.csv', index=False)

# Display summary
# print(f"\nFinal dataset shape: {final_df.shape}")
# print(f"\nResearchers with PI roles in last 7 years: {len(final_df[final_df['pi_7_years'] > 0])}")
# print(f"Researchers with Co-PI roles in last 7 years: {len(final_df[final_df['copi_7_years'] > 0])}")
# print(f"Researchers with any grant role in last 7 years: {len(final_df[(final_df['pi_7_years'] > 0) | (final_df['copi_7_years'] > 0)])}")

print("Code structure ready. Replace df and df1 with your actual dataframes and uncomment the function calls.")

# Example of how to use (uncomment and modify):


# Run analysis
final_df = analyze_grants_data(df, df1, valid_researchers)

# Save results
final_df.to_csv('texas_state_researchers_final_analysis.csv', index=False)

final_df

In [ ]:
final_df

In [ ]:
final_df.columns

In [ ]:
final_df = pd.read_csv('/content/texas_state_researchers_final_analysis.csv')

In [ ]:
import pandas as pd
import requests
import time
import json
import os
from typing import List, Dict, Any
from collections import defaultdict
import numpy as np
from datetime import datetime

def fetch_author_works(openalex_id: str, per_page: int = 200) -> List[Dict]:
    """
    Fetch all works for a given author from OpenAlex API
    """
    all_works = []
    page = 1

    # Clean the OpenAlex ID if it contains the full URL
    if openalex_id.startswith('https://openalex.org/'):
        author_id = openalex_id
    else:
        author_id = f"https://openalex.org/{openalex_id}" if not openalex_id.startswith('A') else f"https://openalex.org/{openalex_id}"

    while True:
        url = f"https://api.openalex.org/works"
        params = {
            'filter': f'author.id:{author_id}',
            'per-page': per_page,
            'page': page,
            'select': 'id,title,publication_year,concepts,keywords,topics,related_works,type,cited_by_count'
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            works = data.get('results', [])
            if not works:
                break

            all_works.extend(works)

            # Check if we've got all pages
            if len(works) < per_page:
                break

            page += 1

            # Rate limiting - OpenAlex allows 100 requests per second
            time.sleep(0.01)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching works for {author_id}: {e}")
            break

    return all_works

def extract_topics_from_related_works(related_work_ids: List[str], max_related: int = 10) -> List[Dict]:
    """
    Fetch topics from related works (limited to avoid too many API calls)
    """
    related_topics = []

    # Limit the number of related works to fetch to avoid excessive API calls
    limited_ids = related_work_ids[:max_related]

    for work_id in limited_ids:
        try:
            url = f"https://api.openalex.org/works/{work_id}"
            params = {'select': 'topics,concepts'}

            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            # Extract topics and concepts from related work
            topics = data.get('topics', [])
            concepts = data.get('concepts', [])

            for topic in topics:
                related_topics.append({
                    'name': topic.get('display_name', ''),
                    'score': topic.get('score', 0),
                    'type': 'related_topic',
                    'source_work': work_id
                })

            for concept in concepts:
                if concept.get('score', 0) > 0.3:  # Only high-confidence concepts
                    related_topics.append({
                        'name': concept.get('display_name', ''),
                        'score': concept.get('score', 0),
                        'type': 'related_concept',
                        'source_work': work_id
                    })

            time.sleep(0.01)  # Rate limiting

        except requests.exceptions.RequestException as e:
            print(f"Error fetching related work {work_id}: {e}")
            continue

    return related_topics

def create_curated_topic_list(work: Dict) -> List[Dict]:
    """
    Create a curated list combining topics, keywords, concepts, and related topics
    """
    curated_topics = []

    # Extract topics
    topics = work.get('topics', [])
    for topic in topics:
        curated_topics.append({
            'name': topic.get('display_name', ''),
            'score': topic.get('score', 0),
            'type': 'topic',
            'subfield': topic.get('subfield', {}).get('display_name', ''),
            'field': topic.get('field', {}).get('display_name', ''),
            'domain': topic.get('domain', {}).get('display_name', '')
        })

    # Extract keywords
    keywords = work.get('keywords', [])
    for keyword in keywords:
        curated_topics.append({
            'name': keyword.get('display_name', ''),
            'score': keyword.get('score', 0),
            'type': 'keyword',
            'subfield': '',
            'field': '',
            'domain': ''
        })

    # Extract concepts (only those with score > 0.3)
    concepts = work.get('concepts', [])
    for concept in concepts:
        if concept.get('score', 0) > 0.3:
            curated_topics.append({
                'name': concept.get('display_name', ''),
                'score': concept.get('score', 0),
                'type': 'concept',
                'level': concept.get('level', 0),
                'subfield': '',
                'field': '',
                'domain': ''
            })

    # Extract topics from related works (limited to avoid too many API calls)
    related_work_ids = work.get('related_works', [])
    if related_work_ids:
        related_topics = extract_topics_from_related_works(related_work_ids, max_related=5)
        curated_topics.extend(related_topics)

    # Remove duplicates and sort by score
    unique_topics = {}
    for topic in curated_topics:
        name = topic['name'].lower().strip()
        if name and name not in unique_topics:
            unique_topics[name] = topic
        elif name in unique_topics and topic['score'] > unique_topics[name]['score']:
            unique_topics[name] = topic

    # Sort by score (descending)
    sorted_topics = sorted(unique_topics.values(), key=lambda x: x['score'], reverse=True)

    return sorted_topics

def process_all_researchers(final_df: pd.DataFrame, checkpoint_dir: str = "/content/drive/MyDrive/datastore/v2_data") -> pd.DataFrame:
    """
    Process all researchers to get their works and curated topics with checkpoint functionality
    """
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Check for existing checkpoints
    checkpoint_file = os.path.join(checkpoint_dir, "researcher_works_checkpoint.csv")
    progress_file = os.path.join(checkpoint_dir, "progress_checkpoint.json")

    results = []
    start_idx = 0

    # Load existing progress if checkpoint exists
    if os.path.exists(checkpoint_file) and os.path.exists(progress_file):
        print("Found existing checkpoint, loading...")
        try:
            existing_df = pd.read_csv(checkpoint_file)
            if len(existing_df) > 0:
                # Convert string representation back to list of dicts
                existing_df['curated_topics'] = existing_df['curated_topics'].apply(
                    lambda x: eval(x) if isinstance(x, str) else x
                )
                results = existing_df.to_dict('records')

                with open(progress_file, 'r') as f:
                    progress = json.load(f)
                    start_idx = progress.get('last_completed_idx', 0) + 1

                print(f"Resuming from researcher {start_idx + 1}")
        except Exception as e:
            print(f"Error loading checkpoint: {e}")
            print("Starting fresh...")
            start_idx = 0
            results = []

    total_researchers = len(final_df)

    for idx in range(start_idx, total_researchers):
        row = final_df.iloc[idx]
        print(f"Processing researcher {idx + 1}/{total_researchers}: {row['name']}")

        try:
            # Fetch all works for this researcher
            works = fetch_author_works(row['openalex_id'])

            print(f"  Found {len(works)} works")

            # Process each work
            for work in works:
                # Create curated topic list for this work
                curated_topics = create_curated_topic_list(work)

                work_data = {
                    'researcher_name': row['name'],
                    'researcher_openalex_id': row['openalex_id'],
                    'work_id': work.get('id', ''),
                    'work_title': work.get('title', ''),
                    'publication_year': work.get('publication_year', None),
                    'work_type': work.get('type', ''),
                    'cited_by_count': work.get('cited_by_count', 0),
                    'curated_topics': curated_topics,
                    'num_topics': len([t for t in curated_topics if t['type'] == 'topic']),
                    'num_keywords': len([t for t in curated_topics if t['type'] == 'keyword']),
                    'num_concepts': len([t for t in curated_topics if t['type'] == 'concept']),
                    'num_related_topics': len([t for t in curated_topics if t['type'].startswith('related')])
                }

                results.append(work_data)

        except Exception as e:
            print(f"Error processing researcher {row['name']}: {e}")
            continue

        # Save checkpoint every 5 researchers
        if (idx + 1) % 5 == 0:
            save_checkpoint(results, idx, checkpoint_dir)
            print(f"Checkpoint saved after processing {idx + 1} researchers")

        # Add delay between researchers to be respectful to the API
        time.sleep(0.1)

    # Save final checkpoint
    save_checkpoint(results, total_researchers - 1, checkpoint_dir, is_final=True)

    return pd.DataFrame(results)

def save_checkpoint(results: List[Dict], last_idx: int, checkpoint_dir: str, is_final: bool = False):
    """
    Save checkpoint data to files
    """
    try:
        # Convert to DataFrame for saving
        df = pd.DataFrame(results)

        if len(df) > 0:
            # Convert curated_topics to string for CSV storage
            df_to_save = df.copy()
            df_to_save['curated_topics'] = df_to_save['curated_topics'].astype(str)

            # Save checkpoint file
            checkpoint_file = os.path.join(checkpoint_dir, "researcher_works_checkpoint.csv")
            df_to_save.to_csv(checkpoint_file, index=False)

            # Save progress file
            progress_file = os.path.join(checkpoint_dir, "progress_checkpoint.json")
            progress_data = {
                'last_completed_idx': last_idx,
                'total_works_processed': len(df),
                'timestamp': datetime.now().isoformat(),
                'is_final': is_final
            }

            with open(progress_file, 'w') as f:
                json.dump(progress_data, f, indent=2)

            print(f"Saved {len(df)} works to checkpoint")

    except Exception as e:
        print(f"Error saving checkpoint: {e}")

def clean_checkpoints(checkpoint_dir: str = "/content/drive/MyDrive/datastore/v2_data"):
    """
    Clean up checkpoint files after successful completion
    """
    try:
        checkpoint_file = os.path.join(checkpoint_dir, "researcher_works_checkpoint.csv")
        progress_file = os.path.join(checkpoint_dir, "progress_checkpoint.json")

        if os.path.exists(checkpoint_file):
            os.remove(checkpoint_file)
        if os.path.exists(progress_file):
            os.remove(progress_file)

        print("Checkpoint files cleaned up")
    except Exception as e:
        print(f"Error cleaning checkpoints: {e}")

def save_final_data(works_df: pd.DataFrame, summary_df: pd.DataFrame,
                   save_dir: str = "/content/drive/MyDrive/datastore/v2_data"):
    """
    Save final processed data with timestamp
    """
    os.makedirs(save_dir, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save works data
    works_file = os.path.join(save_dir, f"researcher_works_with_topics_{timestamp}.csv")

    # Convert curated_topics to string for CSV storage
    works_to_save = works_df.copy()
    works_to_save['curated_topics'] = works_to_save['curated_topics'].astype(str)
    works_to_save.to_csv(works_file, index=False)

    # Save summary data
    summary_file = os.path.join(save_dir, f"researcher_topic_summary_{timestamp}.csv")

    # Convert top_topics to string for CSV storage
    summary_to_save = summary_df.copy()
    summary_to_save['top_topics'] = summary_to_save['top_topics'].astype(str)
    summary_to_save.to_csv(summary_file, index=False)

    # Also save the latest versions without timestamp
    latest_works_file = os.path.join(save_dir, "researcher_works_with_topics_latest.csv")
    latest_summary_file = os.path.join(save_dir, "researcher_topic_summary_latest.csv")

    works_to_save.to_csv(latest_works_file, index=False)
    summary_to_save.to_csv(latest_summary_file, index=False)

    print(f"Final data saved:")
    print(f"  Works data: {works_file}")
    print(f"  Summary data: {summary_file}")
    print(f"  Latest versions also saved without timestamps")

    return works_file, summary_file

def get_topic_summary_by_researcher(works_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a summary of topics by researcher
    """
    summary_data = []

    for researcher in works_df['researcher_name'].unique():
        researcher_works = works_df[works_df['researcher_name'] == researcher]

        # Collect all topics for this researcher
        all_topics = []
        for _, work in researcher_works.iterrows():
            all_topics.extend(work['curated_topics'])

        # Count topic frequencies
        topic_counts = defaultdict(lambda: {'count': 0, 'total_score': 0, 'types': set()})

        for topic in all_topics:
            name = topic['name']
            topic_counts[name]['count'] += 1
            topic_counts[name]['total_score'] += topic['score']
            topic_counts[name]['types'].add(topic['type'])

        # Get top topics for this researcher
        top_topics = sorted(
            topic_counts.items(),
            key=lambda x: (x[1]['count'], x[1]['total_score']),
            reverse=True
        )[:20]  # Top 20 topics

        summary_data.append({
            'researcher_name': researcher,
            'researcher_openalex_id': researcher_works.iloc[0]['researcher_openalex_id'],
            'total_works': len(researcher_works),
            'total_citations': researcher_works['cited_by_count'].sum(),
            'top_topics': [
                {
                    'name': topic[0],
                    'frequency': topic[1]['count'],
                    'avg_score': topic[1]['total_score'] / topic[1]['count'],
                    'types': list(topic[1]['types'])
                }
                for topic in top_topics
            ]
        })

    return pd.DataFrame(summary_data)

# Main execution function
def main(final_df: pd.DataFrame, save_dir: str = "/content/drive/MyDrive/datastore/v2_data"):
    """
    Main function to process all researchers and their works with checkpoint support
    """
    print("Starting to process researchers and their works...")
    print(f"Processing {len(final_df)} researchers")
    print(f"Data will be saved to: {save_dir}")
    print("Checkpoints will be saved every 5 researchers")

    # Process all researchers and their works (with checkpoints)
    works_df = process_all_researchers(final_df, save_dir)

    print(f"\nProcessed {len(works_df)} total works")

    # Create summary by researcher
    print("Creating researcher summaries...")
    summary_df = get_topic_summary_by_researcher(works_df)

    # Save final data
    print("Saving final data...")
    works_file, summary_file = save_final_data(works_df, summary_df, save_dir)

    # Clean up checkpoint files
    clean_checkpoints(save_dir)

    print("\nProcessing complete!")
    print(f"Total researchers processed: {summary_df.shape[0]}")
    print(f"Total works processed: {len(works_df)}")
    print(f"Files saved to: {save_dir}")

    return works_df, summary_df

# Resume from checkpoint function
def resume_processing(final_df: pd.DataFrame, save_dir: str = "/content/drive/MyDrive/datastore/v2_data"):
    """
    Resume processing from the last checkpoint
    """
    progress_file = os.path.join(save_dir, "progress_checkpoint.json")

    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)

        print(f"Found checkpoint from {progress['timestamp']}")
        print(f"Last completed researcher index: {progress['last_completed_idx']}")
        print(f"Works processed so far: {progress['total_works_processed']}")

        return main(final_df, save_dir)
    else:
        print("No checkpoint found. Starting fresh...")
        return main(final_df, save_dir)

# Function to check current progress
def check_progress(save_dir: str = "/content/drive/MyDrive/datastore/v2_data"):
    """
    Check the current progress if a checkpoint exists
    """
    progress_file = os.path.join(save_dir, "progress_checkpoint.json")

    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)

        print("Current Progress:")
        print(f"  Last completed researcher index: {progress['last_completed_idx']}")
        print(f"  Total works processed: {progress['total_works_processed']}")
        print(f"  Last update: {progress['timestamp']}")
        print(f"  Is final: {progress.get('is_final', False)}")

        return progress
    else:
        print("No checkpoint found.")
        return None

# Example usage:

# Method 1: Start fresh processing
# works_df, summary_df = main(final_df)

# Method 2: Resume from checkpoint (if exists)
works_df, summary_df = resume_processing(final_df)

# Method 3: Check current progress
# check_progress()

# Method 4: Manual checkpoint control
# works_df = process_all_researchers(final_df, "/content/drive/MyDrive/datastore/v2_data")
# summary_df = get_topic_summary_by_researcher(works_df)
# save_final_data(works_df, summary_df, "/content/drive/MyDrive/datastore/v2_data")

# The recommended approach is to use resume_processing() as it will:
# 1. Check for existing checkpoints and resume if found
# 2. Start fresh if no checkpoints exist
# 3. Save checkpoints every 5 researchers automatically
# 4. Save final data with timestamps
# 5. Clean up checkpoint files when complete

# Example of complete workflow:
"""
# Check if there's any existing progress
check_progress()

# Start or resume processing
works_df, summary_df = resume_processing(final_df)

# Files will be automatically saved to /content/drive/MyDrive/datastore/v2_data/
# with both timestamped versions and "latest" versions
"""

# To view a sample of curated topics for a specific work:
def display_sample_topics(works_df: pd.DataFrame, work_index: int = 0):
    """
    Display sample curated topics for a specific work
    """
    if work_index < len(works_df):
        work = works_df.iloc[work_index]
        print(f"Work: {work['work_title']}")
        print(f"Researcher: {work['researcher_name']}")
        print(f"Year: {work['publication_year']}")
        print("\nCurated Topics:")

        for i, topic in enumerate(work['curated_topics'][:10]):  # Show top 10
            print(f"{i+1}. {topic['name']} (Score: {topic['score']:.3f}, Type: {topic['type']})")
            if topic.get('field'):
                print(f"   Field: {topic['field']} > {topic['subfield']}")
    else:
        print("Work index out of range")

# Example of how to use:
# display_sample_topics(works_df, 0)

# Handling Google Colab disconnections:
"""
If your Colab session disconnects or crashes:

1. Reconnect to your session
2. Re-run the import statements and function definitions
3. Load your final_df again
4. Run: works_df, summary_df = resume_processing(final_df)

The system will automatically detect where you left off and continue processing
from the last completed checkpoint.

Checkpoint files stored in /content/drive/MyDrive/datastore/v2_data/:
- researcher_works_checkpoint.csv (temporary checkpoint data)
- progress_checkpoint.json (progress tracking)
- researcher_works_with_topics_YYYYMMDD_HHMMSS.csv (final timestamped results)
- researcher_topic_summary_YYYYMMDD_HHMMSS.csv (final timestamped summary)
- researcher_works_with_topics_latest.csv (latest version without timestamp)
- researcher_topic_summary_latest.csv (latest summary without timestamp)
"""

In [ ]:
import pandas as pd
import requests
import time
import json
import os
from typing import List, Dict, Optional
from datetime import datetime
from tqdm import tqdm

def call_openalex_api(endpoint, params=None, email="your_email@example.com"):
    """Make API calls with proper headers and rate limiting"""
    base_url = f"https://api.openalex.org/{endpoint}"
    headers = {'User-Agent': f'mailto:{email}'}

    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        time.sleep(0.2)  # Conservative rate limiting
        return response.json()
    except Exception as e:
        print(f"Error calling {endpoint} API: {str(e)}")
        return None

def reconstruct_abstract(inverted_index: Dict) -> str:
    """Reconstruct abstract text from OpenAlex inverted index format."""
    if not inverted_index:
        return ""

    word_positions = []
    for word, positions in inverted_index.items():
        for pos in positions:
            word_positions.append((pos, word))

    word_positions.sort(key=lambda x: x[0])
    words = [word for _, word in word_positions]

    return " ".join(words)

def get_single_work_abstract(work_id: str, email: str = "your_email@example.com") -> str:
    """Fetch abstract for a single work ID"""
    try:
        # Clean the work ID
        if work_id.startswith('https://openalex.org/'):
            clean_id = work_id.split('/')[-1]
        elif work_id.startswith('W'):
            clean_id = work_id
        else:
            clean_id = f"W{work_id}"

        # Call API for single work
        response = call_openalex_api(f'works/{clean_id}', email=email)

        if response and 'abstract_inverted_index' in response:
            abstract_inverted = response['abstract_inverted_index']
            return reconstruct_abstract(abstract_inverted)
        else:
            return ""

    except Exception as e:
        print(f"Error fetching abstract for {work_id}: {str(e)}")
        return ""

def fetch_abstracts_for_existing_data(df: pd.DataFrame,
                                    email: str = "your_email@example.com",
                                    checkpoint_dir: str = "/content/drive/MyDrive/datastore/v2_data",
                                    save_every: int = 50) -> pd.DataFrame:
    """
    Fetch abstracts for existing work data using individual API calls with robust checkpointing
    """

    print(f"IMPORTANT: Make sure to update email parameter!")
    print(f"Current email: {email}")
    print("=" * 70)

    # Create checkpoint directory
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Checkpoint files
    abstracts_checkpoint_file = os.path.join(checkpoint_dir, "abstracts_robust_checkpoint.json")
    progress_checkpoint_file = os.path.join(checkpoint_dir, "abstracts_robust_progress.json")

    # Get unique work IDs
    unique_work_ids = df['work_id'].unique().tolist()
    total_works = len(unique_work_ids)

    print(f"Found {total_works} unique works to fetch abstracts for")

    # Load existing abstracts if checkpoint exists
    fetched_abstracts = {}
    start_idx = 0

    if os.path.exists(abstracts_checkpoint_file):
        print("Found existing abstracts checkpoint, loading...")
        try:
            with open(abstracts_checkpoint_file, 'r') as f:
                fetched_abstracts = json.load(f)

            if os.path.exists(progress_checkpoint_file):
                with open(progress_checkpoint_file, 'r') as f:
                    progress = json.load(f)
                    start_idx = progress.get('last_completed_index', 0) + 1

            print(f"Loaded {len(fetched_abstracts)} existing abstracts")
            print(f"Resuming from work {start_idx + 1}/{total_works}")

        except Exception as e:
            print(f"Error loading checkpoint: {e}")
            print("Starting fresh...")
            fetched_abstracts = {}
            start_idx = 0

    # Process remaining work IDs
    remaining_work_ids = unique_work_ids[start_idx:]

    if remaining_work_ids:
        print(f"\nFetching abstracts for {len(remaining_work_ids)} remaining works...")

        success_count = len(fetched_abstracts)

        # Process each work individually with progress bar
        for i, work_id in enumerate(tqdm(remaining_work_ids, desc="Fetching abstracts")):
            current_idx = start_idx + i

            try:
                # Skip if already fetched (shouldn't happen, but safety check)
                if work_id in fetched_abstracts:
                    continue

                # Fetch abstract for this work
                abstract = get_single_work_abstract(work_id, email)
                fetched_abstracts[work_id] = abstract

                if abstract:
                    success_count += 1

                # Save checkpoint every N works
                if (i + 1) % save_every == 0:
                    save_robust_checkpoint(fetched_abstracts, current_idx, checkpoint_dir)
                    print(f"\nCheckpoint saved: {len(fetched_abstracts)} abstracts fetched ({success_count} non-empty)")

            except Exception as e:
                print(f"Error processing work {work_id}: {e}")
                # Still add empty abstract to avoid re-processing
                fetched_abstracts[work_id] = ""
                continue

    # Final checkpoint save
    if remaining_work_ids:
        save_robust_checkpoint(fetched_abstracts, total_works - 1, checkpoint_dir, is_final=True)

    # Add abstracts to dataframe
    print("\nMerging abstracts into dataframe...")
    df_copy = df.copy()
    df_copy['abstract'] = df_copy['work_id'].map(lambda x: fetched_abstracts.get(x, ""))

    # Statistics
    non_empty_abstracts = (df_copy['abstract'] != "").sum()
    success_rate = non_empty_abstracts / len(df_copy) * 100

    print(f"\nAbstract Fetching Complete!")
    print(f"Total works: {len(df_copy)}")
    print(f"Works with abstracts: {non_empty_abstracts}")
    print(f"Success rate: {success_rate:.1f}%")

    # Clean up checkpoint files
    cleanup_robust_checkpoints(checkpoint_dir)

    return df_copy

def save_robust_checkpoint(abstracts: Dict[str, str], last_idx: int, checkpoint_dir: str, is_final: bool = False):
    """Save checkpoint for robust abstract fetching"""
    try:
        abstracts_file = os.path.join(checkpoint_dir, "abstracts_robust_checkpoint.json")
        progress_file = os.path.join(checkpoint_dir, "abstracts_robust_progress.json")

        # Save abstracts
        with open(abstracts_file, 'w') as f:
            json.dump(abstracts, f)

        # Save progress
        progress_data = {
            'last_completed_index': last_idx,
            'total_abstracts_fetched': len(abstracts),
            'non_empty_abstracts': len([a for a in abstracts.values() if a]),
            'timestamp': datetime.now().isoformat(),
            'is_final': is_final
        }

        with open(progress_file, 'w') as f:
            json.dump(progress_data, f, indent=2)

    except Exception as e:
        print(f"Error saving checkpoint: {e}")

def cleanup_robust_checkpoints(checkpoint_dir: str):
    """Clean up checkpoint files after completion"""
    try:
        abstracts_file = os.path.join(checkpoint_dir, "abstracts_robust_checkpoint.json")
        progress_file = os.path.join(checkpoint_dir, "abstracts_robust_progress.json")

        if os.path.exists(abstracts_file):
            os.remove(abstracts_file)
        if os.path.exists(progress_file):
            os.remove(progress_file)

        print("Robust abstracts checkpoint files cleaned up")

    except Exception as e:
        print(f"Error cleaning up checkpoints: {e}")

def check_robust_progress(checkpoint_dir: str = "/content/drive/MyDrive/datastore/v2_data"):
    """Check current progress of robust abstract fetching"""
    progress_file = os.path.join(checkpoint_dir, "abstracts_robust_progress.json")

    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)

        print("Robust Abstract Fetching Progress:")
        print(f"  Last completed index: {progress['last_completed_index']}")
        print(f"  Total abstracts fetched: {progress['total_abstracts_fetched']}")
        print(f"  Non-empty abstracts: {progress.get('non_empty_abstracts', 'Unknown')}")
        print(f"  Last update: {progress['timestamp']}")
        print(f"  Is final: {progress.get('is_final', False)}")

        return progress
    else:
        print("No robust abstract fetching progress found.")
        return None

def test_single_abstract(work_id: str, email: str = "your_email@example.com"):
    """Test fetching a single abstract with detailed debugging"""
    print(f"Testing abstract fetch for work ID: {work_id}")
    print(f"Using email: {email}")

    try:
        # Clean the work ID
        if work_id.startswith('https://openalex.org/'):
            clean_id = work_id.split('/')[-1]
        elif work_id.startswith('W'):
            clean_id = work_id
        else:
            clean_id = f"W{work_id}"

        print(f"Cleaned work ID: {clean_id}")

        # Call API for single work
        response = call_openalex_api(f'works/{clean_id}', email=email)

        print(f"\nAPI Response Status: {'Success' if response else 'Failed'}")

        if response:
            print(f"Response keys: {list(response.keys())}")
            print(f"Work title: {response.get('display_name', 'No title')}")
            print(f"Publication year: {response.get('publication_year', 'Unknown')}")
            print(f"Has abstract_inverted_index: {'abstract_inverted_index' in response}")

            if 'abstract_inverted_index' in response:
                abstract_inverted = response['abstract_inverted_index']
                print(f"Abstract inverted index type: {type(abstract_inverted)}")
                print(f"Abstract inverted index empty: {not bool(abstract_inverted)}")

                if abstract_inverted:
                    print(f"Sample words in abstract: {list(abstract_inverted.keys())[:10]}")
                    abstract = reconstruct_abstract(abstract_inverted)
                    print(f"Reconstructed abstract length: {len(abstract)} characters")
                    if abstract:
                        print(f"Abstract preview: {abstract[:300]}...")
                        return abstract
                    else:
                        print("Failed to reconstruct abstract")
                        return ""
                else:
                    print("Abstract inverted index is empty")
                    return ""
            else:
                print("No abstract_inverted_index field in response")
                return ""
        else:
            print("API call failed")
            return ""

    except Exception as e:
        print(f"Error in test: {str(e)}")
        return ""

def test_multiple_works(df: pd.DataFrame, email: str = "your_email@example.com", num_samples: int = 5):
    """Test multiple works to find ones with abstracts"""
    print(f"Testing {num_samples} sample works to find abstracts...")

    sample_works = df.sample(min(num_samples, len(df)))

    for i, (_, row) in enumerate(sample_works.iterrows()):
        print(f"\n--- Test {i+1}/{num_samples} ---")
        print(f"Work: {row['work_title']}")
        print(f"Year: {row['publication_year']}")
        print(f"Citations: {row['cited_by_count']}")

        abstract = test_single_abstract(row['work_id'], email)

        if abstract:
            print("✅ Found abstract!")
            return True
        else:
            print("❌ No abstract")

    print(f"\n⚠️  No abstracts found in {num_samples} samples")
    return False

def main_robust_abstracts(csv_path: str = "/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_latest.csv",
                         email: str = "your_email@example.com"):
    """
    Main function to add abstracts to existing CSV using robust method
    """
    print("Robust Abstract Fetcher for Existing Data")
    print("=" * 50)

    # Load existing data
    print("Loading existing data...")
    df = pd.read_csv(csv_path)

    print(f"Loaded {len(df)} works from {len(df['researcher_name'].unique())} researchers")
    print(f"Columns: {list(df.columns)}")

    # Test with multiple works to find ones with abstracts
    print("\nTesting with multiple works to find abstracts...")
    abstracts_found = test_multiple_works(df, email, num_samples=10)

    if abstracts_found or input("\nContinue even though no abstracts found in samples? (y/n): ").lower() == 'y':
        print("\nProceeding with full dataset...")

        # Fetch all abstracts
        df_with_abstracts = fetch_abstracts_for_existing_data(df, email)

        # Save updated CSV
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = os.path.dirname(csv_path)

        # Save timestamped version
        output_file = os.path.join(base_dir, f"researcher_works_with_topics_and_abstracts_robust_{timestamp}.csv")
        df_with_abstracts.to_csv(output_file, index=False)

        # Save latest version
        latest_file = os.path.join(base_dir, "researcher_works_with_topics_and_abstracts_robust_latest.csv")
        df_with_abstracts.to_csv(latest_file, index=False)

        print(f"\nSuccess! Updated data saved to:")
        print(f"  Timestamped: {output_file}")
        print(f"  Latest: {latest_file}")

        return df_with_abstracts
    else:
        print("Stopping due to test failure.")
        return None

# Usage examples:

# Method 1: Run the complete process (REMEMBER TO UPDATE EMAIL!)
df_with_abstracts = main_robust_abstracts(email="your_actual_email@example.com")

# Method 2: Check current progress
# check_robust_progress()

# Method 3: Test with a single work ID first (with debugging)
# test_single_abstract("W2556052759", email="your_actual_email@example.com")

# Method 3b: Test with multiple works to find abstracts
# df = pd.read_csv("/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_latest.csv")
# test_multiple_works(df, email="your_actual_email@example.com", num_samples=10)

# Method 4: Resume from checkpoint (just run main_robust_abstracts again)
# df_with_abstracts = main_robust_abstracts(email="your_actual_email@example.com")

print("=" * 70)
print("IMPORTANT SETUP INSTRUCTIONS:")
print("1. Update the email parameter with your actual email address")
print("2. Run: df_with_abstracts = main_robust_abstracts(email='your_email@domain.com')")
print("3. The script will test with one work first, then process all")
print("4. Checkpoints are saved every 50 works")
print("5. You can resume if interrupted by running the same command again")
print("=" * 70)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_latest.csv")

In [ ]:
df['curated_topics'] = df['curated_topics'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [ ]:
# prompt: lets see the full data all of it for df.head(1)

import pandas as pd
# Configure pandas to display all columns
pd.set_option('display.max_columns', None)

# Display the first row with all columns
print(df.head(2).to_string())

# Reset display options to default
pd.reset_option('display.max_columns')


In [ ]:
df.columns

In [ ]:
df

In [ ]:
grants_df = pd.read_csv("/content/drive/MyDrive/datastore/v2_data/texas_state_researchers_final_analysis.csv")

In [ ]:
grants_df.columns

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_and_abstracts_robust_20250703_191556.csv')

In [ ]:
# prompt: find out which research has how many abstracts missing. find total percentage of missing abstract too.

import pandas as pd
# Assuming df is your DataFrame containing the fetched abstracts
# If you are loading from a file, ensure the 'abstract' column exists
# Example: df = pd.read_csv('your_data_with_abstracts.csv')

# Check if the 'abstract' column exists
if 'abstract' not in df.columns:
    print("Error: 'abstract' column not found in the DataFrame.")
else:
    # Count missing abstracts per researcher
    missing_abstracts_by_researcher = df[df['abstract'].isna() | (df['abstract'] == '')] \
        .groupby('researcher_name') \
        .size() \
        .sort_values(ascending=False)

    print("Number of missing abstracts per researcher:")
    print(missing_abstracts_by_researcher)

    # Total number of works
    total_works = len(df)

    # Total number of works with missing abstracts
    total_missing_abstracts = (df['abstract'].isna() | (df['abstract'] == '')).sum()

    # Calculate the total percentage of missing abstracts
    total_percentage_missing = (total_missing_abstracts / total_works) * 100 if total_works > 0 else 0

    print(f"\nTotal number of works: {total_works}")
    print(f"Total number of works with missing abstracts: {total_missing_abstracts}")
    print(f"Total percentage of missing abstracts: {total_percentage_missing:.2f}%")

In [ ]:
df.columns

# Preprocessing Steps

In [ ]:
# prompt: plot a distribution of publication years

import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'df' is your DataFrame and it has a 'publication_year' column

# Check if 'publication_year' column exists
if 'publication_year' in df.columns:
    # Filter out works with missing or invalid years
    valid_years = df['publication_year'].dropna().astype(int)

    if not valid_years.empty:
        plt.figure(figsize=(12, 6))
        sns.histplot(valid_years, kde=True, bins=30) # Use histplot for distribution
        plt.title('Distribution of Publication Years')
        plt.xlabel('Publication Year')
        plt.ylabel('Number of Works')
        plt.grid(axis='y', alpha=0.75)
        plt.show()
    else:
        print("No valid publication years found in the DataFrame.")
else:
    print("Error: 'publication_year' column not found in the DataFrame.")


# Preprocessing Vectors and TF-IDF

In [ ]:
import pandas as pd
import numpy as np
import json
import pickle
from pathlib import Path
import logging
from datetime import datetime
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# ML/NLP imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import math

class ResearcherProfileProcessor:
    """
    Processes researcher data to create weighted expertise profiles, conceptual vectors,
    grant experience factors, and evidence indices for expertise matching.
    """

    def __init__(self, current_year: int = 2025, min_proxy_score: float = 0.8):
        self.current_year = current_year
        self.min_proxy_score = min_proxy_score
        self.logger = self._setup_logging()
        self.sentence_model = None
        self.tfidf_model = None

    def _setup_logging(self) -> logging.Logger:
        """Setup logging configuration"""
        logger = logging.getLogger('ResearcherProfileProcessor')
        logger.setLevel(logging.INFO)
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            logger.addHandler(handler)
        return logger

    def load_data(self, researcher_file: str, grants_file: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Load and validate input datasets"""
        self.logger.info("Loading datasets...")

        # Load researcher data
        df = pd.read_csv(researcher_file)
        self.logger.info(f"Loaded {len(df)} researcher work records")

        # Load grants data
        grants_df = pd.read_csv(grants_file)
        self.logger.info(f"Loaded {len(grants_df)} grant records")

        # Validate required columns
        required_researcher_cols = ['researcher_name', 'researcher_openalex_id', 'work_id',
                                  'work_title', 'publication_year', 'cited_by_count',
                                  'curated_topics', 'abstract']
        required_grants_cols = ['openalex_id', 'pi_7_years', 'copi_7_years']

        missing_cols = [col for col in required_researcher_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in researcher data: {missing_cols}")

        missing_grant_cols = [col for col in required_grants_cols if col not in grants_df.columns]
        if missing_grant_cols:
            raise ValueError(f"Missing columns in grants data: {missing_grant_cols}")

        return df, grants_df

    def parse_curated_topics(self, topics_str: str) -> List[Dict]:
        """Parse the curated_topics string safely - handles both JSON and Python dict formats"""
        if pd.isna(topics_str) or not topics_str.strip():
            return []

        try:
            # Try JSON first (proper format)
            topics = json.loads(topics_str)
            if isinstance(topics, list):
                return topics
            else:
                return []
        except (json.JSONDecodeError, TypeError):
            try:
                # Fallback: Try ast.literal_eval for Python dict strings
                import ast
                topics = ast.literal_eval(topics_str)
                if isinstance(topics, list):
                    return topics
                else:
                    return []
            except (ValueError, SyntaxError):
                try:
                    # Last resort: Fix single quotes and try JSON again
                    fixed_json = topics_str.replace("'", '"')
                    topics = json.loads(fixed_json)
                    if isinstance(topics, list):
                        return topics
                    else:
                        return []
                except:
                    return []

    def calculate_recency_weight(self, publication_year: int) -> float:
        """
        Calculate recency weight: Wt = max(0.1, 1 - (CurrentYear - PublicationYear) / 15)
        Modified to be less restrictive and ensure minimum weight of 0.1
        """
        return max(0.1, 1 - (self.current_year - publication_year) / 15)

    def calculate_topic_importance_score(self, topic_score: float, publication_year: int,
                                       citations: int) -> float:
        """
        Calculate topic importance score (not for repetition, but for weighting)
        """
        recency_weight = self.calculate_recency_weight(publication_year)
        citation_weight = math.log10(1 + citations)
        importance = topic_score * recency_weight * citation_weight
        return importance

    def extract_keywords_from_topic(self, topic_name: str, stop_words: set) -> List[str]:
        """Extract meaningful keywords from a topic name, creating better phrases."""
        import re

        # Clean and split the topic name
        cleaned = re.sub(r'[^\w\s-]', ' ', topic_name.lower())
        words = cleaned.split()

        # Filter out stop words and very short words
        keywords = []
        for word in words:
            word = word.strip('-')
            if (len(word) >= 3 and
                word not in stop_words and
                not word.isdigit()):
                keywords.append(word)

        # Create both individual keywords and meaningful phrases
        result = []

        # Add individual keywords
        result.extend(keywords[:3])  # Limit individual keywords

        # Create 2-word phrases if we have enough words
        if len(keywords) >= 2:
            phrases = []
            for i in range(min(len(keywords)-1, 2)):  # Max 2 phrases
                phrase = f"{keywords[i]}_{keywords[i+1]}"
                phrases.append(phrase)
            result.extend(phrases)

        # If we have 3+ words, create one longer phrase
        if len(keywords) >= 3:
            long_phrase = "_".join(keywords[:3])
            result.append(long_phrase)

        return result

    def create_weighted_expertise_document(self, researcher_papers: pd.DataFrame) -> str:
        """Create weighted expertise document with better keyword distribution."""
        import re

        # Common stop words to remove
        stop_words = {'and', 'in', 'of', 'for', 'the', 'a', 'an', 'to', 'with', 'on', 'at', 'by'}

        # Collect topics with their importance scores
        weighted_topics = []

        for _, paper in researcher_papers.iterrows():
            topics = self.parse_curated_topics(paper['curated_topics'])
            publication_year = paper['publication_year']
            citations = paper['cited_by_count'] if pd.notna(paper['cited_by_count']) else 0

            for topic in topics:
                if topic.get('type') == 'topic':
                    topic_score = topic.get('score', 0)
                    topic_name = topic.get('name', '').strip()

                    if topic_name and topic_score > 0:
                        # Calculate importance score
                        importance = self.calculate_topic_importance_score(
                            topic_score, publication_year, citations
                        )

                        # Extract keywords and phrases
                        keywords = self.extract_keywords_from_topic(topic_name, stop_words)

                        for keyword in keywords:
                            weighted_topics.append((keyword, importance))

        # Sort by importance and remove duplicates while preserving weights
        topic_weights = {}
        for keyword, importance in weighted_topics:
            if keyword in topic_weights:
                topic_weights[keyword] += importance
            else:
                topic_weights[keyword] = importance

        # Sort by weight and select top topics
        sorted_topics = sorted(topic_weights.items(), key=lambda x: x[1], reverse=True)

        # Create document with weighted representation
        # Instead of repeating words, we'll include them based on importance tiers
        document_words = []

        # Tier 1: Top 20% - include multiple times but not excessively
        tier1_count = max(1, len(sorted_topics) // 5)
        for keyword, weight in sorted_topics[:tier1_count]:
            # Include 2-3 times for top tier
            repeat_count = min(3, max(1, int(weight / max(1, sorted_topics[0][1]) * 3)))
            document_words.extend([keyword] * repeat_count)

        # Tier 2: Next 30% - include once or twice
        tier2_start = tier1_count
        tier2_end = tier2_start + max(1, len(sorted_topics) * 3 // 10)
        for keyword, weight in sorted_topics[tier2_start:tier2_end]:
            repeat_count = min(2, max(1, int(weight / max(1, sorted_topics[0][1]) * 2)))
            document_words.extend([keyword] * repeat_count)

        # Tier 3: Remaining - include once
        tier3_start = tier2_end
        for keyword, weight in sorted_topics[tier3_start:tier3_start + 20]:  # Limit total
            document_words.append(keyword)

        # Shuffle to avoid patterns and create more natural distribution
        import random
        random.seed(42)  # For reproducibility
        random.shuffle(document_words)

        # Join with commas for better readability and structure
        document = ', '.join(document_words)

        # Fallback if document is too short
        if len(document.split()) < 5:
            fallback_keywords = []
            for _, paper in researcher_papers.iterrows():
                topics = self.parse_curated_topics(paper['curated_topics'])
                for topic in topics:
                    if topic.get('type') == 'topic':
                        topic_name = topic.get('name', '').strip()
                        if topic_name:
                            keywords = self.extract_keywords_from_topic(topic_name, stop_words)
                            fallback_keywords.extend(keywords)

            if fallback_keywords:
                # Remove duplicates while preserving order
                unique_keywords = list(dict.fromkeys(fallback_keywords))
                document = ', '.join(unique_keywords[:20])

        return document

    def create_proxy_abstract(self, topics_str: str) -> str:
        """
        Create proxy abstract from high-quality topics when abstract is missing.
        Uses type: 'topic', 'related_topic', 'concept' with scores > min_proxy_score.
        """
        topics = self.parse_curated_topics(topics_str)
        if not topics:
            return ""

        relevant_topics = []
        for topic in topics:
            topic_type = topic.get('type', '')
            topic_score = topic.get('score', 0)
            topic_name = topic.get('name', '').strip()

            if (topic_type in ['topic', 'related_topic', 'concept'] and
                topic_score >= self.min_proxy_score and topic_name):
                relevant_topics.append(topic_name)

        return '. '.join(relevant_topics)

    def create_conceptual_profiles(self, df: pd.DataFrame) -> Tuple[Dict[str, np.ndarray], Dict[str, int]]:
        """
        Create conceptual profiles using sentence transformers.
        Handles missing abstracts with proxy abstracts.
        """
        self.logger.info("Loading sentence transformer model...")
        if self.sentence_model is None:
            self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

        self.logger.info("Creating conceptual profiles...")

        # Track statistics
        missing_abstracts = 0
        total_papers = len(df)

        texts_to_embed = []
        work_ids = []

        for _, paper in df.iterrows():
            title = paper['work_title'] if pd.notna(paper['work_title']) else ""
            abstract = paper['abstract'] if pd.notna(paper['abstract']) else ""

            if abstract.strip():
                # Use title + abstract
                text_to_embed = f"{title}. {abstract}"
            else:
                # Create proxy abstract
                missing_abstracts += 1
                proxy_abstract = self.create_proxy_abstract(paper['curated_topics'])
                text_to_embed = f"{title}. {proxy_abstract}" if proxy_abstract else title

            texts_to_embed.append(text_to_embed)
            work_ids.append(paper['work_id'])

        self.logger.info(f"Missing abstracts: {missing_abstracts}/{total_papers} ({missing_abstracts/total_papers*100:.1f}%)")

        # Generate embeddings
        self.logger.info("Generating embeddings...")
        embeddings = self.sentence_model.encode(texts_to_embed, show_progress_bar=True)

        # Create mapping from work_id to embedding
        conceptual_profiles = {}
        work_id_to_index = {}
        for i, work_id in enumerate(work_ids):
            conceptual_profiles[work_id] = embeddings[i]
            work_id_to_index[work_id] = i

        return conceptual_profiles, work_id_to_index

    def create_grant_experience_factors(self, df: pd.DataFrame, grants_df: pd.DataFrame) -> Dict[str, float]:
        """
        Calculate grant experience factors: F_ge = (pi_7_years × 1.0) + (copi_7_years × 0.5)
        """
        self.logger.info("Creating grant experience factors...")

        # Get unique researchers
        researchers = df[['researcher_name', 'researcher_openalex_id']].drop_duplicates()

        # Create mapping from openalex_id to grant experience
        grant_experience = {}
        for _, researcher in researchers.iterrows():
            openalex_id = researcher['researcher_openalex_id']

            # Find matching grant record
            grant_record = grants_df[grants_df['openalex_id'] == openalex_id]

            if not grant_record.empty:
                pi_grants = grant_record.iloc[0]['pi_7_years'] if pd.notna(grant_record.iloc[0]['pi_7_years']) else 0
                copi_grants = grant_record.iloc[0]['copi_7_years'] if pd.notna(grant_record.iloc[0]['copi_7_years']) else 0
                f_ge = (pi_grants * 1.0) + (copi_grants * 0.5)
            else:
                f_ge = 0.0  # No grant history

            grant_experience[openalex_id] = f_ge

        self.logger.info(f"Created grant experience factors for {len(grant_experience)} researchers")
        return grant_experience

    def create_evidence_index(self, df: pd.DataFrame) -> Dict[str, Dict[str, List[str]]]:
        """
        Create evidence index mapping: researcher_id -> {topic_name: [work_ids]}
        Only uses topics with type: 'topic'.
        """
        self.logger.info("Creating evidence index...")

        evidence_index = {}

        for _, paper in df.iterrows():
            researcher_id = paper['researcher_openalex_id']
            work_id = paper['work_id']
            topics = self.parse_curated_topics(paper['curated_topics'])

            if researcher_id not in evidence_index:
                evidence_index[researcher_id] = {}

            for topic in topics:
                if topic.get('type') == 'topic':
                    topic_name = topic.get('name', '').strip()
                    if topic_name:
                        if topic_name not in evidence_index[researcher_id]:
                            evidence_index[researcher_id][topic_name] = []
                        evidence_index[researcher_id][topic_name].append(work_id)

        self.logger.info(f"Created evidence index for {len(evidence_index)} researchers")
        return evidence_index

    def comma_tokenizer(self, text: str) -> List[str]:
        """Custom tokenizer that splits on commas and cleans tokens"""
        return [token.strip() for token in text.split(',') if token.strip()]

    def create_tfidf_profiles(self, df: pd.DataFrame) -> Tuple[Dict[str, np.ndarray], TfidfVectorizer]:
        """
        Create TF-IDF profiles from weighted expertise documents.
        """
        self.logger.info("Creating TF-IDF profiles...")

        # Get unique researchers
        researchers = df[['researcher_name', 'researcher_openalex_id']].drop_duplicates()

        # Create weighted expertise documents
        expertise_documents = []
        researcher_ids = []
        empty_docs = 0

        for _, researcher in researchers.iterrows():
            researcher_id = researcher['researcher_openalex_id']
            researcher_papers = df[df['researcher_openalex_id'] == researcher_id]
            expertise_doc = self.create_weighted_expertise_document(researcher_papers)

            # Check if document is empty or too short
            if not expertise_doc.strip() or len(expertise_doc.split()) < 2:
                empty_docs += 1
                # Create a minimal document from researcher's top topics
                all_topics = []
                for _, paper in researcher_papers.iterrows():
                    topics = self.parse_curated_topics(paper['curated_topics'])
                    for topic in topics:
                        if topic.get('type') == 'topic':
                            topic_name = topic.get('name', '').strip()
                            if topic_name:
                                all_topics.append(topic_name)

                # Use unique topics as fallback
                expertise_doc = ', '.join(list(set(all_topics))[:10]) if all_topics else "research, unknown"

            expertise_documents.append(expertise_doc)
            researcher_ids.append(researcher_id)

        self.logger.info(f"Created {len(expertise_documents)} documents, {empty_docs} required fallback")

        # Fit TF-IDF model with appropriate parameters for comma-separated format
        self.logger.info("Fitting TF-IDF model...")
        self.tfidf_model = TfidfVectorizer(
            max_features=5000,
            ngram_range=(1, 2),  # Include both unigrams and bigrams
            min_df=2,  # Require at least 2 documents
            max_df=0.9,
            stop_words='english',
            lowercase=True,
            token_pattern=r'(?u)\b[a-zA-Z_][a-zA-Z0-9_]*\b',  # Handle underscore-separated phrases
            tokenizer=self.comma_tokenizer  # Use the class method instead of lambda
        )

        try:
            tfidf_matrix = self.tfidf_model.fit_transform(expertise_documents)
        except ValueError as e:
            self.logger.warning(f"TF-IDF fitting failed: {e}")
            # Fallback: more lenient parameters
            self.tfidf_model = TfidfVectorizer(
                max_features=2000,
                ngram_range=(1, 1),
                min_df=1,
                max_df=0.95,
                stop_words='english',
                lowercase=True,
                tokenizer=self.comma_tokenizer  # Use the class method instead of lambda
            )
            tfidf_matrix = self.tfidf_model.fit_transform(expertise_documents)

        # Debug: Print sample documents
        for i, (doc, researcher_id) in enumerate(zip(expertise_documents[:3], researcher_ids[:3])):
            print(f"\nDEBUG - Sample document {i}:")
            print(f"Researcher: {researcher_id}")
            print(f"Document: {doc[:200]}...")
            print(f"Unique words: {len(set(doc.split()))}")
            print(f"First 10 words: {doc.split()[:10]}")

        # Create mapping from researcher_id to TF-IDF vector
        researcher_vectors = {}
        for i, researcher_id in enumerate(researcher_ids):
            researcher_vectors[researcher_id] = tfidf_matrix[i].toarray().flatten()

        self.logger.info(f"Created TF-IDF profiles for {len(researcher_vectors)} researchers")
        self.logger.info(f"TF-IDF vocabulary size: {len(self.tfidf_model.get_feature_names_out())}")

        return researcher_vectors, self.tfidf_model

    def create_researcher_metadata(self, df: pd.DataFrame, grant_experience: Dict[str, float]) -> pd.DataFrame:
        """
        Create researcher metadata dataframe with summary statistics.
        """
        self.logger.info("Creating researcher metadata...")

        # Aggregate researcher-level statistics
        researcher_stats = df.groupby(['researcher_name', 'researcher_openalex_id']).agg({
            'work_id': 'count',
            'cited_by_count': ['sum', 'mean', 'max'],
            'publication_year': ['min', 'max']
        }).reset_index()

        # Flatten column names
        researcher_stats.columns = ['researcher_name', 'researcher_openalex_id',
                                   'total_papers', 'total_citations', 'avg_citations',
                                   'max_citations', 'first_publication_year', 'last_publication_year']

        # Add grant experience factors
        researcher_stats['grant_experience_factor'] = researcher_stats['researcher_openalex_id'].map(grant_experience).fillna(0.0)

        # Calculate additional metrics
        researcher_stats['years_active'] = researcher_stats['last_publication_year'] - researcher_stats['first_publication_year'] + 1
        researcher_stats['recent_activity'] = researcher_stats['last_publication_year'] >= (self.current_year - 5)

        return researcher_stats

    def save_processed_data(self, output_dir: str, researcher_vectors: Dict[str, np.ndarray],
                          conceptual_profiles: Dict[str, np.ndarray],
                          work_id_to_index: Dict[str, int],
                          evidence_index: Dict[str, Dict[str, List[str]]],
                          researcher_metadata: pd.DataFrame) -> Dict[str, Any]:
        """
        Save all processed data to specified directory structure.
        """
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        self.logger.info(f"Saving processed data to {output_path}")

        # Save TF-IDF model
        with open(output_path / 'tfidf_model.pkl', 'wb') as f:
            pickle.dump(self.tfidf_model, f)

        # Save researcher TF-IDF vectors
        researcher_ids = list(researcher_vectors.keys())
        vectors_array = np.array([researcher_vectors[rid] for rid in researcher_ids])

        np.savez_compressed(
            output_path / 'researcher_vectors.npz',
            vectors=vectors_array,
            researcher_ids=researcher_ids
        )

        # Save conceptual profiles (paper embeddings)
        work_ids = list(conceptual_profiles.keys())
        embeddings_array = np.array([conceptual_profiles[wid] for wid in work_ids])

        np.savez_compressed(
            output_path / 'conceptual_profiles.npz',
            embeddings=embeddings_array,
            work_ids=work_ids,
            work_id_to_index=work_id_to_index
        )

        # Save evidence index
        with open(output_path / 'evidence_index.json', 'w') as f:
            json.dump(evidence_index, f, indent=2)

        # Save researcher metadata
        researcher_metadata.to_parquet(output_path / 'researcher_metadata.parquet', index=False)

        # Save processing log
        processing_log = {
            'processing_timestamp': datetime.now().isoformat(),
            'current_year': self.current_year,
            'min_proxy_score': self.min_proxy_score,
            'total_researchers': len(researcher_vectors),
            'total_papers': len(conceptual_profiles),
            'tfidf_features': len(self.tfidf_model.get_feature_names_out()) if self.tfidf_model else 0,
            'sentence_model': self.sentence_model.get_sentence_embedding_dimension() if self.sentence_model else 0,
            'files_created': [
                'tfidf_model.pkl',
                'researcher_vectors.npz',
                'conceptual_profiles.npz',
                'evidence_index.json',
                'researcher_metadata.parquet',
                'processing_log.json'
            ]
        }

        with open(output_path / 'processing_log.json', 'w') as f:
            json.dump(processing_log, f, indent=2)

        self.logger.info("All data saved successfully!")
        return processing_log

    def process_all(self, researcher_file: str, grants_file: str, output_dir: str) -> Dict[str, Any]:
        """
        Main pipeline: process all data and save results.
        """
        self.logger.info("Starting complete data preprocessing pipeline...")

        # Load data
        df, grants_df = self.load_data(researcher_file, grants_file)

        # Create all components
        conceptual_profiles, work_id_to_index = self.create_conceptual_profiles(df)
        grant_experience = self.create_grant_experience_factors(df, grants_df)
        evidence_index = self.create_evidence_index(df)
        researcher_vectors, tfidf_model = self.create_tfidf_profiles(df)
        researcher_metadata = self.create_researcher_metadata(df, grant_experience)

        # Save everything
        processing_log = self.save_processed_data(
            output_dir, researcher_vectors, conceptual_profiles,
            work_id_to_index, evidence_index, researcher_metadata
        )

        self.logger.info("Pipeline completed successfully!")
        return processing_log

# Usage example
if __name__ == "__main__":
    # Initialize processor
    processor = ResearcherProfileProcessor(current_year=2025, min_proxy_score=0.8)

    # File paths
    researcher_file = '/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_and_abstracts_robust_20250703_191556.csv'
    grants_file = '/content/drive/MyDrive/datastore/v2_data/texas_state_researchers_final_analysis.csv'
    output_dir = '/content/drive/MyDrive/datastore/v2_DATA'

    # Run complete pipeline
    log = processor.process_all(researcher_file, grants_file, output_dir)
    print("Processing completed!")
    print(f"Created profiles for {log['total_researchers']} researchers")
    print(f"Processed {log['total_papers']} papers")
    print(f"Output saved to: {output_dir}")

# Diagnostic Tests

In [ ]:
# Show me what the curated_topics actually look like
df = pd.read_csv('/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_and_abstracts_robust_20250703_191556.csv')
print("Sample curated_topics:")
print(df['curated_topics'].iloc[0])
print("\nFirst few work_ids:")
print(df['work_id'].head())

In [ ]:
# Test weighted document creation for one researcher
processor = ResearcherProfileProcessor()
researcher_papers = df[df['researcher_openalex_id'] == df['researcher_openalex_id'].iloc[0]]
doc = processor.create_weighted_expertise_document(researcher_papers)
print(f"Document length: {len(doc)}")
print(f"Document: {doc[:200]}...")

In [ ]:
# Test if topics parsing works
import json
sample_topics = df['curated_topics'].iloc[0]
try:
    parsed = json.loads(sample_topics)
    print(f"Parsed topics count: {len(parsed)}")
    print("First topic:", parsed[0] if parsed else "None")

    # Check for type: 'topic'
    topic_types = [t.get('type') for t in parsed if isinstance(t, dict)]
    print(f"Topic types: {set(topic_types)}")
except:
    print("JSON parsing failed")

In [ ]:
import json
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_and_abstracts_robust_20250703_191556.csv')

print("=== JSON PARSING TEST ===")
failed_count = 0
valid_count = 0
valid_topics_count = 0

for i in range(min(50, len(df))):  # Test first 50 rows
    topics_str = df['curated_topics'].iloc[i]
    try:
        parsed = json.loads(topics_str)
        valid_count += 1

        # Count topics with type='topic'
        topic_types = [t.get('type') for t in parsed if isinstance(t, dict)]
        if 'topic' in topic_types:
            valid_topics_count += 1

    except Exception as e:
        failed_count += 1
        if failed_count <= 3:  # Show first 3 failures
            print(f"Row {i} failed: {str(e)[:100]}")

print(f"Valid JSON: {valid_count}")
print(f"Failed JSON: {failed_count}")
print(f"Rows with 'topic' type: {valid_topics_count}")

In [ ]:
# Test the fixed parser
import ast
import json
import pandas as pd

def parse_curated_topics_fixed(topics_str):
    """Fixed parser that handles Python dict strings"""
    if pd.isna(topics_str) or not topics_str.strip():
        return []

    try:
        # Try JSON first
        topics = json.loads(topics_str)
        return topics if isinstance(topics, list) else []
    except:
        try:
            # Try ast.literal_eval for Python dict strings
            topics = ast.literal_eval(topics_str)
            return topics if isinstance(topics, list) else []
        except:
            try:
                # Fix single quotes and try JSON
                fixed_json = topics_str.replace("'", '"')
                topics = json.loads(fixed_json)
                return topics if isinstance(topics, list) else []
            except:
                return []

# Test with your data
df = pd.read_csv('/content/drive/MyDrive/datastore/v2_data/researcher_works_with_topics_and_abstracts_robust_20250703_191556.csv')

print("=== TESTING FIXED PARSER ===")
success_count = 0
topic_count = 0

for i in range(min(10, len(df))):
    topics_str = df['curated_topics'].iloc[i]
    parsed = parse_curated_topics_fixed(topics_str)

    if parsed:
        success_count += 1
        # Count topics with type='topic'
        main_topics = [t for t in parsed if t.get('type') == 'topic']
        topic_count += len(main_topics)
        print(f"Row {i}: {len(parsed)} total topics, {len(main_topics)} main topics")

print(f"\nSuccess rate: {success_count}/10")
print(f"Total main topics found: {topic_count}")

In [ ]:
print("=== PUBLICATION YEAR ANALYSIS ===")
years = df['publication_year'].dropna()
print(f"Year range: {years.min()} to {years.max()}")
print(f"Years distribution:")
print(years.value_counts().sort_index().tail(10))

# Test recency weights
current_year = 2025
for year in [2000, 2010, 2015, 2020, 2023, 2024, 2025]:
    weight = max(0.1, 1 - (current_year - year) / 15)
    print(f"Year {year}: recency weight = {weight:.3f}")

# Solicilitation Parser

In [ ]:
!pip install PyMuPDF

In [ ]:
!pip install anthropic

In [ ]:
# ==============================================================================
# Enhanced Intelligent Solicitation Processor
# ==============================================================================
import fitz  # PyMuPDF
import pandas as pd
import json
import re
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Any
from google.colab import drive, userdata
from transformers import pipeline
import anthropic
from dataclasses import dataclass, asdict
import warnings
import os
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class EnhancedSolicitationObject:
    """
    Enhanced structured object containing comprehensive solicitation analysis.
    """
    # Basic Metadata
    solicitation_id: str
    title: str
    abstract: str
    processed_at: str
    pdf_filename: str

    # Detailed Structured Information
    award_info: Dict[str, Any]  # funding_ceiling, award_type, duration, etc.
    eligibility: Dict[str, Any]  # pi_requirements, institutional_limits, etc.
    submission_requirements: Dict[str, Any]  # deadlines, format_rules, etc.
    program_objectives: List[str]  # scientific goals and priorities
    contact_information: Dict[str, str]  # agency contacts

    # Skills Analysis (backward compatible)
    narrative_skills: List[str]  # From Claude targeted extraction
    formal_topics: List[Dict]    # From OpenAlex classifier
    required_skills_checklist: List[str]  # Final combined skills

    # Processing Metadata
    text_length: int
    sections_found: List[str]
    extraction_confidence: Dict[str, float]
    processing_method: str = "intelligent_deconstruction"
    full_text_context: str = ""  # Complete text for context

    def to_dict(self):
        """Convert to dictionary for JSON serialization."""
        return asdict(self)

    def to_json(self, filepath: str):
        """Save to JSON file."""
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(self.to_dict(), f, indent=2, ensure_ascii=False)

class IntelligentSolicitationProcessor:
    """
    Enhanced processor that intelligently deconstructs solicitations into structured sections
    and extracts comprehensive information using targeted prompts.
    """

    # Section detection patterns for different solicitation types
    SECTION_PATTERNS = {
        'award_info': [
            r'Section\s+II[:\s]*Award\s+Information',
            r'Award\s+Information',
            r'Funding\s+Information',
            r'Budget\s+Information',
            r'Financial\s+Information',
            r'Award\s+Details'
        ],
        'eligibility': [
            r'Section\s+III[:\s]*Eligibility',
            r'Section\s+IV[:\s]*Eligibility',
            r'Eligibility\s+Requirements',
            r'Eligibility\s+Information',
            r'Who\s+May\s+Apply',
            r'Applicant\s+Eligibility',
            r'PI\s+Eligibility'
        ],
        'program_description': [
            r'Program\s+Description',
            r'Scientific\s+Objectives',
            r'Research\s+Areas',
            r'Technical\s+Areas',
            r'Program\s+Objectives',
            r'Research\s+Goals',
            r'Scientific\s+Scope',
            r'Program\s+Overview'
        ],
        'submission_requirements': [
            r'Section\s+V[:\s]*Proposal.*Submission',
            r'Section\s+IV[:\s]*Application.*Submission',
            r'Application\s+Instructions',
            r'Submission\s+Instructions',
            r'Proposal\s+Requirements',
            r'Application\s+Requirements',
            r'Submission\s+Guidelines'
        ],
        'contact_info': [
            r'Section\s+VIII[:\s]*Agency\s+Contacts',
            r'Agency\s+Contacts',
            r'Contact\s+Information',
            r'Program\s+Contacts',
            r'For\s+More\s+Information'
        ]
    }

    def __init__(self):
        self.claude_client = None
        self.topic_classifier = None
        self.setup_models()

    def setup_models(self):
        """Initialize Claude API client and OpenAlex topic classifier."""
        logger.info("Setting up models...")
        try:
            api_key = userdata.get('ANTHROPIC_API_KEY')
            self.claude_client = anthropic.Anthropic(api_key=api_key)
            logger.info("✅ Claude API client initialized")
        except Exception as e:
            logger.warning(f"⚠️ Claude API setup failed: {e}")

        try:
            logger.info("Loading OpenAlex topic classifier...")
            self.topic_classifier = pipeline(
                "text-classification",
                model="OpenAlex/bert-base-multilingual-cased-finetuned-openalex-topic-classification-title-abstract"
            )
            logger.info("✅ OpenAlex topic classifier loaded")
        except Exception as e:
            logger.warning(f"⚠️ Topic classifier setup failed: {e}")

    def _extract_and_clean_text(self, filepath: str) -> Tuple[str, str, str, str]:
        """Enhanced text extraction with better cleaning and formatting."""
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"File not found: {filepath}")

        filename = os.path.basename(filepath)
        logger.info(f"📄 Processing: {filename}")

        doc = None
        try:
            doc = fitz.open(filepath)
            page_count = len(doc)
            full_text = ""

            logger.info(f"📖 PDF has {page_count} pages")

            for page_num in range(page_count):
                try:
                    page = doc[page_num]
                    text = page.get_text()

                    # Basic cleaning
                    text = self._clean_extracted_text(text)
                    full_text += f"\n--- PAGE {page_num + 1} ---\n{text}\n"

                except Exception as page_error:
                    logger.warning(f"⚠️ Error processing page {page_num + 1}: {page_error}")
                    continue

            if not full_text.strip():
                raise ValueError("Extracted text is empty. PDF might be image-based.")

            title, abstract = self._extract_title_and_abstract(full_text, filename)
            cleaned_text = self._final_text_cleaning(full_text)

            logger.info(f"✅ Extracted {len(cleaned_text)} characters from {page_count} pages")
            return filename, title, abstract, cleaned_text

        except Exception as e:
            logger.error(f"❌ Error extracting text from PDF '{filename}': {e}")
            raise
        finally:
            # Ensure document is properly closed
            if doc is not None:
                try:
                    doc.close()
                except:
                    pass

    def _final_text_cleaning(self, text: str) -> str:
        """Final pass text cleaning for analysis."""
        # Remove page markers
        text = re.sub(r'--- PAGE \d+ ---', '', text)

        # Normalize spacing
        text = re.sub(r'\n{3,}', '\n\n', text)
        text = re.sub(r' {2,}', ' ', text)

        return text.strip()

    def _clean_extracted_text(self, text: str) -> str:
        """Clean and normalize extracted text."""
        # Remove excessive whitespace
        text = re.sub(r'\n\s*\n\s*\n', '\n\n', text)
        text = re.sub(r' +', ' ', text)

        # Remove page headers/footers patterns
        text = re.sub(r'\n\s*Page\s+\d+\s*of\s+\d+\s*\n', '\n', text, flags=re.IGNORECASE)
        text = re.sub(r'\n\s*\d+\s*\n', '\n', text)

        # Fix common OCR issues
        text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)  # Add space between words
        text = re.sub(r'(\w)(Section\s+[IVX]+)', r'\1\n\2', text)  # Break before sections

        return text.strip()

    def _simple_pdf_extraction(self, filepath: str) -> Tuple[str, str, str, str]:
        """Simple fallback PDF extraction method."""
        filename = os.path.basename(filepath)
        logger.info(f"🔄 Trying simple extraction for: {filename}")

        doc = None
        try:
            doc = fitz.open(filepath)

            # Extract all text at once
            full_text = ""
            for page in doc:
                full_text += page.get_text() + "\n"

            if not full_text.strip():
                raise ValueError("No text extracted from PDF")

            # Basic title and abstract extraction
            lines = [line.strip() for line in full_text.split('\n') if line.strip()]
            title = filename.replace('.pdf', '').replace('_', ' ')
            abstract = ' '.join(lines[:10]) if lines else ""

            logger.info(f"✅ Simple extraction successful: {len(full_text)} characters")
            return filename, title, abstract, full_text

        except Exception as e:
            logger.error(f"❌ Simple extraction failed: {e}")
            raise
        finally:
            if doc is not None:
                try:
                    doc.close()
                except:
                    pass
        """Final pass text cleaning for analysis."""
        # Remove page markers
        text = re.sub(r'--- PAGE \d+ ---', '', text)

        # Normalize spacing
        text = re.sub(r'\n{3,}', '\n\n', text)
        text = re.sub(r' {2,}', ' ', text)

        return text.strip()

    def _extract_title_and_abstract(self, full_text: str, filename: str) -> Tuple[str, str]:
        """Enhanced title and abstract extraction."""
        lines = [line.strip() for line in full_text.split('\n') if line.strip()]

        # Title extraction
        title = filename.replace('.pdf', '').replace('_', ' ').replace('-', ' ')

        # Look for title in first 20 lines
        for i, line in enumerate(lines[:20]):
            if (30 < len(line) < 300 and
                not line.isupper() and
                not re.match(r'^(Page|Section|\d+)', line, re.IGNORECASE) and
                not line.startswith('http')):
                title = line
                break

        # Abstract extraction
        abstract = ""
        abstract_started = False
        abstract_markers = ['abstract', 'summary', 'overview', 'program description']

        for i, line in enumerate(lines):
            line_lower = line.lower()

            # Start abstract detection
            if not abstract_started:
                for marker in abstract_markers:
                    if marker in line_lower and len(line) > len(marker) + 5:
                        abstract_started = True
                        # Extract text after marker
                        marker_pos = line_lower.find(marker)
                        remaining_text = line[marker_pos + len(marker):].strip()
                        if remaining_text and not remaining_text.startswith(':'):
                            abstract += remaining_text + " "
                        break
                continue

            # Continue abstract collection
            if abstract_started:
                # Stop conditions
                stop_conditions = [
                    'table of contents', 'section i', 'section 1', 'introduction',
                    'background', 'key dates', 'important dates'
                ]

                if any(stop in line_lower for stop in stop_conditions):
                    break

                abstract += line + " "

                # Stop if abstract is getting too long
                if len(abstract) > 2500:
                    break

        # Fallback for abstract
        if not abstract.strip():
            abstract = ' '.join(lines[1:8])  # Use first few lines

        return title.strip(), abstract.strip()[:2000]

    def _detect_sections(self, text: str) -> Dict[str, Tuple[int, int]]:
        """Detect and locate sections in the text."""
        sections_found = {}
        lines = text.split('\n')

        for section_type, patterns in self.SECTION_PATTERNS.items():
            for pattern in patterns:
                for i, line in enumerate(lines):
                    if re.search(pattern, line, re.IGNORECASE):
                        # Find section end (next major section or end of text)
                        end_line = len(lines)
                        for j in range(i + 1, len(lines)):
                            if re.search(r'Section\s+[IVX]+[:\s]', lines[j], re.IGNORECASE):
                                end_line = j
                                break

                        sections_found[section_type] = (i, end_line)
                        logger.info(f"Found section: {section_type} (lines {i}-{end_line})")
                        break

                if section_type in sections_found:
                    break

        return sections_found

    def _extract_section_text(self, text: str, start_line: int, end_line: int) -> str:
        """Extract text from a specific section."""
        lines = text.split('\n')
        section_lines = lines[start_line:end_line]
        return '\n'.join(section_lines).strip()

    def _extract_award_information(self, section_text: str) -> Dict[str, Any]:
        """Extract award information using Claude API."""
        if not self.claude_client or not section_text.strip():
            return {}

        prompt = f"""Extract award information from this solicitation section. Return a JSON object with these fields:
- funding_ceiling: Maximum funding amount (include currency and period if specified)
- award_type: Type of award (e.g., R01, CAREER, SBIR, etc.)
- project_duration: Maximum project length
- number_of_awards: Expected number of awards to be made
- cost_sharing: Any cost sharing requirements
- award_mechanism: Funding mechanism details

Section text:
---
{section_text}
---

Return only valid JSON. If information is not found, use null for that field."""

        try:
            response = self.claude_client.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=1000,
                temperature=0.1,
                messages=[{"role": "user", "content": prompt}]
            )

            result = json.loads(response.content[0].text.strip())
            logger.info("✅ Extracted award information")
            return result

        except Exception as e:
            logger.warning(f"⚠️ Award information extraction failed: {e}")
            return {}

    def _extract_eligibility_requirements(self, section_text: str) -> Dict[str, Any]:
        """Extract eligibility requirements using Claude API."""
        if not self.claude_client or not section_text.strip():
            return {}

        prompt = f"""Extract eligibility requirements from this solicitation section. Return a JSON object with these fields:
- pi_requirements: List of PI eligibility requirements
- institutional_requirements: Institution eligibility requirements
- limited_submission: Whether this is a limited submission opportunity
- career_stage_restrictions: Any career stage limitations (e.g., early-career only)
- collaboration_requirements: Required collaborations or partnerships
- special_restrictions: Any other special eligibility restrictions

Section text:
---
{section_text}
---

Return only valid JSON. Use empty arrays [] for missing list fields and null for missing single fields."""

        try:
            response = self.claude_client.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=1000,
                temperature=0.1,
                messages=[{"role": "user", "content": prompt}]
            )

            result = json.loads(response.content[0].text.strip())
            logger.info("✅ Extracted eligibility requirements")
            return result

        except Exception as e:
            logger.warning(f"⚠️ Eligibility extraction failed: {e}")
            return {}

    def _extract_submission_requirements(self, section_text: str) -> Dict[str, Any]:
        """Extract submission requirements using Claude API."""
        if not self.claude_client or not section_text.strip():
            return {}

        prompt = f"""Extract submission requirements from this solicitation section. Return a JSON object with these fields:
- submission_deadline: Application deadline(s)
- page_limits: Page limits for different sections
- format_requirements: File format requirements (PDF, font size, margins, etc.)
- required_documents: List of required documents/sections
- submission_method: How to submit (Grants.gov, etc.)
- special_instructions: Any special submission instructions

Section text:
---
{section_text}
---

Return only valid JSON. Use empty arrays [] for missing list fields and null for missing single fields."""

        try:
            response = self.claude_client.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=1000,
                temperature=0.1,
                messages=[{"role": "user", "content": prompt}]
            )

            result = json.loads(response.content[0].text.strip())
            logger.info("✅ Extracted submission requirements")
            return result

        except Exception as e:
            logger.warning(f"⚠️ Submission requirements extraction failed: {e}")
            return {}

    def _extract_program_objectives_and_skills(self, section_text: str) -> Tuple[List[str], List[str]]:
        """Extract program objectives and required skills using Claude API."""
        if not self.claude_client or not section_text.strip():
            return [], []

        prompt = f"""Analyze this program description section and extract two things:

1. Program Objectives: The main scientific goals and research priorities
2. Required Skills: Specific technical skills, domain expertise, and methodological knowledge needed

Return a JSON object with:
- program_objectives: Array of 3-7 main program goals/objectives
- required_skills: Array of 5-10 specific skills/expertise areas needed

Section text:
---
{section_text}
---

Return only valid JSON with arrays of strings."""

        try:
            response = self.claude_client.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=1500,
                temperature=0.2,
                messages=[{"role": "user", "content": prompt}]
            )

            result = json.loads(response.content[0].text.strip())
            objectives = result.get('program_objectives', [])
            skills = result.get('required_skills', [])

            logger.info(f"✅ Extracted {len(objectives)} objectives and {len(skills)} skills")
            return objectives, skills

        except Exception as e:
            logger.warning(f"⚠️ Program objectives extraction failed: {e}")
            return [], []

    def _extract_contact_information(self, section_text: str) -> Dict[str, str]:
        """Extract contact information using Claude API."""
        if not self.claude_client or not section_text.strip():
            return {}

        prompt = f"""Extract contact information from this solicitation section. Return a JSON object with:
- program_officer: Name and contact of program officer
- technical_contact: Technical questions contact
- administrative_contact: Administrative questions contact
- general_email: General program email
- phone: Program phone number
- website: Program website

Section text:
---
{section_text}
---

Return only valid JSON. Use null for missing fields."""

        try:
            response = self.claude_client.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=800,
                temperature=0.1,
                messages=[{"role": "user", "content": prompt}]
            )

            result = json.loads(response.content[0].text.strip())
            logger.info("✅ Extracted contact information")
            return result

        except Exception as e:
            logger.warning(f"⚠️ Contact information extraction failed: {e}")
            return {}

    def _extract_formal_topics_openalex(self, title: str, abstract: str) -> List[Dict]:
        """Extract formal topics using OpenAlex classifier (unchanged from original)."""
        if not self.topic_classifier:
            logger.warning("⚠️ Topic classifier not available, skipping formal topics.")
            return []

        formatted_text = f"<TITLE> {title}\n<ABSTRACT> {abstract}"
        logger.info("🔬 Running OpenAlex topic classification...")

        try:
            predictions = self.topic_classifier(formatted_text, top_k=10, truncation=True)

            if not predictions:
                logger.warning("OpenAlex model returned no valid predictions.")
                return []

            formal_topics = []
            for topic in predictions:
                if isinstance(topic, dict) and 'label' in topic and 'score' in topic:
                    if topic['score'] > 0.01:
                        formal_topics.append({
                            'topic': topic['label'],
                            'score': round(topic['score'], 4)
                        })

            logger.info(f"✅ Extracted {len(formal_topics)} formal topics from OpenAlex.")
            return formal_topics

        except Exception as e:
            logger.warning(f"⚠️ Topic classification failed: {e}")
            return []


    def extract_keywords_from_skills(self, skills: List[str]) -> List[str]:
        """Extract keywords from solicitation skills using same logic as researcher topics."""
        import re

        stop_words = {'and', 'in', 'of', 'for', 'the', 'a', 'an', 'to', 'with', 'on', 'at', 'by',
                      'expertise', 'experience', 'knowledge', 'ability', 'skills', 'understanding',
                      'capacity', 'proficiency', 'e.g.', 'eg'}

        all_keywords = []

        for skill in skills:
            # Clean and split
            cleaned = re.sub(r'[^\w\s-]', ' ', skill.lower())
            words = cleaned.split()

            # Extract meaningful keywords
            for word in words:
                word = word.strip('-')
                if (len(word) >= 3 and
                    word not in stop_words and
                    not word.isdigit()):
                    all_keywords.append(word)

        return all_keywords

    def _fusion_logic(self, narrative_skills: List[str], formal_topics: List[Dict]) -> List[str]:
        """Combine narrative skills and formal topics, removing duplicates."""
        logger.info("🔄 Applying fusion logic...")
        combined_skills = list(narrative_skills)
        narrative_lower = ' '.join(narrative_skills).lower()

        for topic in formal_topics:
            topic_name = topic['topic'].split(': ', 1)[-1]
            is_duplicate = topic_name.lower() in narrative_lower
            if not is_duplicate:
                combined_skills.append(f"Expertise in {topic_name}")

        logger.info(f"✅ Created final checklist with {len(combined_skills)} skills.")
        return combined_skills

    def process_solicitation(self, pdf_filepath: str) -> Optional[EnhancedSolicitationObject]:
        """Main processing pipeline with intelligent deconstruction."""
        logger.info("🚀 Starting Enhanced Solicitation Processing Pipeline")
        logger.info("=" * 60)

        try:
            # Extract and clean text - try enhanced method first, then fallback
            try:
                filename, title, abstract, full_text = self._extract_and_clean_text(pdf_filepath)
            except Exception as e:
                logger.warning(f"⚠️ Enhanced extraction failed: {e}")
                logger.info("🔄 Attempting simple extraction...")
                filename, title, abstract, full_text = self._simple_pdf_extraction(pdf_filepath)

            # Detect sections
            sections_locations = self._detect_sections(full_text)
            sections_found = list(sections_locations.keys())

            logger.info(f"📊 Detected {len(sections_found)} sections: {sections_found}")

            # Initialize structured data
            award_info = {}
            eligibility = {}
            submission_requirements = {}
            program_objectives = []
            contact_info = {}
            narrative_skills = []

            # Extract information from each detected section
            if 'award_info' in sections_locations:
                start, end = sections_locations['award_info']
                section_text = self._extract_section_text(full_text, start, end)
                award_info = self._extract_award_information(section_text)

            if 'eligibility' in sections_locations:
                start, end = sections_locations['eligibility']
                section_text = self._extract_section_text(full_text, start, end)
                eligibility = self._extract_eligibility_requirements(section_text)

            if 'submission_requirements' in sections_locations:
                start, end = sections_locations['submission_requirements']
                section_text = self._extract_section_text(full_text, start, end)
                submission_requirements = self._extract_submission_requirements(section_text)

            if 'program_description' in sections_locations:
                start, end = sections_locations['program_description']
                section_text = self._extract_section_text(full_text, start, end)
                program_objectives, narrative_skills = self._extract_program_objectives_and_skills(section_text)

            if 'contact_info' in sections_locations:
                start, end = sections_locations['contact_info']
                section_text = self._extract_section_text(full_text, start, end)
                contact_info = self._extract_contact_information(section_text)

            # Fallback: if no program description found, analyze full text for skills
            if not narrative_skills:
                logger.info("🔄 No program description found, analyzing full text for skills...")
                fallback_text = f"Title: {title}. Abstract: {abstract}"
                if len(fallback_text.strip()) < 100:  # If abstract is too short, use more text
                    fallback_text = full_text[:3000]  # Use first 3000 chars
                _, narrative_skills = self._extract_program_objectives_and_skills(fallback_text)

            # OpenAlex classification
            formal_topics = self._extract_formal_topics_openalex(title, abstract)

            # Fusion logic
            required_skills_checklist = self._fusion_logic(narrative_skills, formal_topics)

            # Calculate extraction confidence
            extraction_confidence = {
                'award_info': 1.0 if award_info else 0.0,
                'eligibility': 1.0 if eligibility else 0.0,
                'submission_requirements': 1.0 if submission_requirements else 0.0,
                'program_objectives': 1.0 if program_objectives else 0.0,
                'contact_info': 1.0 if contact_info else 0.0,
                'overall': len(sections_found) / len(self.SECTION_PATTERNS)
            }

            # Create enhanced object
            solicitation_obj = EnhancedSolicitationObject(
                solicitation_id=f"SOL_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
                title=title,
                abstract=abstract,
                processed_at=datetime.now().isoformat(),
                pdf_filename=filename,
                award_info=award_info,
                eligibility=eligibility,
                submission_requirements=submission_requirements,
                program_objectives=program_objectives,
                contact_information=contact_info,
                narrative_skills=narrative_skills,
                formal_topics=formal_topics,
                required_skills_checklist=required_skills_checklist,
                text_length=len(full_text),
                sections_found=sections_found,
                extraction_confidence=extraction_confidence,
                full_text_context=full_text[:10000]  # First 10k chars for context
            )

            logger.info("✅ Enhanced processing completed successfully!")
            return solicitation_obj

        except Exception as e:
            logger.error(f"❌ Fatal error during processing: {e}")
            return None

    def display_enhanced_results(self, solicitation_obj: Optional[EnhancedSolicitationObject]):
        """Display comprehensive processing results."""
        if not solicitation_obj:
            print("\nNo results to display due to processing error.")
            return

        print("\n" + "="*80)
        print("📋 ENHANCED SOLICITATION ANALYSIS RESULTS")
        print("="*80)

        print(f"🆔 ID: {solicitation_obj.solicitation_id}")
        print(f"📄 File: {solicitation_obj.pdf_filename}")
        print(f"📝 Title: {solicitation_obj.title}")
        print(f"🎯 Sections Found: {', '.join(solicitation_obj.sections_found)}")
        print(f"📊 Overall Confidence: {solicitation_obj.extraction_confidence['overall']:.1%}")

        # Award Information
        if solicitation_obj.award_info:
            print(f"\n💰 Award Information:")
            for key, value in solicitation_obj.award_info.items():
                if value:
                    print(f"   • {key.replace('_', ' ').title()}: {value}")

        # Eligibility
        if solicitation_obj.eligibility:
            print(f"\n👥 Eligibility Requirements:")
            for key, value in solicitation_obj.eligibility.items():
                if value:
                    print(f"   • {key.replace('_', ' ').title()}: {value}")

        # Program Objectives
        if solicitation_obj.program_objectives:
            print(f"\n🎯 Program Objectives:")
            for i, obj in enumerate(solicitation_obj.program_objectives, 1):
                print(f"   {i}. {obj}")

        # Skills Analysis
        print(f"\n🧠 Required Skills (Narrative):")
        for i, skill in enumerate(solicitation_obj.narrative_skills, 1):
            print(f"   {i}. {skill}")

        print(f"\n🔬 Formal Topics (OpenAlex):")
        for i, topic in enumerate(solicitation_obj.formal_topics, 1):
            print(f"   {i}. {topic['topic']} (Score: {topic['score']:.3f})")

        print(f"\n✅ Final Skills Checklist:")
        for i, skill in enumerate(solicitation_obj.required_skills_checklist, 1):
            print(f"   {i}. {skill}")

        # Contact Information
        if solicitation_obj.contact_information:
            print(f"\n📞 Contact Information:")
            for key, value in solicitation_obj.contact_information.items():
                if value:
                    print(f"   • {key.replace('_', ' ').title()}: {value}")

        print("\n" + "="*80)

# ==============================================================================
# Usage Example
# ==============================================================================

def process_solicitation_with_enhanced_pipeline(pdf_path: str, output_dir: str = None):
    """
    Process a solicitation PDF with the enhanced intelligent pipeline.

    Args:
        pdf_path: Path to the PDF file
        output_dir: Directory to save results (optional)

    Returns:
        EnhancedSolicitationObject or None
    """

    # Initialize processor
    processor = IntelligentSolicitationProcessor()

    # Process solicitation
    result = processor.process_solicitation(pdf_path)

    if result:
        # Save results
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            base_filename = Path(pdf_path).stem
            output_file = os.path.join(output_dir, f"{base_filename}_enhanced_analysis.json")
            result.to_json(output_file)
            logger.info(f"💾 Results saved to: {output_file}")

        # Display results
        processor.display_enhanced_results(result)

    return result

# ==============================================================================
# Main Execution
# ==============================================================================

if __name__ == "__main__":
    warnings.filterwarnings('ignore')

    # Configuration
    DRIVE_MOUNT_PATH = '/content/drive'
    DATASTORE_PATH = "/content/drive/MyDrive/datastore/"
    PDF_FILE_PATH = "/content/drive/MyDrive/datastore/v1_analysis/NSF 23-506_ Expanding AI Innovation through Capacity Building and Partnerships (ExpandAI) _ NSF - National Science Foundation.pdf"

    print("🚀 Starting Enhanced Intelligent Solicitation Processing")
    print("=" * 80)

    try:
        # Mount Google Drive if needed
        if not os.path.exists(DRIVE_MOUNT_PATH):
            drive.mount(DRIVE_MOUNT_PATH, force_remount=True)
            logger.info("✅ Google Drive mounted successfully")
        else:
            logger.info("✅ Google Drive already mounted")

        # Generate run ID
        base_filename = os.path.basename(PDF_FILE_PATH)
        run_id = re.sub(r'[^a-zA-Z0-9_-]', '_', os.path.splitext(base_filename)[0])
        logger.info(f"🆔 Run ID: {run_id}")

        # Process solicitation
        result = process_solicitation_with_enhanced_pipeline(
            pdf_path=PDF_FILE_PATH,
            output_dir=DATASTORE_PATH
        )

        if result:
            print("\n✅ Enhanced processing completed successfully!")
            print("Ready for Phase 2: Hybrid Search & Matching!")
        else:
            print("❌ Processing failed - cannot proceed to matching phase")

    except Exception as e:
        logger.error(f"❌ Critical error: {e}")
        print("❌ Pipeline failed - check logs for details")

In [ ]:
# ==============================================================================
# COMPLETE INTEGRATION RUNNER
# ==============================================================================
import json
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# STEP 1: Configure Paths (UPDATE THESE IF NEEDED)
# ==============================================================================

# Paths to your data
DATASTORE_PATH = "/content/drive/MyDrive/datastore"
RESEARCHER_DATA_DIR = "/content/drive/MyDrive/datastore/v2_analysis"  # Your preprocessed data
PDF_FILE_PATH = "/content/drive/MyDrive/datastore/v1_analysis/NSF 23-506_ Expanding AI Innovation through Capacity Building and Partnerships (ExpandAI) _ NSF - National Science Foundation.pdf"

# Generated file paths (these are created automatically)
base_filename = os.path.splitext(os.path.basename(PDF_FILE_PATH))[0]
run_id = base_filename.replace(" ", "_").replace(":", "").replace("|", "").replace("-", "_")
SOLICITATION_JSON_PATH = "/content/drive/MyDrive/datastore/NSF 23-506_ Expanding AI Innovation through Capacity Building and Partnerships (ExpandAI) _ NSF - National Science Foundation_enhanced_analysis.json"

print("🎯 INTEGRATION RUNNER - COMPLETE PIPELINE")
print("=" * 60)
print(f"📄 PDF: {os.path.basename(PDF_FILE_PATH)}")
print(f"🧠 Researcher Data: {RESEARCHER_DATA_DIR}")
print(f"📊 Expected Solicitation JSON: {os.path.basename(SOLICITATION_JSON_PATH)}")
print()

# ==============================================================================
# STEP 2: Load the Enhanced Solicitation Analysis
# ==============================================================================

def load_solicitation_analysis():
    """Load the enhanced solicitation analysis from JSON."""
    print("📋 STEP 2: Loading Solicitation Analysis")
    print("-" * 40)

    if not os.path.exists(SOLICITATION_JSON_PATH):
        print(f"❌ Solicitation analysis not found at: {SOLICITATION_JSON_PATH}")
        print("\n🔧 SOLUTION: Run Phase 1 first:")
        print("   processor = IntelligentSolicitationProcessor()")
        print(f"   result = processor.process_solicitation('{PDF_FILE_PATH}')")
        return None

    try:
        with open(SOLICITATION_JSON_PATH, 'r', encoding='utf-8') as f:
            solicitation_data = json.load(f)

        print(f"✅ Loaded solicitation: {solicitation_data['title'][:60]}...")
        print(f"   📝 Skills to match: {len(solicitation_data['required_skills_checklist'])}")
        print(f"   🎯 Sections found: {len(solicitation_data['sections_found'])}")
        print(f"   💰 Award info: {'Yes' if solicitation_data['award_info'] else 'No'}")
        print()

        return solicitation_data

    except Exception as e:
        print(f"❌ Error loading solicitation analysis: {e}")
        return None

# ==============================================================================
# STEP 3: Verify Researcher Data
# ==============================================================================

def verify_researcher_data():
    """Verify that preprocessed researcher data is available."""
    print("👥 STEP 3: Verifying Researcher Data")
    print("-" * 40)

    required_files = [
        'tfidf_model.pkl',
        'researcher_vectors.npz',
        'conceptual_profiles.npz',
        'evidence_index.json',
        'researcher_metadata.parquet',
        'processing_log.json'
    ]

    missing_files = []
    for file in required_files:
        filepath = os.path.join(RESEARCHER_DATA_DIR, file)
        if not os.path.exists(filepath):
            missing_files.append(file)
        else:
            print(f"   ✅ {file}")

    if missing_files:
        print(f"\n❌ Missing required files: {missing_files}")
        print("\n🔧 SOLUTION: Run the preprocessing pipeline first:")
        print("   processor = ResearcherProfileProcessor()")
        print("   processor.process_all(researcher_file, grants_file, output_dir)")
        return False

    print(f"\n✅ All researcher data files found in {RESEARCHER_DATA_DIR}")
    print()
    return True

# ==============================================================================
# STEP 4: Run Hybrid Search
# ==============================================================================

def run_complete_matching_pipeline():
    """Run the complete matching pipeline."""
    print("🚀 STEP 4: Running Hybrid Search Engine")
    print("-" * 40)

    # Import the hybrid search engine (from the previous artifact)
    try:
        from types import SimpleNamespace

        # Create a simple object from the JSON data
        class SolicitationObject:
            def __init__(self, data):
                for key, value in data.items():
                    setattr(self, key, value)

        # Load solicitation
        solicitation_data = load_solicitation_analysis()
        if not solicitation_data:
            return None

        # Verify researcher data
        if not verify_researcher_data():
            return None

        # Create solicitation object
        solicitation_obj = SolicitationObject(solicitation_data)

        print("🔄 Initializing Hybrid Search Engine...")

        # You'll need to copy the HybridSearchEngine class here or import it
        # For now, let's create a simplified version that shows what would happen

        print("✅ Engine initialized successfully!")
        print(f"📊 Ready to analyze {len(solicitation_obj.required_skills_checklist)} skills")
        print()

        # Show what the engine would analyze
        print("🎯 SKILLS TO ANALYZE:")
        for i, skill in enumerate(solicitation_obj.required_skills_checklist, 1):
            print(f"   {i}. {skill}")

        print()
        print("💡 TO COMPLETE THE ANALYSIS:")
        print("   Copy the HybridSearchEngine class code above")
        print("   Then run: results = run_hybrid_matching_pipeline(solicitation_obj, RESEARCHER_DATA_DIR)")

        return solicitation_obj

    except Exception as e:
        print(f"❌ Error running hybrid search: {e}")
        return None

# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

if __name__ == "__main__":
    print("🎬 Starting Complete Integration Pipeline...")
    print()

    # Run the complete pipeline
    result = run_complete_matching_pipeline()
    # Run the complete pipeline

    if result:
        print("\n🎉 PIPELINE READY!")
        print("Next step: Run the hybrid search engine with the loaded data")
    else:
        print("\n❌ Pipeline setup incomplete - check the steps above")

# ==============================================================================
# QUICK START COMMANDS
# ==============================================================================

print("\n" + "="*60)
print("🚀 QUICK START GUIDE")
print("="*60)
print()
print("1️⃣ FIRST TIME SETUP:")
print("   # Run the integration script to verify everything")
print("   exec(open('integration_runner.py').read())")
print()
print("2️⃣ RUN PHASE 1 (if not done):")
print("   processor = IntelligentSolicitationProcessor()")
print(f"   result = processor.process_solicitation('{PDF_FILE_PATH}')")
print()
print("3️⃣ RUN PHASE 2 (if not done):")
print("   processor = ResearcherProfileProcessor()")
print("   processor.process_all(researcher_file, grants_file, output_dir)")
print()
print("4️⃣ RUN HYBRID SEARCH:")
print("   engine = HybridSearchEngine(RESEARCHER_DATA_DIR)")
print("   results = engine.search(solicitation_obj)")
print("   engine.display_results(results)")
print()
print("="*60)

#Final Run

In [ ]:
# ==============================================================================
# SIMPLE COMPLETE RUNNER - NO NONSENSE
# ==============================================================================
import json
import numpy as np
import pandas as pd
import pickle
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, asdict
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import multiprocessing as mp
import time
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# CONFIGURATION - CHANGE THESE PATHS
# ==============================================================================
solicitation_file = "/content/drive/MyDrive/datastore/NSF 23-506_ Expanding AI Innovation through Capacity Building and Partnerships (ExpandAI) _ NSF - National Science Foundation_enhanced_analysis.json"
researcher_data_dir = "/content/drive/MyDrive/datastore/v2_DATA"
output_dir = "/content/drive/MyDrive/datastore"

# ==============================================================================
# CORE CLASSES
# ==============================================================================
@dataclass
class ResearcherMatch:
    researcher_id: str
    researcher_name: str
    academic_expertise_score: float
    s_sparse: float
    s_dense: float
    f_ge: float
    final_affinity_score: float
    total_papers: int
    eligibility_status: str

@dataclass
class MatchingResults:
    solicitation_title: str
    eligible_researchers: int
    total_researchers: int
    top_matches: List[ResearcherMatch]
    skills_analyzed: List[str]
    processing_time_seconds: float

class SimpleHybridEngine:
    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
        self.alpha = 0.7  # TF-IDF weight
        self.beta = 0.3   # Dense weight
        self.load_data()

    def load_data(self):
        print("📂 Loading preprocessed data...")

        # Load TF-IDF model
        with open(self.data_dir / 'tfidf_model.pkl', 'rb') as f:
            self.tfidf_model = pickle.load(f)

        # Load researcher vectors
        researcher_data = np.load(self.data_dir / 'researcher_vectors.npz', allow_pickle=True)
        vectors = researcher_data['vectors']
        researcher_ids = researcher_data['researcher_ids']
        self.researcher_vectors = dict(zip(researcher_ids, vectors))

        # Load paper embeddings
        conceptual_data = np.load(self.data_dir / 'conceptual_profiles.npz', allow_pickle=True)
        embeddings = conceptual_data['embeddings']
        work_ids = conceptual_data['work_ids']
        self.conceptual_profiles = dict(zip(work_ids, embeddings))

        # Load evidence index
        with open(self.data_dir / 'evidence_index.json', 'r') as f:
            self.evidence_index = json.load(f)

        # Load metadata
        self.researcher_metadata = pd.read_parquet(self.data_dir / 'researcher_metadata.parquet')

        # Load sentence model
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

        print(f"✅ Loaded {len(self.researcher_vectors)} researchers")

        # DIAGNOSTIC: Check data quality
        self.diagnose_data_quality()

    def diagnose_data_quality(self):
        """Diagnose potential data quality issues"""
        print("\n🔍 DIAGNOSING DATA QUALITY")
        print("-" * 40)

        # Check TF-IDF model
        try:
            vocab_size = len(self.tfidf_model.get_feature_names_out())
            print(f"TF-IDF vocabulary size: {vocab_size}")

            # NEW: Show actual vocabulary
            vocab = list(self.tfidf_model.get_feature_names_out())
            print(f"First 20 TF-IDF features: {vocab[:20]}")
            print(f"Last 20 TF-IDF features: {vocab[-20:]}")

            # NEW: Test solicitation keyword matching
            test_skills = ["Expertise in artificial intelligence research areas", "Experience in AI education"]
            test_keywords = self.extract_keywords_from_skills(test_skills)
            vocab_set = set(vocab)
            matching_keywords = [kw for kw in test_keywords if kw in vocab_set]

            print(f"Test solicitation keywords: {test_keywords}")
            print(f"Keywords that match TF-IDF vocab: {matching_keywords}")

            if not matching_keywords:
                print("❌ CRITICAL: No solicitation keywords match TF-IDF vocabulary!")

        except Exception as e:
            print(f"❌ Error accessing TF-IDF vocabulary: {e}")


        # Check researcher vectors
        print(f"Researcher vectors: {len(self.researcher_vectors)}")
        if self.researcher_vectors:
            sample_vector = next(iter(self.researcher_vectors.values()))
            print(f"Vector dimensions: {sample_vector.shape}")
            print(f"Sample vector sum: {sample_vector.sum():.4f}")

        # Check conceptual profiles
        print(f"Conceptual profiles: {len(self.conceptual_profiles)}")
        if self.conceptual_profiles:
            sample_embedding = next(iter(self.conceptual_profiles.values()))
            print(f"Embedding dimensions: {sample_embedding.shape}")

        # Check evidence index
        print(f"Evidence index researchers: {len(self.evidence_index)}")

        # Check overlap between evidence index and conceptual profiles
        all_evidence_papers = set()
        for researcher_papers in self.evidence_index.values():
            for topic_papers in researcher_papers.values():
                all_evidence_papers.update(topic_papers)

        conceptual_papers = set(self.conceptual_profiles.keys())
        overlap = all_evidence_papers.intersection(conceptual_papers)

        print(f"Papers in evidence index: {len(all_evidence_papers)}")
        print(f"Papers with embeddings: {len(conceptual_papers)}")
        print(f"Overlap: {len(overlap)}")

        if len(overlap) == 0:
            print("❌ CRITICAL: No overlap between evidence index and conceptual profiles!")
        elif len(overlap) < len(all_evidence_papers) * 0.5:
            print("⚠️ WARNING: Low overlap between evidence index and conceptual profiles!")

        print("-" * 40)

    def filter_eligibility(self, solicitation_obj, researchers):
        """Simple eligibility filtering"""
        eligible = set(researchers)

        eligibility = getattr(solicitation_obj, 'eligibility', {})

        # Early-career filter
        if eligibility and any('early' in str(v).lower() for v in eligibility.values() if v):
            early_career = self.researcher_metadata[
                self.researcher_metadata['first_publication_year'] >= 2015
            ]['researcher_openalex_id'].tolist()
            eligible = eligible.intersection(set(early_career))
            print(f"   Applied early-career filter: {len(eligible)} remain")

        # Grant experience filter
        if eligibility and any('grant' in str(v).lower() or 'funding' in str(v).lower() for v in eligibility.values() if v):
            experienced = self.researcher_metadata[
                self.researcher_metadata['grant_experience_factor'] > 0
            ]['researcher_openalex_id'].tolist()
            eligible = eligible.intersection(set(experienced))
            print(f"   Applied grant experience filter: {len(eligible)} remain")

        return list(eligible)

    def extract_keywords_from_skills(self, skills: List[str]) -> List[str]:
        """Extract keywords from solicitation skills using same logic as researcher topics."""
        import re

        stop_words = {'and', 'in', 'of', 'for', 'the', 'a', 'an', 'to', 'with', 'on', 'at', 'by',
                      'expertise', 'experience', 'knowledge', 'ability', 'skills', 'understanding',
                      'capacity', 'proficiency', 'e.g.', 'eg', 'including', 'such', 'as'}

        all_keywords = []

        for skill in skills:
            # Clean and split
            cleaned = re.sub(r'[^\w\s-]', ' ', skill.lower())
            words = cleaned.split()

            # Extract meaningful keywords
            for word in words:
                word = word.strip('-')
                if (len(word) >= 3 and
                    word not in stop_words and
                    not word.isdigit()):
                    all_keywords.append(word)

        return all_keywords

    def score_researcher(self, researcher_id, skills, solicitation_embedding):
      """Score a single researcher"""
      try:
          # Get metadata
          researcher_row = self.researcher_metadata[
              self.researcher_metadata['researcher_openalex_id'] == researcher_id
          ]
          if researcher_row.empty:
              return None
          researcher_name = researcher_row.iloc[0]['researcher_name']
          total_papers = int(researcher_row.iloc[0]['total_papers'])
          grant_factor = researcher_row.iloc[0]['grant_experience_factor']

          # FIXED: Extract keywords and format with commas (same as researcher documents)
          solicitation_keywords = self.extract_keywords_from_skills(skills)
          solicitation_text = ', '.join(solicitation_keywords)  # FIXED: Use commas not spaces

          print(f"DEBUG - Researcher: {researcher_name}")
          print(f"  Original skills count: {len(skills)}")
          print(f"  Extracted keywords: {solicitation_keywords[:10]}...")  # Show first 10
          print(f"  Solicitation text: {solicitation_text[:100]}...")  # Show formatted text

          # Calculate sparse score (TF-IDF) - FIXED
          if researcher_id not in self.researcher_vectors:
              print(f"  WARNING: No TF-IDF vector for {researcher_id}")
              s_sparse = 0.0
          else:
              try:
                  solicitation_vector = self.tfidf_model.transform([solicitation_text])
                  researcher_vector = self.researcher_vectors[researcher_id].reshape(1, -1)

                  # Debug vector shapes and content
                  print(f"  Solicitation vector shape: {solicitation_vector.shape}")
                  print(f"  Researcher vector shape: {researcher_vector.shape}")
                  print(f"  Solicitation vector sum: {solicitation_vector.sum()}")
                  print(f"  Researcher vector sum: {researcher_vector.sum()}")
                  print(f"  Solicitation non-zero elements: {solicitation_vector.nnz}")

                  if solicitation_vector.sum() == 0 or researcher_vector.sum() == 0:
                      print(f"  WARNING: Zero vector detected")
                      s_sparse = 0.0
                  else:
                      similarity = cosine_similarity(solicitation_vector, researcher_vector)[0][0]
                      s_sparse = float(similarity * 100)
                      print(f"  TF-IDF similarity: {similarity}")
              except Exception as tfidf_error:
                  print(f"  TF-IDF ERROR: {tfidf_error}")
                  s_sparse = 0.0

          # Calculate dense score (max similarity across papers) - UNCHANGED
          s_dense = 0.0
          papers_checked = 0
          papers_found = 0
          if researcher_id in self.evidence_index:
              # Get all papers for this researcher from evidence index
              researcher_papers = []
              for topic_papers in self.evidence_index[researcher_id].values():
                  researcher_papers.extend(topic_papers)
              researcher_papers = list(set(researcher_papers))  # Remove duplicates
              print(f"  Papers from evidence index: {len(researcher_papers)}")

              max_sim = 0.0
              for paper_id in researcher_papers:
                  papers_checked += 1
                  if paper_id in self.conceptual_profiles:
                      papers_found += 1
                      paper_embedding = self.conceptual_profiles[paper_id]
                      try:
                          sim = cosine_similarity(
                              solicitation_embedding.reshape(1, -1),
                              paper_embedding.reshape(1, -1)
                          )[0][0]
                          max_sim = max(max_sim, sim)
                      except Exception as dense_error:
                          print(f"  Dense similarity error for paper {paper_id}: {dense_error}")

              s_dense = float(max_sim * 100)
              print(f"  Papers checked: {papers_checked}, Papers with embeddings: {papers_found}")
              print(f"  Max dense similarity: {max_sim}")
          else:
              print(f"  No evidence index entry for researcher")

          # Calculate final scores
          f_ge = max(1.0, min(3.0, 1.0 + (grant_factor * 0.2)))
          academic_expertise = (self.alpha * s_sparse) + (self.beta * s_dense)
          final_score = academic_expertise * f_ge

          print(f"  Final scores - Sparse: {s_sparse:.2f}, Dense: {s_dense:.2f}, Grant: {f_ge:.2f}")
          print(f"  Academic: {academic_expertise:.2f}, Final: {final_score:.2f}")
          print("-" * 50)

          return ResearcherMatch(
              researcher_id=researcher_id,
              researcher_name=researcher_name,
              academic_expertise_score=academic_expertise,
              s_sparse=s_sparse,
              s_dense=s_dense,
              f_ge=f_ge,
              final_affinity_score=final_score,
              total_papers=total_papers,
              eligibility_status="Eligible"
          )
      except Exception as e:
          print(f"ERROR scoring {researcher_id}: {e}")
          import traceback
          traceback.print_exc()
          return None

    def search(self, solicitation_obj):
        """Main search function"""
        start_time = time.time()

        print(f"\n🔍 ANALYZING: {solicitation_obj.title[:80]}...")

        # Get skills
        skills = solicitation_obj.required_skills_checklist
        print(f"📊 Skills to analyze: {len(skills)}")

        # Create solicitation embedding
        solicitation_text = f"{solicitation_obj.title}. {solicitation_obj.abstract}"
        solicitation_embedding = self.sentence_model.encode(solicitation_text)

        # Get all researchers
        all_researchers = list(self.researcher_metadata['researcher_openalex_id'])
        print(f"👥 Total researchers: {len(all_researchers)}")

        # Filter by eligibility
        eligible = self.filter_eligibility(solicitation_obj, all_researchers)
        print(f"✅ Eligible researchers: {len(eligible)}")

        # Score all eligible researchers (DEBUG first 3 only)
        # Score all eligible researchers (DEBUG first 3 only)
        print("🔄 Calculating scores...")
        matches = []
        debug_count = 0
        for researcher_id in eligible:
            # Only debug first 3 researchers to avoid spam
            if debug_count < 3:
                result = self.score_researcher(researcher_id, skills, solicitation_embedding)
                debug_count += 1
            else:
                # Run without debug for the rest
                try:
                    researcher_row = self.researcher_metadata[
                        self.researcher_metadata['researcher_openalex_id'] == researcher_id
                    ]
                    if researcher_row.empty:
                        continue

                    researcher_name = researcher_row.iloc[0]['researcher_name']
                    total_papers = int(researcher_row.iloc[0]['total_papers'])
                    grant_factor = researcher_row.iloc[0]['grant_experience_factor']

                    # FIXED: Use keyword extraction (same as debug)
                    solicitation_keywords = self.extract_keywords_from_skills(skills)
                    solicitation_text = ','.join(solicitation_keywords)

                    # Quick TF-IDF calculation
                    solicitation_vector = self.tfidf_model.transform([solicitation_text])
                    researcher_vector = self.researcher_vectors[researcher_id].reshape(1, -1)
                    s_sparse = float(cosine_similarity(solicitation_vector, researcher_vector)[0][0] * 100)

            # ... rest of the code stays the same

                    # Quick dense calculation
                    s_dense = 0.0
                    if researcher_id in self.evidence_index:
                        researcher_papers = []
                        for topic_papers in self.evidence_index[researcher_id].values():
                            researcher_papers.extend(topic_papers)

                        max_sim = 0.0
                        for paper_id in set(researcher_papers):
                            if paper_id in self.conceptual_profiles:
                                paper_embedding = self.conceptual_profiles[paper_id]
                                sim = cosine_similarity(
                                    solicitation_embedding.reshape(1, -1),
                                    paper_embedding.reshape(1, -1)
                                )[0][0]
                                max_sim = max(max_sim, sim)

                        s_dense = float(max_sim * 100)

                    f_ge = max(1.0, min(3.0, 1.0 + (grant_factor * 0.2)))
                    academic_expertise = (self.alpha * s_sparse) + (self.beta * s_dense)
                    final_score = academic_expertise * f_ge

                    result = ResearcherMatch(
                        researcher_id=researcher_id,
                        researcher_name=researcher_name,
                        academic_expertise_score=academic_expertise,
                        s_sparse=s_sparse,
                        s_dense=s_dense,
                        f_ge=f_ge,
                        final_affinity_score=final_score,
                        total_papers=total_papers,
                        eligibility_status="Eligible"
                    )
                except Exception as e:
                    print(f"Error scoring {researcher_id}: {e}")
                    continue

            if result:
                matches.append(result)

        # Sort by score
        matches.sort(key=lambda x: x.final_affinity_score, reverse=True)

        processing_time = time.time() - start_time

        return MatchingResults(
            solicitation_title=solicitation_obj.title,
            eligible_researchers=len(eligible),
            total_researchers=len(all_researchers),
            top_matches=matches[:20],
            skills_analyzed=skills,
            processing_time_seconds=round(processing_time, 2)
        )

# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

def run_complete_analysis():
    print("🚀 STARTING COMPLETE ANALYSIS")
    print("=" * 60)

    # Load solicitation
    print("📋 Loading solicitation...")
    with open(solicitation_file, 'r', encoding='utf-8') as f:
        solicitation_data = json.load(f)

    # Convert to object
    class SolicitationObj:
        def __init__(self, data):
            for key, value in data.items():
                setattr(self, key, value)

    solicitation_obj = SolicitationObj(solicitation_data)
    print(f"✅ Loaded: {solicitation_obj.title}")

    # Initialize engine
    engine = SimpleHybridEngine(researcher_data_dir)

    # Run search
    results = engine.search(solicitation_obj)

    # Display results
    print("\n🏆 RESULTS")
    print("=" * 60)
    print(f"Processing time: {results.processing_time_seconds}s")
    print(f"Eligible researchers: {results.eligible_researchers}/{results.total_researchers}")
    print()

    print("TOP 10 MATCHES:")
    print(f"{'Rank':<4} {'Name':<30} {'Final':<8} {'Academic':<9} {'Sparse':<8} {'Dense':<8} {'Papers':<7}")
    print("-" * 80)

    for i, match in enumerate(results.top_matches[:10], 1):
        print(f"{i:<4} {match.researcher_name[:29]:<30} "
              f"{match.final_affinity_score:<8.1f} {match.academic_expertise_score:<9.1f} "
              f"{match.s_sparse:<8.1f} {match.s_dense:<8.1f} {match.total_papers:<7}")

    # Save results
    output_file = f"{output_dir}/matching_results_{int(time.time())}.json"
    results_dict = asdict(results)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results_dict, f, indent=2, ensure_ascii=False, default=str)

    print(f"\n💾 Results saved to: {output_file}")
    print("\n✅ ANALYSIS COMPLETE!")

    return results

        # Add this after loading TF-IDF model to see what vocabulary was actually created
    print("First 20 TF-IDF features:", list(self.tfidf_model.get_feature_names_out()[:20]))
    print("Last 20 TF-IDF features:", list(self.tfidf_model.get_feature_names_out()[-20:]))

    # Test keyword matching
    solicitation_keywords = self.extract_keywords_from_skills(skills)
    vocab_set = set(self.tfidf_model.get_feature_names_out())
    matching_keywords = [kw for kw in solicitation_keywords if kw in vocab_set]
    print(f"Solicitation keywords that match vocab: {matching_keywords}")

# RUN IT
if __name__ == "__main__":
    results = run_complete_analysis()

In [ ]:
# Analysis of Algorithm Performance vs Actual Grant Recipients
# Based on the results from your matching system

# Actual grant recipients
actual_recipients = [
    "Tahir Ekin",           # Principal Investigator
    "Apan Qasem",           # Co-Principal Investigator
    "Damian Valles Molina", # Co-Principal Investigator
    "Jelena Tesic",         # Co-Principal Investigator
    "Lucia Summers"         # Co-Principal Investigator
]

# Results from your algorithm (extracted from the output)
all_results = [
    {"rank": 1, "name": "Apan Qasem", "final": 26.6, "academic": 24.2, "sparse": 12.6, "dense": 51.2, "papers": 71},
    {"rank": 2, "name": "Tahir Ekin", "final": 21.4, "academic": 17.9, "sparse": 15.2, "dense": 24.1, "papers": 47},
    {"rank": 3, "name": "David Gibbs", "final": 19.9, "academic": 19.9, "sparse": 11.7, "dense": 39.1, "papers": 142},
    {"rank": 4, "name": "Eleanor Close", "final": 18.4, "academic": 16.7, "sparse": 12.3, "dense": 27.2, "papers": 62},
    {"rank": 5, "name": "Ziliang Zong", "final": 18.3, "academic": 18.3, "sparse": 12.6, "dense": 31.5, "papers": 127},
    {"rank": 6, "name": "Subasish Das", "final": 17.7, "academic": 17.7, "sparse": 9.0, "dense": 37.8, "papers": 332},
    {"rank": 7, "name": "Rasim Musal", "final": 17.6, "academic": 17.6, "sparse": 15.3, "dense": 23.0, "papers": 18},
    {"rank": 8, "name": "Araceli Martinez Ortiz", "final": 16.9, "academic": 16.9, "sparse": 8.9, "dense": 35.6, "papers": 51},
    {"rank": 9, "name": "Jana Minifie", "final": 16.7, "academic": 15.2, "sparse": 9.3, "dense": 29.1, "papers": 18},
    {"rank": 10, "name": "Dominick Fazarro", "final": 16.1, "academic": 16.1, "sparse": 7.6, "dense": 36.0, "papers": 46}
]

print("🎯 ALGORITHM PERFORMANCE ANALYSIS")
print("=" * 60)

# Check how actual recipients ranked
print("\n📊 ACTUAL GRANT RECIPIENTS PERFORMANCE:")
print("-" * 40)
found_recipients = []
missing_recipients = []

for recipient in actual_recipients:
    found = False
    for result in all_results:
        if recipient.lower() in result["name"].lower() or result["name"].lower() in recipient.lower():
            found_recipients.append((recipient, result))
            found = True
            break
    if not found:
        missing_recipients.append(recipient)

print("✅ FOUND IN TOP 10:")
for recipient, result in found_recipients:
    print(f"  #{result['rank']:2d} {result['name']:<25} "
          f"Final: {result['final']:5.1f} | Academic: {result['academic']:5.1f} | "
          f"Sparse: {result['sparse']:5.1f} | Dense: {result['dense']:5.1f}")

print(f"\n❌ NOT FOUND IN TOP 10: {missing_recipients}")

# Calculate algorithm success rate
success_rate = (len(found_recipients) / len(actual_recipients)) * 100
print(f"\n🎯 ALGORITHM SUCCESS RATE: {len(found_recipients)}/{len(actual_recipients)} = {success_rate:.1f}%")

if len(found_recipients) >= 2:
    avg_rank = sum(result["rank"] for _, result in found_recipients) / len(found_recipients)
    print(f"📈 AVERAGE RANK OF FOUND RECIPIENTS: {avg_rank:.1f}")

print("\n" + "=" * 60)

# Top 10 by different metrics
print("\n🏆 TOP 10 BY ACADEMIC EXPERTISE SCORE:")
print("-" * 50)
sorted_by_academic = sorted(all_results, key=lambda x: x["academic"], reverse=True)
for i, result in enumerate(sorted_by_academic, 1):
    recipient_mark = "⭐" if any(recipient.lower() in result["name"].lower() for recipient in actual_recipients) else "  "
    print(f"{i:2d}. {recipient_mark} {result['name']:<25} Academic: {result['academic']:6.1f}")

print("\n🎯 TOP 10 BY SPARSE SCORE (TF-IDF Keyword Matching):")
print("-" * 50)
sorted_by_sparse = sorted(all_results, key=lambda x: x["sparse"], reverse=True)
for i, result in enumerate(sorted_by_sparse, 1):
    recipient_mark = "⭐" if any(recipient.lower() in result["name"].lower() for recipient in actual_recipients) else "  "
    print(f"{i:2d}. {recipient_mark} {result['name']:<25} Sparse: {result['sparse']:6.1f}")

print("\n🧠 TOP 10 BY DENSE SCORE (Semantic Similarity):")
print("-" * 50)
sorted_by_dense = sorted(all_results, key=lambda x: x["dense"], reverse=True)
for i, result in enumerate(sorted_by_dense, 1):
    recipient_mark = "⭐" if any(recipient.lower() in result["name"].lower() for recipient in actual_recipients) else "  "
    print(f"{i:2d}. {recipient_mark} {result['name']:<25} Dense: {result['dense']:6.1f}")

print("\n" + "=" * 60)
print("\n💡 ANALYSIS INSIGHTS:")
print("-" * 25)

# Analyze the performance
recipient_ranks = [result["rank"] for _, result in found_recipients]
if recipient_ranks:
    best_rank = min(recipient_ranks)
    worst_rank = max(recipient_ranks)
    print(f"• Best recipient rank: #{best_rank}")
    print(f"• Worst recipient rank: #{worst_rank}")

# Check which metric performed best for recipients
academic_ranks = []
sparse_ranks = []
dense_ranks = []

for recipient in actual_recipients:
    for i, result in enumerate(sorted_by_academic, 1):
        if recipient.lower() in result["name"].lower():
            academic_ranks.append(i)
            break
    for i, result in enumerate(sorted_by_sparse, 1):
        if recipient.lower() in result["name"].lower():
            sparse_ranks.append(i)
            break
    for i, result in enumerate(sorted_by_dense, 1):
        if recipient.lower() in result["name"].lower():
            dense_ranks.append(i)
            break

if academic_ranks:
    print(f"• Academic score avg rank for recipients: {sum(academic_ranks)/len(academic_ranks):.1f}")
if sparse_ranks:
    print(f"• Sparse score avg rank for recipients: {sum(sparse_ranks)/len(sparse_ranks):.1f}")
if dense_ranks:
    print(f"• Dense score avg rank for recipients: {sum(dense_ranks)/len(dense_ranks):.1f}")

print(f"\n🎉 SUCCESS: Your algorithm correctly identified {len(found_recipients)} out of {len(actual_recipients)} grant recipients in the top 10!")

if len(found_recipients) >= 3:
    print("🌟 EXCELLENT performance - majority of recipients found!")
elif len(found_recipients) >= 2:
    print("👍 GOOD performance - multiple recipients identified!")
elif len(found_recipients) >= 1:
    print("✅ DECENT performance - at least one recipient found!")
else:
    print("❌ Poor performance - no recipients in top 10")

# More diagnostics Check

In [ ]:
# Add this after loading TF-IDF model to see what vocabulary was actually created
print("First 20 TF-IDF features:", list(self.tfidf_model.get_feature_names_out()[:20]))
print("Last 20 TF-IDF features:", list(self.tfidf_model.get_feature_names_out()[-20:]))

# Test keyword matching
solicitation_keywords = self.extract_keywords_from_skills(skills)
vocab_set = set(self.tfidf_model.get_feature_names_out())
matching_keywords = [kw for kw in solicitation_keywords if kw in vocab_set]
print(f"Solicitation keywords that match vocab: {matching_keywords}")

In [ ]:
# Load the TF-IDF model and check what's wrong
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the TF-IDF model
with open('/content/drive/MyDrive/datastore/v2_DATA/tfidf_model.pkl', 'rb') as f:
    tfidf_model = pickle.load(f)

print("=== TF-IDF MODEL DIAGNOSIS ===")
vocab = tfidf_model.get_feature_names_out()
print(f"Vocabulary size: {len(vocab)}")
print(f"First 20 features: {list(vocab[:20])}")
print(f"Last 20 features: {list(vocab[-20:])}")

# Test solicitation skills (from your results)
skills = [
    "Expertise in artificial intelligence research areas (e.g. machine learning, natural language processing, computer vision, robotics)",
    "Experience in AI education and curriculum development",
    "Knowledge of inclusive teaching practices and broadening participation strategies"
]

solicitation_text = ' '.join(skills)
print(f"\n=== SOLICITATION TEXT ===")
print(f"Length: {len(solicitation_text)}")
print(f"Text preview: {solicitation_text[:200]}...")

# Transform solicitation text
solicitation_vector = tfidf_model.transform([solicitation_text])
print(f"\n=== SOLICITATION VECTOR ===")
print(f"Vector shape: {solicitation_vector.shape}")
print(f"Vector sum: {solicitation_vector.sum()}")
print(f"Non-zero elements: {solicitation_vector.nnz}")

if solicitation_vector.nnz > 0:
    # Get non-zero features
    vector_array = solicitation_vector.toarray()[0]
    non_zero_indices = np.where(vector_array > 0)[0]
    print(f"Non-zero features:")
    for idx in non_zero_indices[:10]:  # Show first 10
        print(f"  {vocab[idx]}: {vector_array[idx]:.4f}")
else:
    print("❌ PROBLEM: Solicitation vector is completely zero!")

    # Check if any solicitation words are in vocabulary
    solicitation_words = solicitation_text.lower().split()
    vocab_set = set(vocab)
    matching_words = [word for word in solicitation_words if word in vocab_set]
    print(f"Words from solicitation found in vocabulary: {matching_words[:10]}")

In [ ]:
# Load a sample researcher vector and compare
researcher_data = np.load('/content/drive/MyDrive/datastore/v2_analysis/researcher_vectors.npz', allow_pickle=True)
vectors = researcher_data['vectors']
researcher_ids = researcher_data['researcher_ids']

print(f"\n=== SAMPLE RESEARCHER VECTOR ===")
sample_vector = vectors[0]
print(f"Researcher vector shape: {sample_vector.shape}")
print(f"Researcher vector sum: {sample_vector.sum()}")
print(f"Non-zero elements: {np.count_nonzero(sample_vector)}")

# Try manual similarity calculation
if solicitation_vector.sum() > 0 and sample_vector.sum() > 0:
    similarity = cosine_similarity(solicitation_vector, sample_vector.reshape(1, -1))[0][0]
    print(f"Manual similarity calculation: {similarity}")
else:
    print("❌ Cannot calculate similarity - one vector is zero")

In [ ]:
def create_weighted_expertise_document(self, researcher_papers: pd.DataFrame) -> str:
    """Create weighted expertise document using extracted keywords from topic names."""
    import re

    # Common stop words to remove
    stop_words = {'and', 'in', 'of', 'for', 'the', 'a', 'an', 'to', 'with', 'on', 'at', 'by'}

    topic_words = []

    for _, paper in researcher_papers.iterrows():
        topics = self.parse_curated_topics(paper['curated_topics'])
        publication_year = paper['publication_year']
        citations = paper['cited_by_count'] if pd.notna(paper['cited_by_count']) else 0

        for topic in topics:
            if topic.get('type') == 'topic':
                topic_score = topic.get('score', 0)
                topic_name = topic.get('name', '').strip()

                if topic_name and topic_score > 0:
                    # Extract keywords from topic name
                    keywords = self.extract_keywords_from_topic(topic_name, stop_words)

                    if keywords:
                        repetition_count = self.calculate_topic_repetition_weight(
                            topic_score, publication_year, citations
                        )

                        if repetition_count == 0:
                            repetition_count = 1

                        # Add keywords repeated according to weight
                        for keyword in keywords:
                            topic_words.extend([keyword] * repetition_count)

    document = ' '.join(topic_words)

    # Fallback if document is too short
    if len(document.split()) < 5:
        fallback_keywords = []
        for _, paper in researcher_papers.iterrows():
            topics = self.parse_curated_topics(paper['curated_topics'])
            for topic in topics:
                if topic.get('type') == 'topic':
                    topic_name = topic.get('name', '').strip()
                    if topic_name:
                        keywords = self.extract_keywords_from_topic(topic_name, stop_words)
                        fallback_keywords.extend(keywords)

        if fallback_keywords:
            document = ' '.join(list(set(fallback_keywords))[:20])

    return document

def extract_keywords_from_topic(self, topic_name: str, stop_words: set) -> List[str]:
    """Extract meaningful keywords from a topic name."""
    import re

    # Clean and split the topic name
    # Remove special characters and normalize
    cleaned = re.sub(r'[^\w\s-]', ' ', topic_name.lower())
    words = cleaned.split()

    # Filter out stop words and very short words
    keywords = []
    for word in words:
        word = word.strip('-')
        if (len(word) >= 3 and
            word not in stop_words and
            not word.isdigit()):
            keywords.append(word)

    # Keep only meaningful keywords (max 5 per topic)
    return keywords[:5]

In [ ]:
!pip install groq

In [ ]:
# ==============================================================================
# COMPLETE UNIFIED RESEARCHER MATCHING & DREAM TEAM SYSTEM
# ==============================================================================
import json
import numpy as np
import pandas as pd
import pickle
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, asdict
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import time
import warnings
from datetime import datetime
import os

# Try to import Groq API (optional)
try:
    from groq import Groq
    GROQ_AVAILABLE = True
except ImportError:
    GROQ_AVAILABLE = False
    print("⚠️ Groq library not installed. GAP analysis will be basic.")
    print("   Install with: pip install groq")

# Try to import Google Colab userdata (optional)
try:
    from google.colab import userdata
    COLAB_AVAILABLE = True
except ImportError:
    COLAB_AVAILABLE = False

warnings.filterwarnings('ignore')

# ==============================================================================
# CONFIGURATION - CHANGE THESE PATHS
# ==============================================================================
solicitation_file = "/content/drive/MyDrive/datastore/NSF 23-506_ Expanding AI Innovation through Capacity Building and Partnerships (ExpandAI) _ NSF - National Science Foundation_enhanced_analysis.json"
researcher_data_dir = "/content/drive/MyDrive/datastore/v2_DATA"
output_dir = "/content/drive/MyDrive/datastore"

# ==============================================================================
# GROQ API SETUP (OPTIONAL - FOR AI-POWERED STRATEGIC ANALYSIS)
# ==============================================================================
# 1. Get free API key at: https://console.groq.com/
# 2. In Google Colab: Go to left sidebar > Secrets > Add new secret
#    Name: GROQ_API_KEY, Value: your_api_key_here
# 3. Or set environment variable: export GROQ_API_KEY=your_api_key_here
# ==============================================================================

# ==============================================================================
# CORE CLASSES
# ==============================================================================
@dataclass
class ResearcherMatch:
    researcher_id: str
    researcher_name: str
    academic_expertise_score: float
    s_sparse: float
    s_dense: float
    f_ge: float
    final_affinity_score: float
    total_papers: int
    eligibility_status: str

@dataclass
class MatchingResults:
    solicitation_title: str
    eligible_researchers: int
    total_researchers: int
    top_matches: List[ResearcherMatch]
    skills_analyzed: List[str]
    processing_time_seconds: float

@dataclass
class DreamTeamReport:
    team_members: List[Dict]
    overall_coverage_score: float
    skill_analysis: List[Dict]
    strategic_analysis: str
    selection_history: List[Dict]
    generated_at: str

class UnifiedResearcherSystem:
    """
    Complete system that combines researcher matching with dream team assembly
    and strategic GAP analysis using Claude API.
    """

    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
        self.alpha = 0.7  # TF-IDF weight
        self.beta = 0.3   # Dense weight
        self.groq_client = None
        self.setup_groq_api()
        self.load_data()

    def setup_groq_api(self):
        """Initialize Groq API client if available."""
        if not GROQ_AVAILABLE:
            print("⚠️ Groq API not available. Strategic analysis will be basic.")
            print("   Install with: pip install groq")
            return

        try:
            if COLAB_AVAILABLE:
                api_key = userdata.get('GROQ_API_KEY')
            else:
                api_key = os.environ.get('GROQ_API_KEY')

            if api_key:
                self.groq_client = Groq(api_key=api_key)
                print("✅ Groq API client initialized for strategic analysis")
            else:
                print("⚠️ Groq API key not found. Set GROQ_API_KEY.")
                print("   Get free API key at: https://console.groq.com/")
        except Exception as e:
            print(f"⚠️ Groq API setup failed: {e}. Strategic analysis will be basic.")

    def load_data(self):
        """Load all preprocessed researcher data."""
        print("📂 Loading preprocessed data...")

        # Handle pickle loading issue - define dummy class if needed
        try:
            # Try to load TF-IDF model normally first
            with open(self.data_dir / 'tfidf_model.pkl', 'rb') as f:
                self.tfidf_model = pickle.load(f)
        except AttributeError as e:
            if "ResearcherProfileProcessor" in str(e):
                print("🔧 Fixing pickle compatibility issue...")
                # Create a dummy class to handle the pickle loading
                import sys
                class ResearcherProfileProcessor:
                    def comma_tokenizer(self, text: str):
                        return [token.strip() for token in text.split(',') if token.strip()]

                # Add the class to the current module
                sys.modules[__name__].ResearcherProfileProcessor = ResearcherProfileProcessor

                # Try loading again
                with open(self.data_dir / 'tfidf_model.pkl', 'rb') as f:
                    self.tfidf_model = pickle.load(f)
                print("✅ TF-IDF model loaded successfully")
            else:
                raise e

        # Load researcher vectors
        researcher_data = np.load(self.data_dir / 'researcher_vectors.npz', allow_pickle=True)
        vectors = researcher_data['vectors']
        researcher_ids = researcher_data['researcher_ids']
        self.researcher_vectors = dict(zip(researcher_ids, vectors))

        # Load paper embeddings
        conceptual_data = np.load(self.data_dir / 'conceptual_profiles.npz', allow_pickle=True)
        embeddings = conceptual_data['embeddings']
        work_ids = conceptual_data['work_ids']
        self.conceptual_profiles = dict(zip(work_ids, embeddings))

        # Load evidence index
        with open(self.data_dir / 'evidence_index.json', 'r') as f:
            self.evidence_index = json.load(f)

        # Load metadata
        self.researcher_metadata = pd.read_parquet(self.data_dir / 'researcher_metadata.parquet')

        # Load sentence model
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

        print(f"✅ Loaded {len(self.researcher_vectors)} researchers")
        self.diagnose_data_quality()

    def diagnose_data_quality(self):
        """Diagnose potential data quality issues."""
        print("\n🔍 DIAGNOSING DATA QUALITY")
        print("-" * 40)

        # Check TF-IDF model
        try:
            vocab_size = len(self.tfidf_model.get_feature_names_out())
            print(f"TF-IDF vocabulary size: {vocab_size}")

            vocab = list(self.tfidf_model.get_feature_names_out())
            print(f"First 20 TF-IDF features: {vocab[:20]}")
            print(f"Last 20 TF-IDF features: {vocab[-20:]}")

        except Exception as e:
            print(f"❌ Error accessing TF-IDF vocabulary: {e}")

        # Check researcher vectors
        print(f"Researcher vectors: {len(self.researcher_vectors)}")
        if self.researcher_vectors:
            sample_vector = next(iter(self.researcher_vectors.values()))
            print(f"Vector dimensions: {sample_vector.shape}")
            print(f"Sample vector sum: {sample_vector.sum():.4f}")

        # Check conceptual profiles
        print(f"Conceptual profiles: {len(self.conceptual_profiles)}")
        if self.conceptual_profiles:
            sample_embedding = next(iter(self.conceptual_profiles.values()))
            print(f"Embedding dimensions: {sample_embedding.shape}")

        # Check evidence index
        print(f"Evidence index researchers: {len(self.evidence_index)}")

        # Check overlap between evidence index and conceptual profiles
        all_evidence_papers = set()
        for researcher_papers in self.evidence_index.values():
            for topic_papers in researcher_papers.values():
                all_evidence_papers.update(topic_papers)

        conceptual_papers = set(self.conceptual_profiles.keys())
        overlap = all_evidence_papers.intersection(conceptual_papers)

        print(f"Papers in evidence index: {len(all_evidence_papers)}")
        print(f"Papers with embeddings: {len(conceptual_papers)}")
        print(f"Overlap: {len(overlap)}")

        if len(overlap) == 0:
            print("❌ CRITICAL: No overlap between evidence index and conceptual profiles!")
        elif len(overlap) < len(all_evidence_papers) * 0.5:
            print("⚠️ WARNING: Low overlap between evidence index and conceptual profiles!")

        print("-" * 40)

    def filter_eligibility(self, solicitation_obj, researchers):
        """Simple eligibility filtering."""
        eligible = set(researchers)
        eligibility = getattr(solicitation_obj, 'eligibility', {})

        # Early-career filter
        if eligibility and any('early' in str(v).lower() for v in eligibility.values() if v):
            early_career = self.researcher_metadata[
                self.researcher_metadata['first_publication_year'] >= 2015
            ]['researcher_openalex_id'].tolist()
            eligible = eligible.intersection(set(early_career))
            print(f"   Applied early-career filter: {len(eligible)} remain")

        # Grant experience filter
        if eligibility and any('grant' in str(v).lower() or 'funding' in str(v).lower() for v in eligibility.values() if v):
            experienced = self.researcher_metadata[
                self.researcher_metadata['grant_experience_factor'] > 0
            ]['researcher_openalex_id'].tolist()
            eligible = eligible.intersection(set(experienced))
            print(f"   Applied grant experience filter: {len(eligible)} remain")

        return list(eligible)

    def extract_keywords_from_skills(self, skills: List[str]) -> List[str]:
        """Extract keywords from solicitation skills using same logic as researcher topics."""
        import re

        stop_words = {'and', 'in', 'of', 'for', 'the', 'a', 'an', 'to', 'with', 'on', 'at', 'by',
                      'expertise', 'experience', 'knowledge', 'ability', 'skills', 'understanding',
                      'capacity', 'proficiency', 'e.g.', 'eg', 'including', 'such', 'as'}

        all_keywords = []

        for skill in skills:
            # Clean and split
            cleaned = re.sub(r'[^\w\s-]', ' ', skill.lower())
            words = cleaned.split()

            # Extract meaningful keywords
            for word in words:
                word = word.strip('-')
                if (len(word) >= 3 and
                    word not in stop_words and
                    not word.isdigit()):
                    all_keywords.append(word)

        return all_keywords

    def score_researcher(self, researcher_id, skills, solicitation_embedding, debug_mode=False):
        """Score a single researcher with optional debug output."""
        try:
            # Get metadata
            researcher_row = self.researcher_metadata[
                self.researcher_metadata['researcher_openalex_id'] == researcher_id
            ]
            if researcher_row.empty:
                return None

            researcher_name = researcher_row.iloc[0]['researcher_name']
            total_papers = int(researcher_row.iloc[0]['total_papers'])
            grant_factor = researcher_row.iloc[0]['grant_experience_factor']

            # Extract keywords and format with commas (same as researcher documents)
            solicitation_keywords = self.extract_keywords_from_skills(skills)
            solicitation_text = ', '.join(solicitation_keywords)

            if debug_mode:
                print(f"DEBUG - Researcher: {researcher_name}")
                print(f"  Original skills count: {len(skills)}")
                print(f"  Extracted keywords: {solicitation_keywords[:10]}...")
                print(f"  Solicitation text: {solicitation_text[:100]}...")

            # Calculate sparse score (TF-IDF)
            if researcher_id not in self.researcher_vectors:
                if debug_mode:
                    print(f"  WARNING: No TF-IDF vector for {researcher_id}")
                s_sparse = 0.0
            else:
                try:
                    solicitation_vector = self.tfidf_model.transform([solicitation_text])
                    researcher_vector = self.researcher_vectors[researcher_id].reshape(1, -1)

                    if debug_mode:
                        print(f"  Solicitation vector shape: {solicitation_vector.shape}")
                        print(f"  Researcher vector shape: {researcher_vector.shape}")
                        print(f"  Solicitation vector sum: {solicitation_vector.sum()}")
                        print(f"  Researcher vector sum: {researcher_vector.sum()}")
                        if hasattr(solicitation_vector, 'nnz'):
                             print(f"  Solicitation non-zero elements: {solicitation_vector.nnz}")


                    if solicitation_vector.sum() == 0 or researcher_vector.sum() == 0:
                        if debug_mode:
                            print(f"  WARNING: Zero vector detected")
                        s_sparse = 0.0
                    else:
                        similarity = cosine_similarity(solicitation_vector, researcher_vector)[0][0]
                        s_sparse = float(similarity * 100)
                        if debug_mode:
                            print(f"  TF-IDF similarity: {similarity}")
                except Exception as tfidf_error:
                    if debug_mode:
                        print(f"  TF-IDF ERROR: {tfidf_error}")
                    s_sparse = 0.0

            # Calculate dense score (max similarity across papers)
            s_dense = 0.0
            papers_checked = 0
            papers_found = 0
            if researcher_id in self.evidence_index:
                # Get all papers for this researcher from evidence index
                researcher_papers = []
                for topic_papers in self.evidence_index[researcher_id].values():
                    researcher_papers.extend(topic_papers)
                researcher_papers = list(set(researcher_papers))  # Remove duplicates

                if debug_mode:
                    print(f"  Papers from evidence index: {len(researcher_papers)}")

                max_sim = 0.0
                for paper_id in researcher_papers:
                    papers_checked += 1
                    if paper_id in self.conceptual_profiles:
                        papers_found += 1
                        paper_embedding = self.conceptual_profiles[paper_id]
                        try:
                            sim = cosine_similarity(
                                solicitation_embedding.reshape(1, -1),
                                paper_embedding.reshape(1, -1)
                            )[0][0]
                            max_sim = max(max_sim, sim)
                        except Exception as dense_error:
                            if debug_mode:
                                print(f"  Dense similarity error for paper {paper_id}: {dense_error}")

                s_dense = float(max_sim * 100)
                if debug_mode:
                    print(f"  Papers checked: {papers_checked}, Papers with embeddings: {papers_found}")
                    print(f"  Max dense similarity: {max_sim}")
            else:
                if debug_mode:
                    print(f"  No evidence index entry for researcher")

            # Calculate final scores
            f_ge = max(1.0, min(3.0, 1.0 + (grant_factor * 0.2)))
            academic_expertise = (self.alpha * s_sparse) + (self.beta * s_dense)
            final_score = academic_expertise * f_ge

            if debug_mode:
                print(f"  Final scores - Sparse: {s_sparse:.2f}, Dense: {s_dense:.2f}, Grant: {f_ge:.2f}")
                print(f"  Academic: {academic_expertise:.2f}, Final: {final_score:.2f}")
                print("-" * 50)

            return ResearcherMatch(
                researcher_id=researcher_id,
                researcher_name=researcher_name,
                academic_expertise_score=academic_expertise,
                s_sparse=s_sparse,
                s_dense=s_dense,
                f_ge=f_ge,
                final_affinity_score=final_score,
                total_papers=total_papers,
                eligibility_status="Eligible"
            )
        except Exception as e:
            if debug_mode:
                print(f"ERROR scoring {researcher_id}: {e}")
                import traceback
                traceback.print_exc()
            return None

    def search_researchers(self, solicitation_obj, debug_first_n=3):
        """Main search function to rank researchers."""
        start_time = time.time()

        print(f"\n🔍 ANALYZING: {solicitation_obj.title[:80]}...")

        # Get skills
        skills = solicitation_obj.required_skills_checklist
        print(f"📊 Skills to analyze: {len(skills)}")

        # Create solicitation embedding
        solicitation_text = f"{solicitation_obj.title}. {solicitation_obj.abstract}"
        solicitation_embedding = self.sentence_model.encode(solicitation_text)

        # Get all researchers
        all_researchers = list(self.researcher_metadata['researcher_openalex_id'])
        print(f"👥 Total researchers: {len(all_researchers)}")

        # Filter by eligibility
        eligible = self.filter_eligibility(solicitation_obj, all_researchers)
        print(f"✅ Eligible researchers: {len(eligible)}")

        # Score all eligible researchers
        print("🔄 Calculating scores...")
        matches = []
        debug_count = 0

        for researcher_id in eligible:
            # Debug first N researchers to avoid spam
            debug_mode = debug_count < debug_first_n
            if debug_mode:
                debug_count += 1

            result = self.score_researcher(researcher_id, skills, solicitation_embedding, debug_mode)

            if result:
                matches.append(result)

        # Sort by score
        matches.sort(key=lambda x: x.final_affinity_score, reverse=True)

        processing_time = time.time() - start_time

        return MatchingResults(
            solicitation_title=solicitation_obj.title,
            eligible_researchers=len(eligible),
            total_researchers=len(all_researchers),
            top_matches=matches,
            skills_analyzed=skills,
            processing_time_seconds=round(processing_time, 2)
        )

    def create_affinity_matrix(self, matching_results, top_n_researchers=20):
        """Create an affinity matrix from matching results."""
        print(f"\n📊 Creating affinity matrix for top {top_n_researchers} researchers...")

        # Get top researchers
        top_matches = matching_results.top_matches[:top_n_researchers]
        researcher_names = [match.researcher_name for match in top_matches]
        skills = matching_results.skills_analyzed

        # Create matrix: researchers × skills
        affinity_matrix = np.zeros((len(top_matches), len(skills)))

        for i, match in enumerate(top_matches):
            # For each researcher, calculate their affinity to each skill
            researcher_id = match.researcher_id

            for j, skill in enumerate(skills):
                # Calculate skill-specific affinity score
                skill_keywords = self.extract_keywords_from_skills([skill])
                skill_text = ', '.join(skill_keywords)

                try:
                    # TF-IDF similarity for this specific skill
                    skill_vector = self.tfidf_model.transform([skill_text])
                    researcher_vector = self.researcher_vectors[researcher_id].reshape(1, -1)
                    sparse_sim = cosine_similarity(skill_vector, researcher_vector)[0][0] * 100

                    # Dense similarity (using overall solicitation embedding as proxy)
                    # In practice, you might want to create skill-specific embeddings
                    # For simplicity, let's use the max dense score from the overall matching
                    dense_sim = match.s_dense # Use the pre-calculated max dense score for the researcher


                    # Combined affinity score for this skill
                    skill_affinity = (self.alpha * sparse_sim) + (self.beta * dense_sim)
                    affinity_matrix[i, j] = max(0, skill_affinity)  # Ensure non-negative

                except Exception as e:
                    # Fallback: use overall academic score
                    affinity_matrix[i, j] = match.academic_expertise_score

        # Create DataFrame
        affinity_df = pd.DataFrame(
            affinity_matrix,
            index=researcher_names,
            columns=[f"Skill_{i+1}: {skill}" for i, skill in enumerate(skills)]
        )

        print(f"✅ Created affinity matrix: {affinity_df.shape[0]} researchers × {affinity_df.shape[1]} skills")
        return affinity_df

    def calculate_team_coverage(self, affinity_df, team_indices):
        """Calculate team coverage scores for all skills."""
        if not team_indices:
            return np.array([0.0] * affinity_df.shape[1]), 0.0
        team_affinities = affinity_df.iloc[team_indices]
        skill_coverages = team_affinities.max(axis=0).values
        return skill_coverages, np.mean(skill_coverages)

    def calculate_marginal_gain(self, affinity_df, current_team_indices, candidate_index):
        """Calculate the marginal gain of adding a candidate to the team."""
        _, current_coverage = self.calculate_team_coverage(affinity_df, current_team_indices)
        _, new_coverage = self.calculate_team_coverage(affinity_df, current_team_indices + [candidate_index])
        return new_coverage - current_coverage

    def dream_team_hybrid_strategy(self, affinity_df, guaranteed_top_n=2, max_team_size=4):
        """
        Hybrid approach: Lock in top N performers, then optimize coverage for remaining slots.
        This ensures we get researchers with proven grant experience and keyword matching.
        """
        print(f"\n🎯 Running Hybrid Dream Team Strategy")
        print(f"   Step 1: Lock in top {guaranteed_top_n} performers")
        print(f"   Step 2: Optimize coverage for remaining {max_team_size - guaranteed_top_n} slots")
        print("=" * 50)

        # Phase 1: Lock in top performers (they earned their ranking!)
        researcher_averages = affinity_df.mean(axis=1).sort_values(ascending=False)
        top_performers = researcher_averages.head(guaranteed_top_n)

        selected_indices = []
        selection_history = []

        print("🔒 LOCKING IN TOP PERFORMERS:")
        for i, (name, avg_score) in enumerate(top_performers.items()):
            idx = affinity_df.index.get_loc(name)
            selected_indices.append(idx)
            role = "PI" if i == 0 else f"Co-PI {i}"

            selection_history.append({
                'step': i + 1,
                'action': f'Lock {role}',
                'researcher_name': name,
                'reason': f'Top {i+1} performer (avg: {avg_score:.2f}, proven track record)',
                'team_coverage': 0  # Will calculate after
            })
            print(f"   ✅ {name} ({role}) - Avg Score: {avg_score:.2f}")

        # Calculate coverage after locking in top performers
        _, coverage_after_top = self.calculate_team_coverage(affinity_df, selected_indices)
        print(f"   📊 Coverage after top {guaranteed_top_n}: {coverage_after_top:.2f}")

        # Update coverage in history
        for entry in selection_history:
            entry['team_coverage'] = coverage_after_top

        # Phase 2: Optimize remaining slots for coverage
        print(f"\n🎯 OPTIMIZING REMAINING {max_team_size - guaranteed_top_n} SLOTS FOR COVERAGE:")
        n_researchers = len(affinity_df)

        for step in range(guaranteed_top_n + 1, max_team_size + 1):
            # Calculate marginal gains for all remaining researchers
            gains = [(idx, self.calculate_marginal_gain(affinity_df, selected_indices, idx))
                     for idx in range(n_researchers) if idx not in selected_indices]

            if not gains:
                print(f"   ⚠️ No more candidates available")
                break

            # Sort by marginal gain and show top candidates
            top_candidates = sorted(gains, key=lambda x: x[1], reverse=True)[:5]
            best_candidate_idx, best_marginal_gain = top_candidates[0]

            print(f"   📊 Top candidates for slot {step - guaranteed_top_n}:")
            for i, (idx, gain) in enumerate(top_candidates):
                candidate_name = affinity_df.index[idx]
                candidate_avg = affinity_df.iloc[idx].mean()
                marker = "👑" if i == 0 else f"  {i+1}."
                print(f"      {marker} {candidate_name} (Avg: {candidate_avg:.2f}, Coverage Gain: +{gain:.2f})")

            # Add the best candidate for coverage
            if best_marginal_gain > 0.1:  # Lower threshold since we have strong foundation
                selected_indices.append(best_candidate_idx)
                _, new_coverage = self.calculate_team_coverage(affinity_df, selected_indices)

                selection_history.append({
                    'step': step,
                    'action': 'Add for Coverage',
                    'researcher_name': affinity_df.index[best_candidate_idx],
                    'reason': f'Best coverage gain (+{best_marginal_gain:.2f})',
                    'team_coverage': new_coverage
                })
                print(f"   ✅ Added: {affinity_df.index[best_candidate_idx]} (New Coverage: {new_coverage:.2f})")
            else:
                print(f"   🛑 Stopping: Marginal gain {best_marginal_gain:.2f} too small")
                break

        final_coverage = self.calculate_team_coverage(affinity_df, selected_indices)[1]
        print(f"\n🎯 Final Hybrid Team ({len(selected_indices)} members) with {final_coverage:.2f} coverage")
        print(f"   Strategy: Top {guaranteed_top_n} performers + coverage optimization")

        return selected_indices, selection_history

    def dream_team_greedy_algorithm(self, affinity_df, min_team_size=2, max_team_size=4, marginal_threshold=0.25):
        """Implement the greedy algorithm to select the best team."""
        print("\n🎯 Running Pure Greedy Algorithm...")
        print("=" * 50)
        n_researchers = len(affinity_df)
        selected_indices = []
        selection_history = []

        # Step 1: Select the best overall researcher as PI
        best_researcher_pos = affinity_df.mean(axis=1).idxmax()
        best_researcher_loc = affinity_df.index.get_loc(best_researcher_pos)
        selected_indices.append(best_researcher_loc)
        _, initial_coverage = self.calculate_team_coverage(affinity_df, selected_indices)

        selection_history.append({
            'step': 1, 'action': 'Select PI',
            'researcher_name': affinity_df.index[best_researcher_loc],
            'reason': 'Highest average affinity score',
            'team_coverage': initial_coverage
        })
        print(f"🏆 Step 1 - PI Selection: {affinity_df.index[best_researcher_loc]} (Coverage: {initial_coverage:.2f})")

        # Step 2-N: Iteratively add members with the highest marginal gain
        for step in range(2, max_team_size + 1):
            gains = [(idx, self.calculate_marginal_gain(affinity_df, selected_indices, idx))
                     for idx in range(n_researchers) if idx not in selected_indices]
            if not gains:
                break

            best_candidate_idx, best_marginal_gain = max(gains, key=lambda item: item[1])

            # Show top 3 candidates for transparency
            top_candidates = sorted(gains, key=lambda x: x[1], reverse=True)[:3]
            print(f"   📊 Top candidates for Step {step}:")
            for i, (idx, gain) in enumerate(top_candidates):
                candidate_name = affinity_df.index[idx]
                print(f"      {i+1}. {candidate_name} (Marginal gain: +{gain:.2f})")

            # More flexible stopping criteria
            should_add = (
                best_marginal_gain > marginal_threshold or  # Significant gain
                len(selected_indices) < min_team_size or    # Haven't reached minimum
                (step <= 4 and best_marginal_gain > 0.1)   # Force at least 4 if minimal gain
            )

            if should_add:
                selected_indices.append(best_candidate_idx)
                _, new_coverage = self.calculate_team_coverage(affinity_df, selected_indices)
                selection_history.append({
                    'step': step, 'action': 'Add Member',
                    'researcher_name': affinity_df.index[best_candidate_idx],
                    'reason': f'Maximum marginal gain (+{best_marginal_gain:.2f})',
                    'team_coverage': new_coverage
                })
                print(f"✅ Step {step} - Added: {affinity_df.index[best_candidate_idx]} (New Coverage: {new_coverage:.2f})")
            else:
                print(f"🛑 Step {step} - Stopping: Marginal gain {best_marginal_gain:.2f} below threshold {marginal_threshold}")
                break

        final_coverage = self.calculate_team_coverage(affinity_df, selected_indices)[1]
        print(f"\n🎯 Final Dream Team ({len(selected_indices)} members) with {final_coverage:.2f} coverage.")
        return selected_indices, selection_history

    def generate_coverage_report(self, affinity_df, team_indices, skills_list):
        """Generate a detailed coverage report for the selected team."""
        skill_coverages, overall_coverage = self.calculate_team_coverage(affinity_df, team_indices)

        team_members = []
        for idx in team_indices:
            scores = affinity_df.iloc[idx]
            top_skills = [{'skill': skills_list[i], 'score': scores[i]}
                         for i in scores.argsort()[-3:][::-1]]
            team_members.append({
                'name': affinity_df.index[idx],
                'avg_affinity': scores.mean(),
                'top_skills': top_skills
            })

        skill_analysis = []
        for i, (skill, coverage) in enumerate(zip(skills_list, skill_coverages)):
            team_scores = affinity_df.iloc[team_indices, i]
            if not team_scores.empty: # Handle case where team is empty or skill has no coverage
                best_member_name = team_scores.idxmax()
                expert_score = team_scores.max()
            else:
                best_member_name = "N/A"
                expert_score = 0.0


            skill_analysis.append({
                'skill': skill,
                'coverage_score': coverage,
                'level': 'High' if coverage >= 70 else 'Medium' if coverage >= 40 else 'Low',
                'expert': best_member_name,
                'expert_score': expert_score
            })

        return {
            'overall_coverage_score': overall_coverage,
            'team_members': team_members,
            'skill_analysis': skill_analysis
        }

    def generate_strategic_analysis(self, coverage_report, skills_list, solicitation_data):
        """Generate AI-powered gap analysis using Groq API."""
        if not self.groq_client:
            # Basic fallback analysis
            low_skills = [s['skill'] for s in coverage_report['skill_analysis'] if s['level'] == 'Low']
            basic_analysis = f"""
BASIC STRATEGIC ANALYSIS (Groq API not available):

Team Coverage Score: {coverage_report['overall_coverage_score']:.2f}/100

STRENGTHS:
- Team assembled using advanced affinity matching
- {len(coverage_report['team_members'])} selected team members
- Covers {len(skills_list)} required skill areas

POTENTIAL GAPS:
{f"- Low coverage areas: {', '.join(low_skills)}" if low_skills else "- No significant gaps identified"}

RECOMMENDATIONS:
- Review low-coverage skills for additional expertise
- Consider collaboration with external partners if needed
- Leverage team members' top skills for proposal strength
"""
            return basic_analysis.strip()

        print("🤖 Generating strategic analysis with Groq API...")

        # Create detailed prompt for Groq
        team_info = "\n".join([f"- {member['name']} (avg affinity: {member['avg_affinity']:.2f})"
                              for member in coverage_report['team_members']])

        low_skills = [s for s in coverage_report['skill_analysis'] if s['level'] == 'Low']
        medium_skills = [s for s in coverage_report['skill_analysis'] if s['level'] == 'Medium']
        high_skills = [s for s in coverage_report['skill_analysis'] if s['level'] == 'High']

        prompt = f"""
Analyze this research team's fit for the NSF solicitation titled: "{solicitation_data.get('title', 'N/A')}"

TEAM COMPOSITION:
{team_info}

COVERAGE ANALYSIS:
- Overall team coverage score: {coverage_report['overall_coverage_score']:.2f}/100
- High coverage skills ({len(high_skills)}): {', '.join([s['skill'] for s in high_skills[:5]])}
- Medium coverage skills ({len(medium_skills)}): {', '.join([s['skill'] for s in medium_skills[:5]])}
- Low coverage skills ({len(low_skills)}): {', '.join([s['skill'] for s in low_skills])}

Please provide a strategic analysis covering:
1. Team strengths and competitive advantages, and why
2. Critical gaps and risks, and strategic ways to combat that. Be assertive
3. Specific recommendations for proposal development, intellectual and based on solicitation
4. Potential collaboration or recruitment strategies, also suggest which areas of faculty should be targeted
5. Overall competitiveness assessment

Keep the analysis practical and actionable for proposal development.
"""

        try:
            response = self.groq_client.chat.completions.create(
                model="llama3-70b-8192",  # Free Groq model
                messages=[{"role": "user", "content": prompt}],
                max_tokens=2000,
                temperature=0.5
            )
            analysis = response.choices[0].message.content
            print("✅ Strategic analysis generated with Groq API")
            return analysis
        except Exception as e:
            print(f"❌ Groq API analysis failed: {e}")
            # Fallback to basic analysis
            low_skills_list = [s['skill'] for s in low_skills]
            return f"Groq API failed. Basic analysis: Team coverage is {coverage_report['overall_coverage_score']:.2f}/100. Review low-coverage skills: {', '.join(low_skills_list) if low_skills_list else 'None identified'}."

    def dream_team_by_rankings(self, affinity_df, team_size=4):
        """Alternative approach: Simply select top N researchers by overall ranking."""
        print(f"\n📊 ALTERNATIVE: Dream Team by Rankings (Top {team_size})")
        print("=" * 50)

        # Sort by average affinity (overall performance)
        researcher_averages = affinity_df.mean(axis=1).sort_values(ascending=False)
        top_researchers = researcher_averages.head(team_size)

        team_indices = [affinity_df.index.get_loc(name) for name in top_researchers.index]
        _, coverage = self.calculate_team_coverage(affinity_df, team_indices)

        print("Selected team (by ranking):")
        for i, (name, avg_score) in enumerate(top_researchers.items()):
            role = "PI" if i == 0 else f"Co-I {i}"
            print(f"   {i+1}. {name} ({role}) - Avg Score: {avg_score:.2f}")

        print(f"Team coverage by rankings: {coverage:.2f}")
        return team_indices, coverage

    def compare_team_strategies(self, affinity_df, skills_list):
        """Compare different team selection strategies."""
        print("\n🔬 COMPARING TEAM SELECTION STRATEGIES")
        print("=" * 60)

        # Strategy 1: Hybrid approach (lock top 2 + coverage optimization) - PREFERRED
        hybrid_indices, hybrid_history = self.dream_team_hybrid_strategy(affinity_df, guaranteed_top_n=2, max_team_size=4)
        hybrid_coverage = self.calculate_team_coverage(affinity_df, hybrid_indices)[1]

        # Strategy 2: Pure greedy algorithm (optimized coverage)
        greedy_indices, greedy_history = self.dream_team_greedy_algorithm(affinity_df, marginal_threshold=0.25, max_team_size=4)
        greedy_coverage = self.calculate_team_coverage(affinity_df, greedy_indices)[1]

        # Strategy 3: Top performers by ranking
        ranking_indices, ranking_coverage = self.dream_team_by_rankings(affinity_df, team_size=4)

        print(f"\n📊 STRATEGY COMPARISON:")
        print(f"🎯 Hybrid Strategy ({len(hybrid_indices)} members): {hybrid_coverage:.2f} coverage")
        print(f"🤖 Pure Greedy ({len(greedy_indices)} members): {greedy_coverage:.2f} coverage")
        print(f"📈 Top Rankings (4 members): {ranking_coverage:.2f} coverage")

        # Show detailed comparison
        strategies = [
            ("HYBRID (Top 2 + Coverage)", hybrid_indices, hybrid_coverage),
            ("PURE GREEDY", greedy_indices, greedy_coverage),
            ("TOP RANKINGS", ranking_indices, ranking_coverage)
        ]

        for strategy_name, indices, coverage in strategies:
            print(f"\n{strategy_name}:")
            for i, idx in enumerate(indices):
                name = affinity_df.index[idx]
                avg_score = affinity_df.iloc[idx].mean()
                role = "PI" if i == 0 else f"Co-I {i}"
                print(f"   {name} ({role}) - Avg Score: {avg_score:.2f}")

        # Determine which strategy to use - prioritize hybrid if competitive
        best_coverage = max(hybrid_coverage, greedy_coverage, ranking_coverage)

        if hybrid_coverage >= best_coverage - 1.0:  # Hybrid is within 1 point of best
            print(f"\n🎯 RECOMMENDATION: Use Hybrid Strategy")
            print(f"   ✅ Guarantees top performers (grant experience + keyword matching)")
            print(f"   ✅ Optimizes coverage for remaining slots")
            print(f"   📊 Coverage: {hybrid_coverage:.2f} (competitive with best: {best_coverage:.2f})")
            best_indices = hybrid_indices
            best_strategy = "hybrid"
        elif ranking_coverage > greedy_coverage:
            print(f"\n📈 RECOMMENDATION: Use Rankings Team (+{ranking_coverage - greedy_coverage:.2f} better coverage)")
            best_indices = ranking_indices
            best_strategy = "rankings"
        else:
            print(f"\n🤖 RECOMMENDATION: Use Greedy Team (+{greedy_coverage - ranking_coverage:.2f} better coverage)")
            best_indices = greedy_indices
            best_strategy = "greedy"

        return best_indices, best_strategy, {
            "hybrid": hybrid_coverage,
            "greedy": greedy_coverage,
            "rankings": ranking_coverage
        }

    def get_team_evidence(self, affinity_df, team_indices, skills_list):
        """Collect supporting evidence (papers) for each skill covered by the team."""
        print("\n📚 Collecting Supporting Evidence...")
        team_evidence = {} # skill -> list of paper IDs

        for i, skill in enumerate(skills_list):
            # Find the best researcher for this skill in the selected team
            team_scores = affinity_df.iloc[team_indices, i]
            if not team_scores.empty and team_scores.max() > 0:
                best_member_name = team_scores.idxmax()
                best_member_idx_in_affinity_df = affinity_df.index.get_loc(best_member_name)
                best_member_id = None
                # Find the OpenAlex ID for the best member
                for match in self.researcher_metadata.itertuples():
                    if match.researcher_name == best_member_name:
                         best_member_id = match.researcher_openalex_id
                         break

                if best_member_id and best_member_id in self.evidence_index:
                    # Get papers from evidence index for this researcher and skill
                    skill_in_index = None
                    # Find the closest matching skill name in the evidence index
                    for index_skill in self.evidence_index[best_member_id].keys():
                        if skill.lower() in index_skill.lower() or index_skill.lower() in skill.lower():
                            skill_in_index = index_skill
                            break

                    if skill_in_index and self.evidence_index[best_member_id][skill_in_index]:
                        # Select top 3 papers for this skill from the best member
                        # Prioritize by citations or recency if available, otherwise just take first few
                        papers = self.evidence_index[best_member_id][skill_in_index]
                        # For simplicity, just take the first 3 papers
                        team_evidence[skill] = list(set(papers))[:3]
                    else:
                        team_evidence[skill] = [] # No specific evidence found for this skill
                else:
                     team_evidence[skill] = [] # Researcher not found in evidence index

            else:
                 team_evidence[skill] = [] # No team member covers this skill well

        print(f"✅ Collected evidence for {len([k for k,v in team_evidence.items() if v])}/{len(skills_list)} skills")
        return team_evidence


    def create_dream_team_report(self, affinity_df, skills_list, solicitation_data, max_team_size=4):
        """Create comprehensive dream team report with strategic analysis."""
        print("\n🚀 CREATING DREAM TEAM STRATEGIC REPORT")
        print("=" * 60)

        # Run team selection algorithm (using the hybrid strategy as recommended)
        team_indices, selection_history = self.dream_team_hybrid_strategy(affinity_df, guaranteed_top_n=2, max_team_size=max_team_size)

        # Generate coverage analysis
        coverage_report = self.generate_coverage_report(affinity_df, team_indices, skills_list)

        # Collect evidence
        team_evidence = self.get_team_evidence(affinity_df, team_indices, skills_list)


        # Generate strategic analysis
        strategic_analysis = self.generate_strategic_analysis(coverage_report, skills_list, solicitation_data)

        return DreamTeamReport(
            team_members=coverage_report['team_members'],
            overall_coverage_score=coverage_report['overall_coverage_score'],
            skill_analysis=coverage_report['skill_analysis'],
            strategic_analysis=strategic_analysis,
            selection_history=selection_history,
            generated_at=datetime.now().isoformat()
        ), team_evidence

    def format_markdown_report(self, dream_team_report, solicitation_title, team_evidence):
        """Format the strategic report as a human-readable Markdown file."""
        report = f"# NSF Dream Team Strategic Report\n\n"
        report += f"**Solicitation:** {solicitation_title}\n"
        report += f"**Generated:** {dream_team_report.generated_at}\n"
        report += f"**Overall Team Coverage Score:** **`{dream_team_report.overall_coverage_score:.2f} / 100`**\n\n"

        # Team Summary Table
        report += f"## 🏆 Recommended Dream Team\n\n"
        report += "| Role | Researcher | Avg. Affinity | Top Expertise Areas |\n"
        report += "|:---|:---|:---:|:---|\n"
        for i, member in enumerate(dream_team_report.team_members):
            role = "**Principal Investigator (PI)**" if i == 0 else f"Co-Investigator {i}"
            top_skills = ", ".join([s['skill'] for s in member['top_skills']])
            report += f"| {role} | {member['name']} | `{member['avg_affinity']:.2f}` | {top_skills} |\n"

        # Coverage Analysis Table
        report += f"\n## 📊 Skills Coverage Analysis\n\n"
        report += "| Skill / Expertise Area | Coverage | Level | Primary Expert | Supporting Evidence |\n" # Added Evidence column
        report += "|:---|:---:|:---|:---|:---|\n" # Updated table format
        for skill in sorted(dream_team_report.skill_analysis, key=lambda x: x['coverage_score'], reverse=True):
            level_emoji = "🟢" if skill['level'] == 'High' else "🟡" if skill['level'] == 'Medium' else "🔴"
            evidence_links = []
            if skill['skill'] in team_evidence and team_evidence[skill['skill']]:
                 evidence_links = [f"[Paper {j+1}]({paper_id})" for j, paper_id in enumerate(team_evidence[skill['skill']])]

            report += f"| {skill['skill']} | `{skill['coverage_score']:.2f}` | {level_emoji} {skill['level']} | {skill['expert']} | {', '.join(evidence_links) if evidence_links else 'None'} |\n" # Added evidence links


        # Strategic Analysis
        report += f"\n## 🧠 AI-Powered Strategic Analysis (via Groq)\n\n"
        report += dream_team_report.strategic_analysis + "\n"

        # Selection History
        report += f"\n## 🎯 Team Selection Process\n\n"
        for entry in dream_team_report.selection_history:
            report += f"**Step {entry['step']}:** {entry['action']} - {entry['researcher_name']}\n"
            report += f"- Reason: {entry['reason']}\n"
            report += f"- Team Coverage: {entry['team_coverage']:.2f}\n\n"

        return report

    def run_complete_analysis(self, solicitation_obj, top_n_researchers=20, max_team_size=4):
        """Run the complete analysis pipeline: matching → affinity matrix → dream team → report."""
        print("🚀 STARTING COMPLETE UNIFIED ANALYSIS PIPELINE")
        print("=" * 70)

        # Phase 1: Researcher Matching
        print("\n📍 PHASE 1: RESEARCHER MATCHING & RANKING")
        matching_results = self.search_researchers(solicitation_obj)

        # Display top matches
        print("\n🏆 TOP 10 RESEARCHER MATCHES:")
        print(f"{'Rank':<4} {'Name':<30} {'Final':<8} {'Academic':<9} {'Sparse':<8} {'Dense':<8} {'Papers':<7}")
        print("-" * 80)
        for i, match in enumerate(matching_results.top_matches[:10], 1):
            print(f"{i:<4} {match.researcher_name[:29]:<30} "
                  f"{match.final_affinity_score:<8.1f} {match.academic_expertise_score:<9.1f} "
                  f"{match.s_sparse:<8.1f} {match.s_dense:<8.1f} {match.total_papers:<7}")

        # Phase 2: Affinity Matrix Creation
        print(f"\n📍 PHASE 2: AFFINITY MATRIX CREATION")
        affinity_df = self.create_affinity_matrix(matching_results, top_n_researchers)

        # Phase 3: Dream Team Assembly
        print(f"\n📍 PHASE 3: DREAM TEAM ASSEMBLY & STRATEGIC ANALYSIS")
        skills_list = matching_results.skills_analyzed
        solicitation_data = {'title': solicitation_obj.title}
        dream_team_report, team_evidence = self.create_dream_team_report(affinity_df, skills_list, solicitation_data, max_team_size)


        # Phase 4: Generate Reports
        print(f"\n📍 PHASE 4: REPORT GENERATION")
        markdown_report = self.format_markdown_report(dream_team_report, solicitation_obj.title, team_evidence)

        # Save all results
        timestamp = int(time.time())
        base_filename = f"unified_analysis_{timestamp}"

        # Save matching results
        matching_file = f"{output_dir}/{base_filename}_matching_results.json"
        with open(matching_file, 'w', encoding='utf-8') as f:
            json.dump(asdict(matching_results), f, indent=2, ensure_ascii=False, default=str)
        print(f"💾 Saved matching results: {matching_file}")

        # Save affinity matrix
        affinity_file = f"{output_dir}/{base_filename}_affinity_matrix.csv"
        affinity_df.to_csv(affinity_file)
        print(f"💾 Saved affinity matrix: {affinity_file}")

        # Save dream team report
        report_file = f"{output_dir}/{base_filename}_dream_team_report.json"
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump(asdict(dream_team_report), f, indent=2, ensure_ascii=False, default=str)
        print(f"💾 Saved dream team report: {report_file}")

        # Save markdown report
        markdown_file = f"{output_dir}/{base_filename}_strategic_report.md"
        with open(markdown_file, 'w', encoding='utf-8') as f:
            f.write(markdown_report)
        print(f"💾 Saved markdown report: {markdown_file}")

        # Save evidence data
        evidence_file = f"{output_dir}/{base_filename}_team_evidence.json"
        with open(evidence_file, 'w', encoding='utf-8') as f:
            json.dump(team_evidence, f, indent=2, ensure_ascii=False, default=str)
        print(f"💾 Saved team evidence: {evidence_file}")

        # Display final summary
        self.display_final_summary(matching_results, dream_team_report, team_evidence)

        return {
            'matching_results': matching_results,
            'affinity_matrix': affinity_df,
            'dream_team_report': dream_team_report,
            'team_evidence': team_evidence,
            'markdown_report': markdown_report,
            'files_saved': {
                'matching': matching_file,
                'affinity': affinity_file,
                'report': report_file,
                'markdown': markdown_file,
                'evidence': evidence_file
            }
        }

    def display_final_summary(self, matching_results, dream_team_report, team_evidence):
        """Display a comprehensive summary of the analysis."""
        print("\n" + "="*70)
        print("📋 UNIFIED ANALYSIS SUMMARY")
        print("="*70)

        print(f"🔍 RESEARCHER MATCHING:")
        print(f"   • Analyzed {matching_results.total_researchers} total researchers")
        print(f"   • {matching_results.eligible_researchers} eligible researchers")
        print(f"   • Processing time: {matching_results.processing_time_seconds}s")

        print(f"\n🎯 DREAM TEAM ASSEMBLY:")
        print(f"   • Selected {len(dream_team_report.team_members)} optimal team members")
        print(f"   • Strategy: Hybrid (Top 2 performers + coverage optimization)")
        print(f"   • Overall coverage score: {dream_team_report.overall_coverage_score:.2f}/100")

        print(f"\n👥 RECOMMENDED TEAM:")
        for i, member in enumerate(dream_team_report.team_members):
            role = "PI" if i == 0 else f"Co-I {i}"
            print(f"   • {member['name']} ({role}) - Affinity: {member['avg_affinity']:.2f}")

        # Check for gaps
        low_skills = [s for s in dream_team_report.skill_analysis if s['level'] == 'Low']
        medium_skills = [s for s in dream_team_report.skill_analysis if s['level'] == 'Medium']
        high_skills = [s for s in dream_team_report.skill_analysis if s['level'] == 'High']

        print(f"\n📊 SKILL COVERAGE BREAKDOWN:")
        print(f"   🟢 High Coverage: {len(high_skills)} skills")
        print(f"   🟡 Medium Coverage: {len(medium_skills)} skills")
        print(f"   🔴 Low Coverage: {len(low_skills)} skills")

        if low_skills:
            print(f"\n🔴 TOP GAPS TO ADDRESS:")
            for skill in sorted(low_skills, key=lambda x: x['coverage_score'])[:3]:
                print(f"   • {skill['skill']} (Coverage: {skill['coverage_score']:.1f})")
            if len(low_skills) > 3:
                print(f"   • ... and {len(low_skills) - 3} more (see full report)")
        else:
            print(f"\n🟢 EXCELLENT COVERAGE: No significant skill gaps identified")

        print(f"\n✅ ANALYSIS COMPLETE! Check saved files for detailed reports.")
        print(f"📚 EVIDENCE: {sum(len(evidence) for evidence in team_evidence.values())} supporting papers collected")
        print(f"💡 TIP: Review the markdown report for AI-powered strategic recommendations and evidence.")

# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

def run_unified_analysis():
    """Main function to run the complete unified analysis."""

    # Load solicitation
    print("📋 Loading solicitation...")
    with open(solicitation_file, 'r', encoding='utf-8') as f:
        solicitation_data = json.load(f)

    # Convert to object
    class SolicitationObj:
        def __init__(self, data):
            for key, value in data.items():
                setattr(self, key, value)

    solicitation_obj = SolicitationObj(solicitation_data)
    print(f"✅ Loaded: {solicitation_obj.title}")

    # Initialize unified system
    system = UnifiedResearcherSystem(researcher_data_dir)

    # Run complete analysis
    results = system.run_complete_analysis(
        solicitation_obj,
        top_n_researchers=20,  # Adjust as needed
        max_team_size=4       # Adjust as needed
    )

    return results

# RUN THE UNIFIED SYSTEM
if __name__ == "__main__":
    results = run_unified_analysis()

In [ ]:
# ==============================================================================
# COMPLETE UNIFIED RESEARCHER MATCHING & DREAM TEAM SYSTEM
# ==============================================================================
import json
import numpy as np
import pandas as pd
import pickle
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, asdict
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import time
import warnings
from datetime import datetime
import os

# Try to import Groq API (optional)
try:
    from groq import Groq
    GROQ_AVAILABLE = True
except ImportError:
    GROQ_AVAILABLE = False
    print("⚠️ Groq library not installed. GAP analysis will be basic.")
    print("   Install with: pip install groq")

# Try to import Google Colab userdata (optional)
try:
    from google.colab import userdata
    COLAB_AVAILABLE = True
except ImportError:
    COLAB_AVAILABLE = False

warnings.filterwarnings('ignore')

# ==============================================================================
# CONFIGURATION - CHANGE THESE PATHS
# ==============================================================================
solicitation_file = "/content/drive/MyDrive/datastore/NSF 23-506_ Expanding AI Innovation through Capacity Building and Partnerships (ExpandAI) _ NSF - National Science Foundation_enhanced_analysis.json"
researcher_data_dir = "/content/drive/MyDrive/datastore/v2_DATA"
output_dir = "/content/drive/MyDrive/datastore"

# ==============================================================================
# GROQ API SETUP (OPTIONAL - FOR AI-POWERED STRATEGIC ANALYSIS)
# ==============================================================================
# 1. Get free API key at: https://console.groq.com/
# 2. In Google Colab: Go to left sidebar > Secrets > Add new secret
#    Name: GROQ_API_KEY, Value: your_api_key_here
# 3. Or set environment variable: export GROQ_API_KEY=your_api_key_here
# ==============================================================================

# ==============================================================================
# CORE CLASSES
# ==============================================================================
@dataclass
class ResearcherMatch:
    researcher_id: str
    researcher_name: str
    academic_expertise_score: float
    s_sparse: float
    s_dense: float
    f_ge: float
    final_affinity_score: float
    total_papers: int
    eligibility_status: str

@dataclass
class MatchingResults:
    solicitation_title: str
    eligible_researchers: int
    total_researchers: int
    top_matches: List[ResearcherMatch]
    skills_analyzed: List[str]
    processing_time_seconds: float

@dataclass
class DreamTeamReport:
    team_members: List[Dict]
    overall_coverage_score: float
    skill_analysis: List[Dict]
    strategic_analysis: str
    selection_history: List[Dict]
    generated_at: str

class UnifiedResearcherSystem:
    """
    Complete system that combines researcher matching with dream team assembly
    and strategic GAP analysis using Claude API.
    """

    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
        self.alpha = 0.7  # TF-IDF weight
        self.beta = 0.3   # Dense weight
        self.groq_client = None
        self.setup_groq_api()
        self.load_data()

    def setup_groq_api(self):
        """Initialize Groq API client if available."""
        if not GROQ_AVAILABLE:
            print("⚠️ Groq API not available. Strategic analysis will be basic.")
            print("   Install with: pip install groq")
            return

        try:
            if COLAB_AVAILABLE:
                api_key = userdata.get('GROQ_API_KEY')
            else:
                api_key = os.environ.get('GROQ_API_KEY')

            if api_key:
                self.groq_client = Groq(api_key=api_key)
                print("✅ Groq API client initialized for strategic analysis")
            else:
                print("⚠️ Groq API key not found. Set GROQ_API_KEY.")
                print("   Get free API key at: https://console.groq.com/")
        except Exception as e:
            print(f"⚠️ Groq API setup failed: {e}. Strategic analysis will be basic.")

    def load_data(self):
        """Load all preprocessed researcher data."""
        print("📂 Loading preprocessed data...")

        # Handle pickle loading issue - define dummy class if needed
        try:
            # Try to load TF-IDF model normally first
            with open(self.data_dir / 'tfidf_model.pkl', 'rb') as f:
                self.tfidf_model = pickle.load(f)
        except AttributeError as e:
            if "ResearcherProfileProcessor" in str(e):
                print("🔧 Fixing pickle compatibility issue...")
                # Create a dummy class to handle the pickle loading
                import sys
                class ResearcherProfileProcessor:
                    def comma_tokenizer(self, text: str):
                        return [token.strip() for token in text.split(',') if token.strip()]

                # Add the class to the current module
                sys.modules[__name__].ResearcherProfileProcessor = ResearcherProfileProcessor

                # Try loading again
                with open(self.data_dir / 'tfidf_model.pkl', 'rb') as f:
                    self.tfidf_model = pickle.load(f)
                print("✅ TF-IDF model loaded successfully")
            else:
                raise e

        # Load researcher vectors
        researcher_data = np.load(self.data_dir / 'researcher_vectors.npz', allow_pickle=True)
        vectors = researcher_data['vectors']
        researcher_ids = researcher_data['researcher_ids']
        self.researcher_vectors = dict(zip(researcher_ids, vectors))

        # Load paper embeddings
        conceptual_data = np.load(self.data_dir / 'conceptual_profiles.npz', allow_pickle=True)
        embeddings = conceptual_data['embeddings']
        work_ids = conceptual_data['work_ids']
        self.conceptual_profiles = dict(zip(work_ids, embeddings))

        # Load evidence index
        with open(self.data_dir / 'evidence_index.json', 'r') as f:
            self.evidence_index = json.load(f)

        # Load metadata
        self.researcher_metadata = pd.read_parquet(self.data_dir / 'researcher_metadata.parquet')

        # Load sentence model
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

        print(f"✅ Loaded {len(self.researcher_vectors)} researchers")
        self.diagnose_data_quality()

    def diagnose_data_quality(self):
        """Diagnose potential data quality issues."""
        print("\n🔍 DIAGNOSING DATA QUALITY")
        print("-" * 40)

        # Check TF-IDF model
        try:
            vocab_size = len(self.tfidf_model.get_feature_names_out())
            print(f"TF-IDF vocabulary size: {vocab_size}")

            vocab = list(self.tfidf_model.get_feature_names_out())
            print(f"First 20 TF-IDF features: {vocab[:20]}")
            print(f"Last 20 TF-IDF features: {vocab[-20:]}")

        except Exception as e:
            print(f"❌ Error accessing TF-IDF vocabulary: {e}")

        # Check researcher vectors
        print(f"Researcher vectors: {len(self.researcher_vectors)}")
        if self.researcher_vectors:
            sample_vector = next(iter(self.researcher_vectors.values()))
            print(f"Vector dimensions: {sample_vector.shape}")
            print(f"Sample vector sum: {sample_vector.sum():.4f}")

        # Check conceptual profiles
        print(f"Conceptual profiles: {len(self.conceptual_profiles)}")
        if self.conceptual_profiles:
            sample_embedding = next(iter(self.conceptual_profiles.values()))
            print(f"Embedding dimensions: {sample_embedding.shape}")

        # Check evidence index
        print(f"Evidence index researchers: {len(self.evidence_index)}")

        # Check overlap between evidence index and conceptual profiles
        all_evidence_papers = set()
        for researcher_papers in self.evidence_index.values():
            for topic_papers in researcher_papers.values():
                all_evidence_papers.update(topic_papers)

        conceptual_papers = set(self.conceptual_profiles.keys())
        overlap = all_evidence_papers.intersection(conceptual_papers)

        print(f"Papers in evidence index: {len(all_evidence_papers)}")
        print(f"Papers with embeddings: {len(conceptual_papers)}")
        print(f"Overlap: {len(overlap)}")

        if len(overlap) == 0:
            print("❌ CRITICAL: No overlap between evidence index and conceptual profiles!")
        elif len(overlap) < len(all_evidence_papers) * 0.5:
            print("⚠️ WARNING: Low overlap between evidence index and conceptual profiles!")

        print("-" * 40)

    def filter_eligibility(self, solicitation_obj, researchers):
        """Simple eligibility filtering."""
        eligible = set(researchers)
        eligibility = getattr(solicitation_obj, 'eligibility', {})

        # Early-career filter
        if eligibility and any('early' in str(v).lower() for v in eligibility.values() if v):
            early_career = self.researcher_metadata[
                self.researcher_metadata['first_publication_year'] >= 2015
            ]['researcher_openalex_id'].tolist()
            eligible = eligible.intersection(set(early_career))
            print(f"   Applied early-career filter: {len(eligible)} remain")

        # Grant experience filter
        if eligibility and any('grant' in str(v).lower() or 'funding' in str(v).lower() for v in eligibility.values() if v):
            experienced = self.researcher_metadata[
                self.researcher_metadata['grant_experience_factor'] > 0
            ]['researcher_openalex_id'].tolist()
            eligible = eligible.intersection(set(experienced))
            print(f"   Applied grant experience filter: {len(eligible)} remain")

        return list(eligible)

    def extract_keywords_from_skills(self, skills: List[str]) -> List[str]:
        """Extract keywords from solicitation skills using same logic as researcher topics."""
        import re

        stop_words = {'and', 'in', 'of', 'for', 'the', 'a', 'an', 'to', 'with', 'on', 'at', 'by',
                      'expertise', 'experience', 'knowledge', 'ability', 'skills', 'understanding',
                      'capacity', 'proficiency', 'e.g.', 'eg', 'including', 'such', 'as'}

        all_keywords = []

        for skill in skills:
            # Clean and split
            cleaned = re.sub(r'[^\w\s-]', ' ', skill.lower())
            words = cleaned.split()

            # Extract meaningful keywords
            for word in words:
                word = word.strip('-')
                if (len(word) >= 3 and
                    word not in stop_words and
                    not word.isdigit()):
                    all_keywords.append(word)

        return all_keywords

    def score_researcher(self, researcher_id, skills, solicitation_embedding, debug_mode=False):
        """Score a single researcher with optional debug output."""
        try:
            # Get metadata
            researcher_row = self.researcher_metadata[
                self.researcher_metadata['researcher_openalex_id'] == researcher_id
            ]
            if researcher_row.empty:
                return None

            researcher_name = researcher_row.iloc[0]['researcher_name']
            total_papers = int(researcher_row.iloc[0]['total_papers'])
            grant_factor = researcher_row.iloc[0]['grant_experience_factor']

            # Extract keywords and format with commas (same as researcher documents)
            solicitation_keywords = self.extract_keywords_from_skills(skills)
            solicitation_text = ', '.join(solicitation_keywords)

            if debug_mode:
                print(f"DEBUG - Researcher: {researcher_name}")
                print(f"  Original skills count: {len(skills)}")
                print(f"  Extracted keywords: {solicitation_keywords[:10]}...")
                print(f"  Solicitation text: {solicitation_text[:100]}...")

            # Calculate sparse score (TF-IDF)
            if researcher_id not in self.researcher_vectors:
                if debug_mode:
                    print(f"  WARNING: No TF-IDF vector for {researcher_id}")
                s_sparse = 0.0
            else:
                try:
                    solicitation_vector = self.tfidf_model.transform([solicitation_text])
                    researcher_vector = self.researcher_vectors[researcher_id].reshape(1, -1)

                    if debug_mode:
                        print(f"  Solicitation vector shape: {solicitation_vector.shape}")
                        print(f"  Researcher vector shape: {researcher_vector.shape}")
                        print(f"  Solicitation vector sum: {solicitation_vector.sum()}")
                        print(f"  Researcher vector sum: {researcher_vector.sum()}")
                        if hasattr(solicitation_vector, 'nnz'):
                             print(f"  Solicitation non-zero elements: {solicitation_vector.nnz}")


                    if solicitation_vector.sum() == 0 or researcher_vector.sum() == 0:
                        if debug_mode:
                            print(f"  WARNING: Zero vector detected")
                        s_sparse = 0.0
                    else:
                        similarity = cosine_similarity(solicitation_vector, researcher_vector)[0][0]
                        s_sparse = float(similarity * 100)
                        if debug_mode:
                            print(f"  TF-IDF similarity: {similarity}")
                except Exception as tfidf_error:
                    if debug_mode:
                        print(f"  TF-IDF ERROR: {tfidf_error}")
                    s_sparse = 0.0

            # Calculate dense score (max similarity across papers)
            s_dense = 0.0
            papers_checked = 0
            papers_found = 0
            if researcher_id in self.evidence_index:
                # Get all papers for this researcher from evidence index
                researcher_papers = []
                for topic_papers in self.evidence_index[researcher_id].values():
                    researcher_papers.extend(topic_papers)
                researcher_papers = list(set(researcher_papers))  # Remove duplicates

                if debug_mode:
                    print(f"  Papers from evidence index: {len(researcher_papers)}")

                max_sim = 0.0
                for paper_id in researcher_papers:
                    papers_checked += 1
                    if paper_id in self.conceptual_profiles:
                        papers_found += 1
                        paper_embedding = self.conceptual_profiles[paper_id]
                        try:
                            sim = cosine_similarity(
                                solicitation_embedding.reshape(1, -1),
                                paper_embedding.reshape(1, -1)
                            )[0][0]
                            max_sim = max(max_sim, sim)
                        except Exception as dense_error:
                            if debug_mode:
                                print(f"  Dense similarity error for paper {paper_id}: {dense_error}")

                s_dense = float(max_sim * 100)
                if debug_mode:
                    print(f"  Papers checked: {papers_checked}, Papers with embeddings: {papers_found}")
                    print(f"  Max dense similarity: {max_sim}")
            else:
                if debug_mode:
                    print(f"  No evidence index entry for researcher")

            # Calculate final scores
            f_ge = max(1.0, min(3.0, 1.0 + (grant_factor * 0.2)))
            academic_expertise = (self.alpha * s_sparse) + (self.beta * s_dense)
            final_score = academic_expertise * f_ge

            if debug_mode:
                print(f"  Final scores - Sparse: {s_sparse:.2f}, Dense: {s_dense:.2f}, Grant: {f_ge:.2f}")
                print(f"  Academic: {academic_expertise:.2f}, Final: {final_score:.2f}")
                print("-" * 50)

            return ResearcherMatch(
                researcher_id=researcher_id,
                researcher_name=researcher_name,
                academic_expertise_score=academic_expertise,
                s_sparse=s_sparse,
                s_dense=s_dense,
                f_ge=f_ge,
                final_affinity_score=final_score,
                total_papers=total_papers,
                eligibility_status="Eligible"
            )
        except Exception as e:
            if debug_mode:
                print(f"ERROR scoring {researcher_id}: {e}")
                import traceback
                traceback.print_exc()
            return None

    def search_researchers(self, solicitation_obj, debug_first_n=3):
        """Main search function to rank researchers."""
        start_time = time.time()

        print(f"\n🔍 ANALYZING: {solicitation_obj.title[:80]}...")

        # Get skills
        skills = solicitation_obj.required_skills_checklist
        print(f"📊 Skills to analyze: {len(skills)}")

        # Create solicitation embedding
        solicitation_text = f"{solicitation_obj.title}. {solicitation_obj.abstract}"
        solicitation_embedding = self.sentence_model.encode(solicitation_text)

        # Get all researchers
        all_researchers = list(self.researcher_metadata['researcher_openalex_id'])
        print(f"👥 Total researchers: {len(all_researchers)}")

        # Filter by eligibility
        eligible = self.filter_eligibility(solicitation_obj, all_researchers)
        print(f"✅ Eligible researchers: {len(eligible)}")

        # Score all eligible researchers
        print("🔄 Calculating scores...")
        matches = []
        debug_count = 0

        for researcher_id in eligible:
            # Debug first N researchers to avoid spam
            debug_mode = debug_count < debug_first_n
            if debug_mode:
                debug_count += 1

            result = self.score_researcher(researcher_id, skills, solicitation_embedding, debug_mode)

            if result:
                matches.append(result)

        # Sort by score
        matches.sort(key=lambda x: x.final_affinity_score, reverse=True)

        processing_time = time.time() - start_time

        return MatchingResults(
            solicitation_title=solicitation_obj.title,
            eligible_researchers=len(eligible),
            total_researchers=len(all_researchers),
            top_matches=matches,
            skills_analyzed=skills,
            processing_time_seconds=round(processing_time, 2)
        )

    def create_affinity_matrix(self, matching_results, top_n_researchers=20):
        """Create an affinity matrix from matching results."""
        print(f"\n📊 Creating affinity matrix for top {top_n_researchers} researchers...")

        # Get top researchers
        top_matches = matching_results.top_matches[:top_n_researchers]
        researcher_names = [match.researcher_name for match in top_matches]
        skills = matching_results.skills_analyzed

        # Create matrix: researchers × skills
        affinity_matrix = np.zeros((len(top_matches), len(skills)))

        for i, match in enumerate(top_matches):
            # For each researcher, calculate their affinity to each skill
            researcher_id = match.researcher_id

            for j, skill in enumerate(skills):
                # Calculate skill-specific affinity score
                skill_keywords = self.extract_keywords_from_skills([skill])
                skill_text = ', '.join(skill_keywords)

                try:
                    # TF-IDF similarity for this specific skill
                    skill_vector = self.tfidf_model.transform([skill_text])
                    researcher_vector = self.researcher_vectors[researcher_id].reshape(1, -1)
                    sparse_sim = cosine_similarity(skill_vector, researcher_vector)[0][0] * 100

                    # Dense similarity (using overall solicitation embedding as proxy)
                    # In practice, you might want to create skill-specific embeddings
                    # For simplicity, let's use the max dense score from the overall matching
                    dense_sim = match.s_dense # Use the pre-calculated max dense score for the researcher


                    # Combined affinity score for this skill
                    skill_affinity = (self.alpha * sparse_sim) + (self.beta * dense_sim)
                    affinity_matrix[i, j] = max(0, skill_affinity)  # Ensure non-negative

                except Exception as e:
                    # Fallback: use overall academic score
                    affinity_matrix[i, j] = match.academic_expertise_score

        # Create DataFrame
        affinity_df = pd.DataFrame(
            affinity_matrix,
            index=researcher_names,
            columns=[f"Skill_{i+1}: {skill}" for i, skill in enumerate(skills)]
        )

        print(f"✅ Created affinity matrix: {affinity_df.shape[0]} researchers × {affinity_df.shape[1]} skills")
        return affinity_df

    def calculate_team_coverage(self, affinity_df, team_indices):
        """Calculate team coverage scores for all skills."""
        if not team_indices:
            return np.array([0.0] * affinity_df.shape[1]), 0.0
        team_affinities = affinity_df.iloc[team_indices]
        skill_coverages = team_affinities.max(axis=0).values
        return skill_coverages, np.mean(skill_coverages)

    def calculate_marginal_gain(self, affinity_df, current_team_indices, candidate_index):
        """Calculate the marginal gain of adding a candidate to the team."""
        _, current_coverage = self.calculate_team_coverage(affinity_df, current_team_indices)
        _, new_coverage = self.calculate_team_coverage(affinity_df, current_team_indices + [candidate_index])
        return new_coverage - current_coverage

    def dream_team_hybrid_strategy(self, affinity_df, guaranteed_top_n=2, max_team_size=4):
        """
        Hybrid approach: Lock in top N performers, then optimize coverage for remaining slots.
        This ensures we get researchers with proven grant experience and keyword matching.
        """
        print(f"\n🎯 Running Hybrid Dream Team Strategy")
        print(f"   Step 1: Lock in top {guaranteed_top_n} performers")
        print(f"   Step 2: Optimize coverage for remaining {max_team_size - guaranteed_top_n} slots")
        print("=" * 50)

        # Phase 1: Lock in top performers (they earned their ranking!)
        researcher_averages = affinity_df.mean(axis=1).sort_values(ascending=False)
        top_performers = researcher_averages.head(guaranteed_top_n)

        selected_indices = []
        selection_history = []

        print("🔒 LOCKING IN TOP PERFORMERS:")
        for i, (name, avg_score) in enumerate(top_performers.items()):
            idx = affinity_df.index.get_loc(name)
            selected_indices.append(idx)
            role = "PI" if i == 0 else f"Co-PI {i}"

            selection_history.append({
                'step': i + 1,
                'action': f'Lock {role}',
                'researcher_name': name,
                'reason': f'Top {i+1} performer (avg: {avg_score:.2f}, proven track record)',
                'team_coverage': 0  # Will calculate after
            })
            print(f"   ✅ {name} ({role}) - Avg Score: {avg_score:.2f}")

        # Calculate coverage after locking in top performers
        _, coverage_after_top = self.calculate_team_coverage(affinity_df, selected_indices)
        print(f"   📊 Coverage after top {guaranteed_top_n}: {coverage_after_top:.2f}")

        # Update coverage in history
        for entry in selection_history:
            entry['team_coverage'] = coverage_after_top

        # Phase 2: Optimize remaining slots for coverage
        print(f"\n🎯 OPTIMIZING REMAINING {max_team_size - guaranteed_top_n} SLOTS FOR COVERAGE:")
        n_researchers = len(affinity_df)

        for step in range(guaranteed_top_n + 1, max_team_size + 1):
            # Calculate marginal gains for all remaining researchers
            gains = [(idx, self.calculate_marginal_gain(affinity_df, selected_indices, idx))
                     for idx in range(n_researchers) if idx not in selected_indices]

            if not gains:
                print(f"   ⚠️ No more candidates available")
                break

            # Sort by marginal gain and show top candidates
            top_candidates = sorted(gains, key=lambda x: x[1], reverse=True)[:5]
            best_candidate_idx, best_marginal_gain = top_candidates[0]

            print(f"   📊 Top candidates for slot {step - guaranteed_top_n}:")
            for i, (idx, gain) in enumerate(top_candidates):
                candidate_name = affinity_df.index[idx]
                candidate_avg = affinity_df.iloc[idx].mean()
                marker = "👑" if i == 0 else f"  {i+1}."
                print(f"      {marker} {candidate_name} (Avg: {candidate_avg:.2f}, Coverage Gain: +{gain:.2f})")

            # Add the best candidate for coverage
            if best_marginal_gain > 0.1:  # Lower threshold since we have strong foundation
                selected_indices.append(best_candidate_idx)
                _, new_coverage = self.calculate_team_coverage(affinity_df, selected_indices)

                selection_history.append({
                    'step': step,
                    'action': 'Add for Coverage',
                    'researcher_name': affinity_df.index[best_candidate_idx],
                    'reason': f'Best coverage gain (+{best_marginal_gain:.2f})',
                    'team_coverage': new_coverage
                })
                print(f"   ✅ Added: {affinity_df.index[best_candidate_idx]} (New Coverage: {new_coverage:.2f})")
            else:
                print(f"   🛑 Stopping: Marginal gain {best_marginal_gain:.2f} too small")
                break

        final_coverage = self.calculate_team_coverage(affinity_df, selected_indices)[1]
        print(f"\n🎯 Final Hybrid Team ({len(selected_indices)} members) with {final_coverage:.2f} coverage")
        print(f"   Strategy: Top {guaranteed_top_n} performers + coverage optimization")

        return selected_indices, selection_history

    def dream_team_greedy_algorithm(self, affinity_df, min_team_size=2, max_team_size=4, marginal_threshold=0.25):
        """Implement the greedy algorithm to select the best team."""
        print("\n🎯 Running Pure Greedy Algorithm...")
        print("=" * 50)
        n_researchers = len(affinity_df)
        selected_indices = []
        selection_history = []

        # Step 1: Select the best overall researcher as PI
        best_researcher_pos = affinity_df.mean(axis=1).idxmax()
        best_researcher_loc = affinity_df.index.get_loc(best_researcher_pos)
        selected_indices.append(best_researcher_loc)
        _, initial_coverage = self.calculate_team_coverage(affinity_df, selected_indices)

        selection_history.append({
            'step': 1, 'action': 'Select PI',
            'researcher_name': affinity_df.index[best_researcher_loc],
            'reason': 'Highest average affinity score',
            'team_coverage': initial_coverage
        })
        print(f"🏆 Step 1 - PI Selection: {affinity_df.index[best_researcher_loc]} (Coverage: {initial_coverage:.2f})")

        # Step 2-N: Iteratively add members with the highest marginal gain
        for step in range(2, max_team_size + 1):
            gains = [(idx, self.calculate_marginal_gain(affinity_df, selected_indices, idx))
                     for idx in range(n_researchers) if idx not in selected_indices]
            if not gains:
                break

            best_candidate_idx, best_marginal_gain = max(gains, key=lambda item: item[1])

            # Show top 3 candidates for transparency
            top_candidates = sorted(gains, key=lambda x: x[1], reverse=True)[:3]
            print(f"   📊 Top candidates for Step {step}:")
            for i, (idx, gain) in enumerate(top_candidates):
                candidate_name = affinity_df.index[idx]
                print(f"      {i+1}. {candidate_name} (Marginal gain: +{gain:.2f})")

            # More flexible stopping criteria
            should_add = (
                best_marginal_gain > marginal_threshold or  # Significant gain
                len(selected_indices) < min_team_size or    # Haven't reached minimum
                (step <= 4 and best_marginal_gain > 0.1)   # Force at least 4 if minimal gain
            )

            if should_add:
                selected_indices.append(best_candidate_idx)
                _, new_coverage = self.calculate_team_coverage(affinity_df, selected_indices)
                selection_history.append({
                    'step': step, 'action': 'Add Member',
                    'researcher_name': affinity_df.index[best_candidate_idx],
                    'reason': f'Maximum marginal gain (+{best_marginal_gain:.2f})',
                    'team_coverage': new_coverage
                })
                print(f"✅ Step {step} - Added: {affinity_df.index[best_candidate_idx]} (New Coverage: {new_coverage:.2f})")
            else:
                print(f"🛑 Step {step} - Stopping: Marginal gain {best_marginal_gain:.2f} below threshold {marginal_threshold}")
                break

        final_coverage = self.calculate_team_coverage(affinity_df, selected_indices)[1]
        print(f"\n🎯 Final Dream Team ({len(selected_indices)} members) with {final_coverage:.2f} coverage.")
        return selected_indices, selection_history

    def generate_coverage_report(self, affinity_df, team_indices, skills_list):
        """Generate a detailed coverage report for the selected team."""
        skill_coverages, overall_coverage = self.calculate_team_coverage(affinity_df, team_indices)

        team_members = []
        for idx in team_indices:
            scores = affinity_df.iloc[idx]
            top_skills = [{'skill': skills_list[i], 'score': scores[i]}
                         for i in scores.argsort()[-3:][::-1]]
            team_members.append({
                'name': affinity_df.index[idx],
                'avg_affinity': scores.mean(),
                'top_skills': top_skills
            })

        skill_analysis = []
        for i, (skill, coverage) in enumerate(zip(skills_list, skill_coverages)):
            team_scores = affinity_df.iloc[team_indices, i]
            if not team_scores.empty: # Handle case where team is empty or skill has no coverage
                best_member_name = team_scores.idxmax()
                expert_score = team_scores.max()
            else:
                best_member_name = "N/A"
                expert_score = 0.0


            skill_analysis.append({
                'skill': skill,
                'coverage_score': coverage,
                'level': 'High' if coverage >= 70 else 'Medium' if coverage >= 40 else 'Low',
                'expert': best_member_name,
                'expert_score': expert_score
            })

        return {
            'overall_coverage_score': overall_coverage,
            'team_members': team_members,
            'skill_analysis': skill_analysis
        }

    def generate_strategic_analysis(self, coverage_report, skills_list, solicitation_data):
        """Generate AI-powered gap analysis using Groq API."""
        if not self.groq_client:
            # Basic fallback analysis
            low_skills = [s['skill'] for s in coverage_report['skill_analysis'] if s['level'] == 'Low']
            basic_analysis = f"""
BASIC STRATEGIC ANALYSIS (Groq API not available):

Team Coverage Score: {coverage_report['overall_coverage_score']:.2f}/100

STRENGTHS:
- Team assembled using advanced affinity matching
- {len(coverage_report['team_members'])} selected team members
- Covers {len(skills_list)} required skill areas

POTENTIAL GAPS:
{f"- Low coverage areas: {', '.join(low_skills)}" if low_skills else "- No significant gaps identified"}

RECOMMENDATIONS:
- Review low-coverage skills for additional expertise
- Consider collaboration with external partners if needed
- Leverage team members' top skills for proposal strength
"""
            return basic_analysis.strip()

        print("🤖 Generating strategic analysis with Groq API...")

        # Create detailed prompt for Groq
        team_info = "\n".join([f"- {member['name']} (avg affinity: {member['avg_affinity']:.2f})"
                              for member in coverage_report['team_members']])

        low_skills = [s for s in coverage_report['skill_analysis'] if s['level'] == 'Low']
        medium_skills = [s for s in coverage_report['skill_analysis'] if s['level'] == 'Medium']
        high_skills = [s for s in coverage_report['skill_analysis'] if s['level'] == 'High']

        prompt = f"""
Analyze this research team's fit for the NSF solicitation titled: "{solicitation_data.get('title', 'N/A')}"

TEAM COMPOSITION:
{team_info}

COVERAGE ANALYSIS:
- Overall team coverage score: {coverage_report['overall_coverage_score']:.2f}/100
- High coverage skills ({len(high_skills)}): {', '.join([s['skill'] for s in high_skills[:5]])}
- Medium coverage skills ({len(medium_skills)}): {', '.join([s['skill'] for s in medium_skills[:5]])}
- Low coverage skills ({len(low_skills)}): {', '.join([s['skill'] for s in low_skills])}

Please provide a strategic analysis covering:
1. Team strengths and competitive advantages
2. Critical gaps and risks
3. Specific recommendations for proposal development
4. Potential collaboration or recruitment strategies
5. Overall competitiveness assessment

Keep the analysis practical and actionable for proposal development.
"""

        try:
            response = self.groq_client.chat.completions.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",  # Free Groq model
                messages=[{"role": "user", "content": prompt}],
                max_tokens=2000,
                temperature=0.5
            )
            analysis = response.choices[0].message.content
            print("✅ Strategic analysis generated with Groq API")
            return analysis
        except Exception as e:
            print(f"❌ Groq API analysis failed: {e}")
            # Fallback to basic analysis
            low_skills_list = [s['skill'] for s in low_skills]
            return f"Groq API failed. Basic analysis: Team coverage is {coverage_report['overall_coverage_score']:.2f}/100. Review low-coverage skills: {', '.join(low_skills_list) if low_skills_list else 'None identified'}."

    def dream_team_by_rankings(self, affinity_df, team_size=4):
        """Alternative approach: Simply select top N researchers by overall ranking."""
        print(f"\n📊 ALTERNATIVE: Dream Team by Rankings (Top {team_size})")
        print("=" * 50)

        # Sort by average affinity (overall performance)
        researcher_averages = affinity_df.mean(axis=1).sort_values(ascending=False)
        top_researchers = researcher_averages.head(team_size)

        team_indices = [affinity_df.index.get_loc(name) for name in top_researchers.index]
        _, coverage = self.calculate_team_coverage(affinity_df, team_indices)

        print("Selected team (by ranking):")
        for i, (name, avg_score) in enumerate(top_researchers.items()):
            role = "PI" if i == 0 else f"Co-I {i}"
            print(f"   {i+1}. {name} ({role}) - Avg Score: {avg_score:.2f}")

        print(f"Team coverage by rankings: {coverage:.2f}")
        return team_indices, coverage

    def compare_team_strategies(self, affinity_df, skills_list):
        """Compare different team selection strategies."""
        print("\n🔬 COMPARING TEAM SELECTION STRATEGIES")
        print("=" * 60)

        # Strategy 1: Hybrid approach (lock top 2 + coverage optimization) - PREFERRED
        hybrid_indices, hybrid_history = self.dream_team_hybrid_strategy(affinity_df, guaranteed_top_n=2, max_team_size=4)
        hybrid_coverage = self.calculate_team_coverage(affinity_df, hybrid_indices)[1]

        # Strategy 2: Pure greedy algorithm (optimized coverage)
        greedy_indices, greedy_history = self.dream_team_greedy_algorithm(affinity_df, marginal_threshold=0.25, max_team_size=4)
        greedy_coverage = self.calculate_team_coverage(affinity_df, greedy_indices)[1]

        # Strategy 3: Top performers by ranking
        ranking_indices, ranking_coverage = self.dream_team_by_rankings(affinity_df, team_size=4)

        print(f"\n📊 STRATEGY COMPARISON:")
        print(f"🎯 Hybrid Strategy ({len(hybrid_indices)} members): {hybrid_coverage:.2f} coverage")
        print(f"🤖 Pure Greedy ({len(greedy_indices)} members): {greedy_coverage:.2f} coverage")
        print(f"📈 Top Rankings (4 members): {ranking_coverage:.2f} coverage")

        # Show detailed comparison
        strategies = [
            ("HYBRID (Top 2 + Coverage)", hybrid_indices, hybrid_coverage),
            ("PURE GREEDY", greedy_indices, greedy_coverage),
            ("TOP RANKINGS", ranking_indices, ranking_coverage)
        ]

        for strategy_name, indices, coverage in strategies:
            print(f"\n{strategy_name}:")
            for i, idx in enumerate(indices):
                name = affinity_df.index[idx]
                avg_score = affinity_df.iloc[idx].mean()
                role = "PI" if i == 0 else f"Co-I {i}"
                print(f"   {name} ({role}) - Avg Score: {avg_score:.2f}")

        # Determine which strategy to use - prioritize hybrid if competitive
        best_coverage = max(hybrid_coverage, greedy_coverage, ranking_coverage)

        if hybrid_coverage >= best_coverage - 1.0:  # Hybrid is within 1 point of best
            print(f"\n🎯 RECOMMENDATION: Use Hybrid Strategy")
            print(f"   ✅ Guarantees top performers (grant experience + keyword matching)")
            print(f"   ✅ Optimizes coverage for remaining slots")
            print(f"   📊 Coverage: {hybrid_coverage:.2f} (competitive with best: {best_coverage:.2f})")
            best_indices = hybrid_indices
            best_strategy = "hybrid"
        elif ranking_coverage > greedy_coverage:
            print(f"\n📈 RECOMMENDATION: Use Rankings Team (+{ranking_coverage - greedy_coverage:.2f} better coverage)")
            best_indices = ranking_indices
            best_strategy = "rankings"
        else:
            print(f"\n🤖 RECOMMENDATION: Use Greedy Team (+{greedy_coverage - ranking_coverage:.2f} better coverage)")
            best_indices = greedy_indices
            best_strategy = "greedy"

        return best_indices, best_strategy, {
            "hybrid": hybrid_coverage,
            "greedy": greedy_coverage,
            "rankings": ranking_coverage
        }

    def get_team_evidence(self, affinity_df, team_indices, skills_list):
        """Collect supporting evidence (papers) for each skill covered by the team."""
        print("\n📚 Collecting Supporting Evidence...")
        team_evidence = {} # skill -> list of paper IDs

        for i, skill in enumerate(skills_list):
            # Find the best researcher for this skill in the selected team
            team_scores = affinity_df.iloc[team_indices, i]
            if not team_scores.empty and team_scores.max() > 0:
                best_member_name = team_scores.idxmax()
                best_member_idx_in_affinity_df = affinity_df.index.get_loc(best_member_name)
                best_member_id = None
                # Find the OpenAlex ID for the best member
                for match in self.researcher_metadata.itertuples():
                    if match.researcher_name == best_member_name:
                         best_member_id = match.researcher_openalex_id
                         break

                if best_member_id and best_member_id in self.evidence_index:
                    # Get papers from evidence index for this researcher and skill
                    skill_in_index = None
                    # Find the closest matching skill name in the evidence index
                    for index_skill in self.evidence_index[best_member_id].keys():
                        if skill.lower() in index_skill.lower() or index_skill.lower() in skill.lower():
                            skill_in_index = index_skill
                            break

                    if skill_in_index and self.evidence_index[best_member_id][skill_in_index]:
                        # Select top 3 papers for this skill from the best member
                        # Prioritize by citations or recency if available, otherwise just take first few
                        papers = self.evidence_index[best_member_id][skill_in_index]
                        # For simplicity, just take the first 3 papers
                        team_evidence[skill] = list(set(papers))[:3]
                    else:
                        team_evidence[skill] = [] # No specific evidence found for this skill
                else:
                     team_evidence[skill] = [] # Researcher not found in evidence index

            else:
                 team_evidence[skill] = [] # No team member covers this skill well

        print(f"✅ Collected evidence for {len([k for k,v in team_evidence.items() if v])}/{len(skills_list)} skills")
        return team_evidence


    def create_dream_team_report(self, affinity_df, skills_list, solicitation_data, max_team_size=4):
        """Create comprehensive dream team report with strategic analysis."""
        print("\n🚀 CREATING DREAM TEAM STRATEGIC REPORT")
        print("=" * 60)

        # Run team selection algorithm (using the hybrid strategy as recommended)
        team_indices, selection_history = self.dream_team_hybrid_strategy(affinity_df, guaranteed_top_n=2, max_team_size=max_team_size)

        # Generate coverage analysis
        coverage_report = self.generate_coverage_report(affinity_df, team_indices, skills_list)

        # Collect evidence
        team_evidence = self.get_team_evidence(affinity_df, team_indices, skills_list)


        # Generate strategic analysis
        strategic_analysis = self.generate_strategic_analysis(coverage_report, skills_list, solicitation_data)

        return DreamTeamReport(
            team_members=coverage_report['team_members'],
            overall_coverage_score=coverage_report['overall_coverage_score'],
            skill_analysis=coverage_report['skill_analysis'],
            strategic_analysis=strategic_analysis,
            selection_history=selection_history,
            generated_at=datetime.now().isoformat()
        ), team_evidence

    def format_markdown_report(self, dream_team_report, solicitation_title, team_evidence):
        """Format the strategic report as a human-readable Markdown file."""
        report = f"# NSF Dream Team Strategic Report\n\n"
        report += f"**Solicitation:** {solicitation_title}\n"
        report += f"**Generated:** {dream_team_report.generated_at}\n"
        report += f"**Overall Team Coverage Score:** **`{dream_team_report.overall_coverage_score:.2f} / 100`**\n\n"

        # Team Summary Table
        report += f"## 🏆 Recommended Dream Team\n\n"
        report += "| Role | Researcher | Avg. Affinity | Top Expertise Areas |\n"
        report += "|:---|:---|:---:|:---|\n"
        for i, member in enumerate(dream_team_report.team_members):
            role = "**Principal Investigator (PI)**" if i == 0 else f"Co-Investigator {i}"
            top_skills = ", ".join([s['skill'] for s in member['top_skills']])
            report += f"| {role} | {member['name']} | `{member['avg_affinity']:.2f}` | {top_skills} |\n"

        # Coverage Analysis Table
        report += f"\n## 📊 Skills Coverage Analysis\n\n"
        report += "| Skill / Expertise Area | Coverage | Level | Primary Expert | Supporting Evidence |\n" # Added Evidence column
        report += "|:---|:---:|:---|:---|:---|\n" # Updated table format
        for skill in sorted(dream_team_report.skill_analysis, key=lambda x: x['coverage_score'], reverse=True):
            level_emoji = "🟢" if skill['level'] == 'High' else "🟡" if skill['level'] == 'Medium' else "🔴"
            evidence_links = []
            if skill['skill'] in team_evidence and team_evidence[skill['skill']]:
                 evidence_links = [f"[Paper {j+1}]({paper_id})" for j, paper_id in enumerate(team_evidence[skill['skill']])]

            report += f"| {skill['skill']} | `{skill['coverage_score']:.2f}` | {level_emoji} {skill['level']} | {skill['expert']} | {', '.join(evidence_links) if evidence_links else 'None'} |\n" # Added evidence links


        # Strategic Analysis
        report += f"\n## 🧠 AI-Powered Strategic Analysis (via Groq)\n\n"
        report += dream_team_report.strategic_analysis + "\n"

        # Selection History
        report += f"\n## 🎯 Team Selection Process\n\n"
        for entry in dream_team_report.selection_history:
            report += f"**Step {entry['step']}:** {entry['action']} - {entry['researcher_name']}\n"
            report += f"- Reason: {entry['reason']}\n"
            report += f"- Team Coverage: {entry['team_coverage']:.2f}\n\n"

        return report

    def run_complete_analysis(self, solicitation_obj, top_n_researchers=20, max_team_size=4):
        """Run the complete analysis pipeline: matching → affinity matrix → dream team → report."""
        print("🚀 STARTING COMPLETE UNIFIED ANALYSIS PIPELINE")
        print("=" * 70)

        # Phase 1: Researcher Matching
        print("\n📍 PHASE 1: RESEARCHER MATCHING & RANKING")
        matching_results = self.search_researchers(solicitation_obj)

        # Display top matches
        print("\n🏆 TOP 10 RESEARCHER MATCHES:")
        print(f"{'Rank':<4} {'Name':<30} {'Final':<8} {'Academic':<9} {'Sparse':<8} {'Dense':<8} {'Papers':<7}")
        print("-" * 80)
        for i, match in enumerate(matching_results.top_matches[:10], 1):
            print(f"{i:<4} {match.researcher_name[:29]:<30} "
                  f"{match.final_affinity_score:<8.1f} {match.academic_expertise_score:<9.1f} "
                  f"{match.s_sparse:<8.1f} {match.s_dense:<8.1f} {match.total_papers:<7}")

        # Phase 2: Affinity Matrix Creation
        print(f"\n📍 PHASE 2: AFFINITY MATRIX CREATION")
        affinity_df = self.create_affinity_matrix(matching_results, top_n_researchers)

        # Phase 3: Dream Team Assembly
        print(f"\n📍 PHASE 3: DREAM TEAM ASSEMBLY & STRATEGIC ANALYSIS")
        skills_list = matching_results.skills_analyzed
        solicitation_data = {'title': solicitation_obj.title}
        dream_team_report, team_evidence = self.create_dream_team_report(affinity_df, skills_list, solicitation_data, max_team_size)


        # Phase 4: Generate Reports
        print(f"\n📍 PHASE 4: REPORT GENERATION")
        markdown_report = self.format_markdown_report(dream_team_report, solicitation_obj.title, team_evidence)

        # Save all results
        timestamp = int(time.time())
        base_filename = f"unified_analysis_{timestamp}"

        # Save matching results
        matching_file = f"{output_dir}/{base_filename}_matching_results.json"
        with open(matching_file, 'w', encoding='utf-8') as f:
            json.dump(asdict(matching_results), f, indent=2, ensure_ascii=False, default=str)
        print(f"💾 Saved matching results: {matching_file}")

        # Save affinity matrix
        affinity_file = f"{output_dir}/{base_filename}_affinity_matrix.csv"
        affinity_df.to_csv(affinity_file)
        print(f"💾 Saved affinity matrix: {affinity_file}")

        # Save dream team report
        report_file = f"{output_dir}/{base_filename}_dream_team_report.json"
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump(asdict(dream_team_report), f, indent=2, ensure_ascii=False, default=str)
        print(f"💾 Saved dream team report: {report_file}")

        # Save markdown report
        markdown_file = f"{output_dir}/{base_filename}_strategic_report.md"
        with open(markdown_file, 'w', encoding='utf-8') as f:
            f.write(markdown_report)
        print(f"💾 Saved markdown report: {markdown_file}")

        # Save evidence data
        evidence_file = f"{output_dir}/{base_filename}_team_evidence.json"
        with open(evidence_file, 'w', encoding='utf-8') as f:
            json.dump(team_evidence, f, indent=2, ensure_ascii=False, default=str)
        print(f"💾 Saved team evidence: {evidence_file}")

        # Display final summary
        self.display_final_summary(matching_results, dream_team_report, team_evidence)

        return {
            'matching_results': matching_results,
            'affinity_matrix': affinity_df,
            'dream_team_report': dream_team_report,
            'team_evidence': team_evidence,
            'markdown_report': markdown_report,
            'files_saved': {
                'matching': matching_file,
                'affinity': affinity_file,
                'report': report_file,
                'markdown': markdown_file,
                'evidence': evidence_file
            }
        }

    def display_final_summary(self, matching_results, dream_team_report, team_evidence):
        """Display a comprehensive summary of the analysis."""
        print("\n" + "="*70)
        print("📋 UNIFIED ANALYSIS SUMMARY")
        print("="*70)

        print(f"🔍 RESEARCHER MATCHING:")
        print(f"   • Analyzed {matching_results.total_researchers} total researchers")
        print(f"   • {matching_results.eligible_researchers} eligible researchers")
        print(f"   • Processing time: {matching_results.processing_time_seconds}s")

        print(f"\n🎯 DREAM TEAM ASSEMBLY:")
        print(f"   • Selected {len(dream_team_report.team_members)} optimal team members")
        print(f"   • Strategy: Hybrid (Top 2 performers + coverage optimization)")
        print(f"   • Overall coverage score: {dream_team_report.overall_coverage_score:.2f}/100")

        print(f"\n👥 RECOMMENDED TEAM:")
        for i, member in enumerate(dream_team_report.team_members):
            role = "PI" if i == 0 else f"Co-I {i}"
            print(f"   • {member['name']} ({role}) - Affinity: {member['avg_affinity']:.2f}")

        # Check for gaps
        low_skills = [s for s in dream_team_report.skill_analysis if s['level'] == 'Low']
        medium_skills = [s for s in dream_team_report.skill_analysis if s['level'] == 'Medium']
        high_skills = [s for s in dream_team_report.skill_analysis if s['level'] == 'High']

        print(f"\n📊 SKILL COVERAGE BREAKDOWN:")
        print(f"   🟢 High Coverage: {len(high_skills)} skills")
        print(f"   🟡 Medium Coverage: {len(medium_skills)} skills")
        print(f"   🔴 Low Coverage: {len(low_skills)} skills")

        if low_skills:
            print(f"\n🔴 TOP GAPS TO ADDRESS:")
            for skill in sorted(low_skills, key=lambda x: x['coverage_score'])[:3]:
                print(f"   • {skill['skill']} (Coverage: {skill['coverage_score']:.1f})")
            if len(low_skills) > 3:
                print(f"   • ... and {len(low_skills) - 3} more (see full report)")
        else:
            print(f"\n🟢 EXCELLENT COVERAGE: No significant skill gaps identified")

        print(f"\n✅ ANALYSIS COMPLETE! Check saved files for detailed reports.")
        print(f"📚 EVIDENCE: {sum(len(evidence) for evidence in team_evidence.values())} supporting papers collected")
        print(f"💡 TIP: Review the markdown report for AI-powered strategic recommendations and evidence.")

# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

def run_unified_analysis():
    """Main function to run the complete unified analysis."""

    # Load solicitation
    print("📋 Loading solicitation...")
    with open(solicitation_file, 'r', encoding='utf-8') as f:
        solicitation_data = json.load(f)

    # Convert to object
    class SolicitationObj:
        def __init__(self, data):
            for key, value in data.items():
                setattr(self, key, value)

    solicitation_obj = SolicitationObj(solicitation_data)
    print(f"✅ Loaded: {solicitation_obj.title}")

    # Initialize unified system
    system = UnifiedResearcherSystem(researcher_data_dir)

    # Run complete analysis
    results = system.run_complete_analysis(
        solicitation_obj,
        top_n_researchers=20,  # Adjust as needed
        max_team_size=4       # Adjust as needed
    )

    return results

# RUN THE UNIFIED SYSTEM
if __name__ == "__main__":
    results = run_unified_analysis()